In [1]:
%config IPCompleter.use_jedi = False
%pdb off
# %load_ext viztracer
# from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
import sys
from typing import Dict, List, Tuple, Optional
from pathlib import Path

# required to enable non-blocking interaction:
%gui qt5

from copy import deepcopy
from numba import jit
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
# pd.options.mode.dtype_backend = 'pyarrow' # use new pyarrow backend instead of numpy
from attrs import define, field, fields, Factory
import tables as tb
from datetime import datetime, timedelta

# Pho's Formatting Preferences
from pyphocorehelpers.preferences_helpers import set_pho_preferences, set_pho_preferences_concise, set_pho_preferences_verbose
set_pho_preferences_concise()

## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager

# NeuroPy (Diba Lab Python Repo) Loading
# from neuropy import core
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.core.epoch import NamedTimerange, Epoch
from neuropy.core.ratemap import Ratemap
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.utils.matplotlib_helpers import matplotlib_file_only, matplotlib_configuration, matplotlib_configuration_update
from neuropy.core.neuron_identities import NeuronIdentityTable, neuronTypesList, neuronTypesEnum
from neuropy.utils.mixins.AttrsClassHelpers import AttrsBasedClassHelperMixin, serialized_field, serialized_attribute_field, non_serialized_field, custom_define
from neuropy.utils.mixins.HDF5_representable import HDF_DeserializationMixin, post_deserialize, HDF_SerializationMixin, HDFMixin, HDF_Converter

## For computation parameters:
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.utils.dynamic_container import DynamicContainer
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from neuropy.core.neurons import NeuronType
from neuropy.core.user_annotations import UserAnnotationsManager
from neuropy.core.position import Position
from neuropy.core.session.dataSession import DataSession
from neuropy.analyses.time_dependent_placefields import PfND_TimeDependent, PlacefieldSnapshot

from pyphocorehelpers.print_helpers import print_object_memory_usage, print_dataframe_memory_usage, print_value_overview_only, DocumentationFilePrinter, print_keys_if_possible, generate_html_string, CapturedException

# pyPhoPlaceCellAnalysis:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import NeuropyPipeline # get_neuron_identities
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import export_pyqtgraph_plot
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_load_session, batch_extended_computations, batch_extended_programmatic_figures
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap
import pyphoplacecellanalysis.External.pyqtgraph as pg

from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_perform_all_plots
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import _find_any_context_neurons
from pyphoplacecellanalysis.General.Batch.runBatch import BatchSessionCompletionHandler # for `post_compute_validate(...)`
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import BasePositionDecoder
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsResults
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends # for `_perform_long_short_instantaneous_spike_rate_groups_analysis`
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import SingleBarResult, InstantaneousSpikeRateGroupsComputation, TruncationCheckingResults # for `BatchSessionCompletionHandler`, `AcrossSessionsAggregator`
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import SplitPartitionMembership

# Plotting
# import pylustrator # customization of figures
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
_bak_rcParams = mpl.rcParams.copy()
# import pylustrator # call `pylustrator.start()` before creating your first figure in code.
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import plot_multiple_raster_plot, plot_raster_plot
from pyphoplacecellanalysis.General.Mixins.DataSeriesColorHelpers import DataSeriesColorHelpers
from pyphoplacecellanalysis.General.Mixins.DataSeriesColorHelpers import UnitColoringMode, DataSeriesColorHelpers
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import _build_default_tick, build_scatter_plot_kwargs
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.Render2DScrollWindowPlot import Render2DScrollWindowPlotMixin, ScatterItemData
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_extended_programmatic_figures, batch_programmatic_figures
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends

from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import PAPER_FIGURE_figure_1_add_replay_epoch_rasters, PAPER_FIGURE_figure_1_full, PAPER_FIGURE_figure_3, main_complete_figure_generations

from pyphoplacecellanalysis.SpecificResults.fourthYearPresentation import *

# Jupyter Widget Interactive
from IPython.display import display
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager
from pyphoplacecellanalysis.GUI.IPyWidgets.pipeline_ipywidgets import interactive_pipeline_widget, fullwidth_path_widget, interactive_pipeline_files

global_data_root_parent_path = find_first_extant_path([Path(r'W:\Data'), Path(r'/home/halechr/FastData'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data'), Path(r'/home/halechr/cloud/turbo/Data')])
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"

Automatic pdb calling has been turned OFF
build_module_logger(module_name="Spike3D.pipeline"):
	 Module logger com.PhoHale.Spike3D.pipeline has file logging enabled and will log to EXTERNAL\TESTING\Logging\debug_com.PhoHale.Spike3D.pipeline.log


# Load Pipeline

In [14]:
%pdb off
# ==================================================================================================================== #
# Load Data                                                                                                            #
# ==================================================================================================================== #

active_data_mode_name = 'kdiba'
local_session_root_parent_context = IdentifyingContext(format_name=active_data_mode_name) # , animal_name='', configuration_name='one', session_name=a_sess.session_name
local_session_root_parent_path = global_data_root_parent_path.joinpath('KDIBA')

# [*] - indicates bad or session with a problem
# 0, 1, 2, 3, 4, 5, 6, 7, [8], [9], 10, 11, [12], 13, 14, [15], [16], 17, 
# curr_context: IdentifyingContext = good_contexts_list[1] # select the session from all of the good sessions here.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15') # DONE. Very good. Many good Pfs, many good replays.
curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43') # DONE, might be the BEST SESSION, good example session with lots of place cells, clean replays, and clear bar graphs.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31') # DONE, Good Pfs but no good replays
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6') # BAD, 2023-07-14, unsure why still.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19') # DONE, GREAT, both good Pfs and replays!
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25') # DONE, Added replay selections. Very "jumpy" between the starts and ends of the track.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40') # DONE, Added replay selections. A TON of putative replays in general, most bad, but some good.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46') # DONE, added replay selections. Very few (like 12) replays each.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-13_15-22-3') # DONE, Good Pfs, no good epochs.
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30') # DONE, okay replays (selected)
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50') # DONE, very few replays (selected)
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_13-34-40') # BAD
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54') # DONE, one replay each (selected)
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3') # BAD, Good Pfs strangely despite horrible map, no good epochs
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_15-25-59') # BAD, No Epochs
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_18-47-52')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-17_12-52-15')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_13-20-55')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_12-38-13')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44') # DONE, good. Many good pfs, many good replays. Noticed very strange jumping off the track in the 3D behavior/spikes viewer. Is there something wrong with this session?
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0') # DONE, good?, replays selected, few
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25') # DONE, very few replays
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_12-15-3')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_22-4-5')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54') # DONE, replays selected, quite a few replays but few are very good.


# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25')

local_session_parent_path: Path = local_session_root_parent_path.joinpath(curr_context.animal, curr_context.exper_name) # 'gor01', 'one' - probably not needed anymore
basedir: Path = local_session_parent_path.joinpath(curr_context.session_name).resolve()
print(f'basedir: {str(basedir)}')

# # Read if possible:
saving_mode = PipelineSavingScheme.SKIP_SAVING
force_reload = False

# # Force write:
# # saving_mode = PipelineSavingScheme.TEMP_THEN_OVERWRITE
# saving_mode = PipelineSavingScheme.OVERWRITE_IN_PLACE
# force_reload = True

## TODO: if loading is not possible, we need to change the `saving_mode` so that the new results are properly saved.

# ==================================================================================================================== #
# Load Pipeline                                                                                                        #
# ==================================================================================================================== #
# with VizTracer(output_file=f"viztracer_{get_now_time_str()}-full_session_LOO_decoding_analysis.json", min_duration=200, tracer_entries=3000000, ignore_frozen=True) as tracer:
# epoch_name_includelist = ['maze']
epoch_name_includelist = None
active_computation_functions_name_includelist=['pf_computation', 'pfdt_computation', 
                                                'firing_rate_trends',
                                                # 'pf_dt_sequential_surprise', 
                                            #    'ratemap_peaks_prominence2d',
                                                'position_decoding', 
                                                # 'position_decoding_two_step', 
                                            #    'long_short_decoding_analyses', 'jonathan_firing_rate_analysis', 'long_short_fr_indicies_analyses', 'short_long_pf_overlap_analyses', 'long_short_post_decoding', 'long_short_rate_remapping',
                                            #     'long_short_inst_spike_rate_groups',
                                            #     'long_short_endcap_analysis'
]

curr_active_pipeline: NeuropyPipeline = batch_load_session(global_data_root_parent_path, active_data_mode_name, basedir, epoch_name_includelist=epoch_name_includelist,
                                        computation_functions_name_includelist=active_computation_functions_name_includelist,
                                        saving_mode=saving_mode, force_reload=force_reload,
                                        skip_extended_batch_computations=True, debug_print=False, fail_on_exception=True) # , active_pickle_filename = 'loadedSessPickle_withParameters.pkl'

if not force_reload: # not just force_reload, needs to recompute whenever the computation fails.
    try:
        curr_active_pipeline.load_pickled_global_computation_results()
    except Exception as e:
        exception_info = sys.exc_info()
        e = CapturedException(e, exception_info)
        print(f'cannot load global results: {e}')
        raise


# 6m 1.1s
# 12m 15.6s


# try:
## Post Compute Validate 2023-05-16:
was_updated = BatchSessionCompletionHandler.post_compute_validate(curr_active_pipeline) ## TODO: need to potentially re-save if was_updated
if was_updated:
    try:
        curr_active_pipeline.save_pipeline(saving_mode=saving_mode)
    except Exception as e:
        ## TODO: catch/log saving error and indicate that it isn't saved.
        exception_info = sys.exc_info()
        e = CapturedException(e, exception_info)
        print(f'ERROR RE-SAVING PIPELINE after update. error: {e}')


curr_active_pipeline.reload_default_computation_functions()

extended_computations_include_includelist=['pf_computation', 'pfdt_computation', 'firing_rate_trends',
    'pf_dt_sequential_surprise',
    #  'ratemap_peaks_prominence2d',
    'long_short_decoding_analyses', 'jonathan_firing_rate_analysis', 'long_short_fr_indicies_analyses', 'short_long_pf_overlap_analyses', 'long_short_post_decoding', 'long_short_rate_remapping',
    'long_short_inst_spike_rate_groups',
    'long_short_endcap_analysis',
    'spike_burst_detection'
] # do only specified


force_recompute_global = force_reload
# force_recompute_global = True
newly_computed_values = batch_extended_computations(curr_active_pipeline, include_includelist=extended_computations_include_includelist, include_global_functions=True, fail_on_exception=True, progress_print=True, force_recompute=force_recompute_global, debug_print=False)
if (len(newly_computed_values) > 0):
    print(f'newly_computed_values: {newly_computed_values}.')
    if (saving_mode.value != 'skip_saving'):
        print(f'Saving global results...')
        try:
            # curr_active_pipeline.global_computation_results.persist_time = datetime.now()
            # Try to write out the global computation function results:
            curr_active_pipeline.save_global_computation_results()
        except Exception as e:
            exception_info = sys.exc_info()
            e = CapturedException(e, exception_info)
            print(f'\n\n!!WARNING!!: saving the global results threw the exception: {e}')
            print(f'\tthe global results are currently unsaved! proceed with caution and save as soon as you can!\n\n\n')
    else:
        print(f'\n\n!!WARNING!!: changes to global results have been made but they will not be saved since saving_mode.value == "skip_saving"')
        print(f'\tthe global results are currently unsaved! proceed with caution and save as soon as you can!\n\n\n')
else:
    print(f'no changes in global results.')

# except Exception as e:
#     exception_info = sys.exc_info()
#     e = CapturedException(e, exception_info)
#     print(f'second half threw: {e}')


# 4m 5.2s for inst fr computations
# Jupyter Widget Interactive
import ipywidgets as widgets
from IPython.display import display
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager
from pyphoplacecellanalysis.GUI.IPyWidgets.pipeline_ipywidgets import interactive_pipeline_widget, fullwidth_path_widget, interactive_pipeline_files

_pipeline_jupyter_widget = interactive_pipeline_widget(curr_active_pipeline=curr_active_pipeline)
# display(_pipeline_jupyter_widget)
_pipeline_jupyter_widget

[autoreload of pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis failed: Traceback (most recent call last):
  File "c:\Users\pho\repos\Spike3DWorkEnv\Spike3D\.venv\lib\site-packages\IPython\extensions\autoreload.py", line 274, in check
    superreload(m, reload, self.old_objects, self.shell)
  File "c:\Users\pho\repos\Spike3DWorkEnv\Spike3D\.venv\lib\site-packages\IPython\extensions\autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "c:\Users\pho\repos\Spike3DWorkEnv\Spike3D\.venv\lib\site-packages\IPython\extensions\autoreload.py", line 397, in update_generic
    update(a, b)
  File "c:\Users\pho\repos\Spike3DWorkEnv\Spike3D\.venv\lib\site-packages\IPython\extensions\autoreload.py", line 365, in update_class
    update_instances(old, new)
  File "c:\Users\pho\repos\Spike3DWorkEnv\Spike3D\.venv\lib\site-packages\IPython\extensions\autoreload.py", line 323, in update_instances
    object.__setattr__(ref, "__class__", new)

Automatic pdb calling has been turned OFF
basedir: W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43
Loading loaded session pickle file results : W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\loadedSessPickle.pkl... 

INFO:com.PhoHale.Spike3D.pipeline:NeuropyPipeline.__setstate__(state="{'pipeline_name': 'kdiba_pipeline', 'session_data_type': 'kdiba', '_stage': DisplayPipelineStage(stage_name='kdiba_pipeline_input', identity=<PipelineStage.Displayed: 4>)}")
INFO:com.PhoHale.Spike3D.pipeline:select_filters(...) with: []
INFO:com.PhoHale.Spike3D.pipeline:Performing perform_action_for_all_contexts with action EvaluationActions.EVALUATE_COMPUTATIONS on filtered_session with filter named "maze1"...
INFO:com.PhoHale.Spike3D.pipeline:WARNING: skipping computation because overwrite_extant_results=False and active_computation_results[maze1] already exists and is non-None
INFO:com.PhoHale.Spike3D.pipeline:	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
INFO:com.PhoHale.Spike3D.pipeline:Performing perform_action_for_all_contexts with action EvaluationActions.EVALUATE_COMPUTATIONS on filtered_session with f

done.
Loading pickled pipeline success: W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\loadedSessPickle.pkl.
properties already present in pickled version. No need to save.
pipeline load success!
using provided computation_functions_name_includelist: ['pf_computation', 'pfdt_computation', 'firing_rate_trends', 'position_decoding']
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
saving_mode.shouldSave == False, so not saving at the end of batch_load_session
Loading loaded session pickle file results : W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\output\global_computation_results.pkl... do

In [15]:
if curr_active_pipeline.updated_since_last_pickle:
	_bak_saving_mode = saving_mode

saving_mode = PipelineSavingScheme.TEMP_THEN_OVERWRITE
try:
	curr_active_pipeline.save_pipeline(saving_mode=saving_mode)
	curr_active_pipeline.save_global_computation_results()

except Exception as e:
	## TODO: catch/log saving error and indicate that it isn't saved.
	exception_info = sys.exc_info()
	e = CapturedException(e, exception_info)
	print(f'ERROR RE-SAVING PIPELINE after update. error: {e}')
finally:
	saving_mode = _bak_saving_mode


[autoreload of pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing failed: Traceback (most recent call last):
  File "c:\Users\pho\repos\Spike3DWorkEnv\Spike3D\.venv\lib\site-packages\IPython\extensions\autoreload.py", line 274, in check
    superreload(m, reload, self.old_objects, self.shell)
  File "c:\Users\pho\repos\Spike3DWorkEnv\Spike3D\.venv\lib\site-packages\IPython\extensions\autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "c:\Users\pho\repos\Spike3DWorkEnv\Spike3D\.venv\lib\site-packages\IPython\extensions\autoreload.py", line 397, in update_generic
    update(a, b)
  File "c:\Users\pho\repos\Spike3DWorkEnv\Spike3D\.venv\lib\site-packages\IPython\extensions\autoreload.py", line 365, in update_class
    update_instances(old, new)
  File "c:\Users\pho\repos\Spike3DWorkEnv\Spike3D\.venv\lib\site-packages\IPython\extensions\autoreload.py", line 323, in update_instances
    object.__setattr__(ref, "__class__", new)
TypeError: can't ap

finalized_loaded_sess_pickle_path: W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\loadedSessPickle.pkl
Saving (file mode 'None') saved session pickle file results : None... pipeline does not have "display_output"
pipeline does not have "render_actions"


INFO:com.PhoHale.Spike3D.pipeline:moving new output at 'W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\20231016110927-loadedSessPickle.pkl' -> to desired location: 'W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\loadedSessPickle.pkl'


done.
moving new output at 'W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\20231016110927-loadedSessPickle.pkl' -> to desired location: 'W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\loadedSessPickle.pkl'


INFO:com.PhoHale.Spike3D.pipeline:	 save complete.


global_computation_results_pickle_path: W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\output\global_computation_results.pkl
Saving (file mode 'W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\output\global_computation_results.pkl') saved session pickle file results : W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\output\global_computation_results.pkl... 	moving new output at 'W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\output\20231016111011-global_computation_results.pkltmp' -> to desired location: 'W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\output\global_computation_results.pkl'
done.


In [ ]:
curr_active_pipeline.clear_display_outputs()	

In [ ]:
from neuropy.core.neuron_identities import NeuronIdentityDataframeAccessor
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionTables
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import build_merged_neuron_firing_rate_indicies

BATCH_DATE_TO_USE = '2023-10-10'
curr_session_context = curr_active_pipeline.get_session_context()
output_file_prefix = curr_session_context.get_description(separator="-", include_property_names=False)
# global_replays.filename = Path(f"output/{output_file_prefix}_global_replays").resolve()
# print(f'global_replays.filename: {global_replays.filename}')
# global_replays.to_neuroscope()

# global_PBEs.filename = Path(f"output/{output_file_prefix}_global_PBEs").resolve()
# print(f'global_PBEs.filename: {global_PBEs.filename}')
# global_PBEs.to_neuroscope('PBE')

joined_neruon_fri_df = build_merged_neuron_firing_rate_indicies(curr_active_pipeline, enable_display_intermediate_results=False)
AcrossSessionTables.write_table_to_files(joined_neruon_fri_df, global_data_root_parent_path=global_data_root_parent_path, output_basename=f'{BATCH_DATE_TO_USE}_{output_file_prefix}_joined_neruon_fri_df')

In [ ]:
joined_neruon_fri_df = AcrossSessionTables.load_table_from_file(global_data_root_parent_path=global_data_root_parent_path, output_basename=f'{BATCH_DATE_TO_USE}_{output_file_prefix}_joined_neruon_fri_df')
joined_neruon_fri_df

In [ ]:
curr_active_pipeline.export_pipeline_to_h5()

In [ ]:
out_man = curr_active_pipeline.get_output_manager()
# out_man.get_figure_output_parent_path(


In [ ]:
from pyphocorehelpers.Filesystem.metadata_helpers import FilesystemMetadata

hdf5_output_path: Path = curr_active_pipeline.get_output_path().joinpath('pipeline_results.h5').resolve()
print(f'pipeline hdf5_output_path: {hdf5_output_path}')


# Only get files newer than date
newest_file_to_overwrite_date = datetime.now() - timedelta(days=1) # don't overwrite any files more recent than 1 day ago
if hdf5_output_path.exists() and (FilesystemMetadata.get_last_modified_time(hdf5_output_path)<=newest_file_to_overwrite_date):
	# file is folder than the date to overwrite, so overwrite it
	print(f'OVERWRITING (or writing) the file {hdf5_output_path}!')
	curr_active_pipeline.export_pipeline_to_h5()
else:
	print(f'file {hdf5_output_path} is newer than the allowed overwrite date, so it will be skipped.')


# End Run

In [3]:
## long_short_decoding_analyses:
curr_long_short_decoding_analyses = curr_active_pipeline.global_computation_results.computed_data['long_short_leave_one_out_decoding_analysis']
long_one_step_decoder_1D, short_one_step_decoder_1D, long_replays, short_replays, global_replays, long_shared_aclus_only_decoder, short_shared_aclus_only_decoder, shared_aclus, long_short_pf_neurons_diff, n_neurons, long_results_obj, short_results_obj, is_global = curr_long_short_decoding_analyses.long_decoder, curr_long_short_decoding_analyses.short_decoder, curr_long_short_decoding_analyses.long_replays, curr_long_short_decoding_analyses.short_replays, curr_long_short_decoding_analyses.global_replays, curr_long_short_decoding_analyses.long_shared_aclus_only_decoder, curr_long_short_decoding_analyses.short_shared_aclus_only_decoder, curr_long_short_decoding_analyses.shared_aclus, curr_long_short_decoding_analyses.long_short_pf_neurons_diff, curr_long_short_decoding_analyses.n_neurons, curr_long_short_decoding_analyses.long_results_obj, curr_long_short_decoding_analyses.short_results_obj, curr_long_short_decoding_analyses.is_global 

# (long_one_step_decoder_1D, short_one_step_decoder_1D), (long_one_step_decoder_2D, short_one_step_decoder_2D) = compute_short_long_constrained_decoders(curr_active_pipeline, recalculate_anyway=True)
long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
long_epoch_context, short_epoch_context, global_epoch_context = [curr_active_pipeline.filtered_contexts[a_name] for a_name in (long_epoch_name, short_epoch_name, global_epoch_name)]
long_epoch_obj, short_epoch_obj = [Epoch(curr_active_pipeline.sess.epochs.to_dataframe().epochs.label_slice(an_epoch_name)) for an_epoch_name in [long_epoch_name, short_epoch_name]]
long_session, short_session, global_session = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_results, short_results, global_results = [curr_active_pipeline.computation_results[an_epoch_name]['computed_data'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_computation_config, short_computation_config, global_computation_config = [curr_active_pipeline.computation_results[an_epoch_name]['computation_config'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_pf1D, short_pf1D, global_pf1D = long_results.pf1D, short_results.pf1D, global_results.pf1D
long_pf2D, short_pf2D, global_pf2D = long_results.pf2D, short_results.pf2D, global_results.pf2D
decoding_time_bin_size = long_one_step_decoder_1D.time_bin_size # 1.0/30.0 # 0.03333333333333333

## Get global `long_short_fr_indicies_analysis`:
long_short_fr_indicies_analysis_results = curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis']
long_laps, long_replays, short_laps, short_replays, global_laps, global_replays = [long_short_fr_indicies_analysis_results[k] for k in ['long_laps', 'long_replays', 'short_laps', 'short_replays', 'global_laps', 'global_replays']]
long_short_fr_indicies_df = long_short_fr_indicies_analysis_results['long_short_fr_indicies_df']

## Get global 'long_short_post_decoding' results:
curr_long_short_post_decoding = curr_active_pipeline.global_computation_results.computed_data['long_short_post_decoding']
expected_v_observed_result, curr_long_short_rr = curr_long_short_post_decoding.expected_v_observed_result, curr_long_short_post_decoding.rate_remapping
rate_remapping_df, high_remapping_cells_only = curr_long_short_rr.rr_df, curr_long_short_rr.high_only_rr_df
Flat_epoch_time_bins_mean, Flat_decoder_time_bin_centers, num_neurons, num_timebins_in_epoch, num_total_flat_timebins, is_short_track_epoch, is_long_track_epoch, short_short_diff, long_long_diff = expected_v_observed_result.Flat_epoch_time_bins_mean, expected_v_observed_result.Flat_decoder_time_bin_centers, expected_v_observed_result.num_neurons, expected_v_observed_result.num_timebins_in_epoch, expected_v_observed_result.num_total_flat_timebins, expected_v_observed_result.is_short_track_epoch, expected_v_observed_result.is_long_track_epoch, expected_v_observed_result.short_short_diff, expected_v_observed_result.long_long_diff

jonathan_firing_rate_analysis_result: JonathanFiringRateAnalysisResult = curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis
(epochs_df_L, epochs_df_S), (filter_epoch_spikes_df_L, filter_epoch_spikes_df_S), (good_example_epoch_indicies_L, good_example_epoch_indicies_S), (short_exclusive, long_exclusive, BOTH_subset, EITHER_subset, XOR_subset, NEITHER_subset), new_all_aclus_sort_indicies, assigning_epochs_obj = PAPER_FIGURE_figure_1_add_replay_epoch_rasters(curr_active_pipeline)
neuron_replay_stats_df, short_exclusive, long_exclusive, BOTH_subset, EITHER_subset, XOR_subset, NEITHER_subset = jonathan_firing_rate_analysis_result.get_cell_track_partitions(frs_index_inclusion_magnitude=0.05)

## Update long_exclusive/short_exclusive properties with `long_short_fr_indicies_df`
# long_exclusive.refine_exclusivity_by_inst_frs_index(long_short_fr_indicies_df, frs_index_inclusion_magnitude=0.5)
# short_exclusive.refine_exclusivity_by_inst_frs_index(long_short_fr_indicies_df, frs_index_inclusion_magnitude=0.5)

# inst_spike_rate_groups_result: InstantaneousSpikeRateGroupsComputation = curr_active_pipeline.global_computation_results.computed_data.long_short_inst_spike_rate_groups
# custom_InstSpikeRateTrends_df = inst_spike_rate_groups_result.all_incl_endPlatforms_InstSpikeRateTrends_df


DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computation_config
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computation_config
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computation_config
WARN: 2023-09-28 16:15: - [ ] fix the combination properties. Would work if we directly used the computed _is_L_only and _is_S_only above
WARN: 2023-09-28 16:15: - [ ] fix the combination properties. Would work if we directly used the computed _is_L_only and _is_S_

In [18]:
active_burst_intervals = curr_active_pipeline.computation_results[global_epoch_name].computed_data['burst_detection']['burst_intervals']
active_burst_intervals

IndexedOrderedDict([(2,     burst_level      t_start        t_end   t_duration                               interval_pair
0             0     0.321515  1731.204408  1730.882892    (0.32151535525918007, 1730.882892348105)
1             1     0.321515     1.733437     1.411921   (0.32151535525918007, 1.4119211971992627)
2           1.0    46.022284    46.395163     0.372879    (46.022284276550636, 0.3728791690664366)
3           1.0    58.234676    62.401811     4.167135      (58.23467610380612, 4.167135146446526)
4           2.0    58.234676    59.038434     0.803758     (58.23467610380612, 0.8037576684728265)
5           3.0    58.234676    58.553949     0.319273     (58.23467610380612, 0.3192727965069935)
..          ...          ...          ...          ...                                         ...
429         4.0   1652.96411   1653.00635      0.04224  (1652.9641098423162, 0.042239978443831205)
430         5.0   1652.96411   1653.00635      0.04224  (1652.9641098423162, 0.042239

In [4]:
# Relative Entropy/Surprise Results:
active_extended_stats = global_results['extended_stats']
active_relative_entropy_results = active_extended_stats['pf_dt_sequential_surprise'] # DynamicParameters
historical_snapshots = active_relative_entropy_results['historical_snapshots']
post_update_times: np.ndarray = active_relative_entropy_results['post_update_times'] # (4152,) = (n_post_update_times,)
snapshot_differences_result_dict = active_relative_entropy_results['snapshot_differences_result_dict']
time_intervals: np.ndarray = active_relative_entropy_results['time_intervals']
surprise_time_bin_duration = (post_update_times[2]-post_update_times[1])
long_short_rel_entr_curves_frames: np.ndarray = active_relative_entropy_results['long_short_rel_entr_curves_frames'] # (4152, 108, 63) = (n_post_update_times, n_neurons, n_xbins)
short_long_rel_entr_curves_frames: np.ndarray = active_relative_entropy_results['short_long_rel_entr_curves_frames'] # (4152, 108, 63) = (n_post_update_times, n_neurons, n_xbins)
flat_relative_entropy_results: np.ndarray = active_relative_entropy_results['flat_relative_entropy_results'] # (149, 63) - (nSnapshots, nXbins)
flat_jensen_shannon_distance_results: np.ndarray = active_relative_entropy_results['flat_jensen_shannon_distance_results'] # (149, 63) - (nSnapshots, nXbins)
flat_jensen_shannon_distance_across_all_positions: np.ndarray = np.sum(np.abs(flat_jensen_shannon_distance_results), axis=1) # sum across all position bins # (4152,) - (nSnapshots)
flat_surprise_across_all_positions: np.ndarray = np.sum(np.abs(flat_relative_entropy_results), axis=1) # sum across all position bins # (4152,) - (nSnapshots)

## Get the placefield dt matrix:
if 'snapshot_occupancy_weighted_tuning_maps' not in active_relative_entropy_results:
	## Compute it if missing:
	occupancy_weighted_tuning_maps_over_time = np.stack([placefield_snapshot.occupancy_weighted_tuning_maps_matrix for placefield_snapshot in historical_snapshots.values()])
	active_relative_entropy_results['snapshot_occupancy_weighted_tuning_maps'] = occupancy_weighted_tuning_maps_over_time
else:
	occupancy_weighted_tuning_maps_over_time = active_relative_entropy_results['snapshot_occupancy_weighted_tuning_maps'] # (n_post_update_times, n_neurons, n_xbins)


In [5]:
# Time-dependent
long_pf1D_dt, short_pf1D_dt, global_pf1D_dt = long_results.pf1D_dt, short_results.pf1D_dt, global_results.pf1D_dt
long_pf2D_dt, short_pf2D_dt, global_pf2D_dt = long_results.pf2D_dt, short_results.pf2D_dt, global_results.pf2D_dt
global_pf1D_dt: PfND_TimeDependent = global_results.pf1D_dt
global_pf2D_dt: PfND_TimeDependent = global_results.pf2D_dt

In [6]:
## long_short_endcap_analysis: checks for cells localized to the endcaps that have their placefields truncated after shortening the track
truncation_checking_result: TruncationCheckingResults = curr_active_pipeline.global_computation_results.computed_data.long_short_endcap
disappearing_endcap_aclus = truncation_checking_result.disappearing_endcap_aclus
# disappearing_endcap_aclus
trivially_remapping_endcap_aclus = truncation_checking_result.minor_remapping_endcap_aclus
# trivially_remapping_endcap_aclus
significant_distant_remapping_endcap_aclus = truncation_checking_result.significant_distant_remapping_endcap_aclus
# significant_distant_remapping_endcap_aclus
appearing_aclus = jonathan_firing_rate_analysis_result.neuron_replay_stats_df[jonathan_firing_rate_analysis_result.neuron_replay_stats_df['track_membership'] == SplitPartitionMembership.RIGHT_ONLY].index
# appearing_aclus

In [ ]:
significant_distant_remapping_endcap_aclus

In [ ]:
from pyphoplacecellanalysis.SpecificResults.fourthYearPresentation import fig_remapping_cells

graphics_output_dict = fig_remapping_cells(curr_active_pipeline=curr_active_pipeline)

## 2023-10-11 - Plot Peak Position on the Long Track vs. LapFRI to see where the remapping occurs:


In [ ]:
# 2023-11-10 - Adds "LxC_PBEsDeltaMinus" for PBEs 


temp = add_extra_spike_rate_trends(curr_active_pipeline)
temp


## 2023-10-04 - Compare the decoded result of each replay epoch between the long/short decoder to determine which is better for that Epoch:

In [ ]:
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import DecodedFilterEpochsResult

# neuron_replay_stats_df.reset_index()
# neuron_replay_stats_df.index = neuron_replay_stats_df['aclu']

a_long_decoder_result: DecodedFilterEpochsResult = long_results_obj.all_included_filter_epochs_decoder_result
a_short_decoder_result: DecodedFilterEpochsResult = short_results_obj.all_included_filter_epochs_decoder_result


# filter_epochs_decoder_result
# curr_results_obj.active_filter_epochs, curr_results_obj.all_included_filter_epochs_decoder_result, xbin=curr_results_obj.original_1D_decoder.xbin


# image_layer_dict = {}
# layer_properties_dict = {
# 	# 'snapshot_occupancy_weighted_tuning_maps': dict(blending='additive', colormap='viridis', name='pf1D_dt'),
# #  'flat_jensen_shannon_distance_results': dict(blending='additive', colormap='gray'),
# 	'long_p_x_given_n': dict(blending='additive', colormap='bop blue'),
# 	'short_p_x_given_n': dict(blending='additive', colormap='red'),
	
# }

# for a_name, layer_properties in layer_properties_dict.items():
# 	# image_layer_dict[a_name] = viewer.add_image(active_relative_entropy_results_xr_dict[a_name].to_numpy().astype(float), name=a_name)
# 	image_layer_dict[a_name] = viewer.add_image(active_relative_entropy_results[a_name].astype(float), **(dict(name=a_name)|layer_properties))

# assert viewer.dims.ndim == 3
# ## Set the dimensions appropriately
# viewer.dims.axis_labels = ('t', 'neuron_id', 'xbin')



# fig, ax = plt.subplots(1,1)
for decoded_epoch_idx in np.arange(a_long_decoder_result.num_filter_epochs):

	if decoded_epoch_idx < 5:
		# decoded_epoch_idx:int = 0
		curr_epoch_time_bin_container = a_long_decoder_result.time_bin_containers[decoded_epoch_idx]
		curr_time_bins = curr_epoch_time_bin_container.centers

		## Long Decoding:
		curr_long_epoch_p_x_given_n = a_long_decoder_result.p_x_given_n_list[decoded_epoch_idx] # .shape: (239, 5) - (n_x_bins, n_epoch_time_bins)
		## Need to exclude estimates from bins that didn't have any spikes in them (in general these glitch around):
		# curr_total_spike_counts_per_window = np.sum(a_decoder_result.spkcount[decoded_epoch_idx], axis=0) # left_out_decoder_result.spkcount[i].shape # (69, 222) - (nCells, nTimeWindowCenters)
		# curr_is_time_bin_non_firing = (curr_total_spike_counts_per_window == 0) # this would mean that no cells fired in this time bin
		curr_long_most_likely_position_indicies = np.squeeze(a_long_decoder_result.most_likely_position_indicies_list[decoded_epoch_idx]) # (n_epoch_time_bins, ) one position for each time bin in the replay
		# curr_most_likely_positions = a_decoder_result.most_likely_positions_list[decoded_epoch_idx]
		curr_long_most_likely_position_confidence = np.max(curr_long_epoch_p_x_given_n, axis=0) # (n_epoch_time_bins, )


		## Short Decoding:
		curr_short_epoch_p_x_given_n = a_short_decoder_result.p_x_given_n_list[decoded_epoch_idx] # .shape: (239, 5) - (n_x_bins, n_epoch_time_bins)
		## Need to exclude estimates from bins that didn't have any spikes in them (in general these glitch around):
		# curr_total_spike_counts_per_window = np.sum(a_decoder_result.spkcount[decoded_epoch_idx], axis=0) # left_out_decoder_result.spkcount[i].shape # (69, 222) - (nCells, nTimeWindowCenters)
		# curr_is_time_bin_non_firing = (curr_total_spike_counts_per_window == 0) # this would mean that no cells fired in this time bin
		curr_short_most_likely_position_indicies = np.squeeze(a_short_decoder_result.most_likely_position_indicies_list[decoded_epoch_idx]) # (n_epoch_time_bins, ) one position for each time bin in the replay
		curr_short_most_likely_position_confidence = np.max(curr_short_epoch_p_x_given_n, axis=0) # (n_epoch_time_bins, )

		# _long_short_decoding_comparison_df = pd.DataFrame({'t': curr_time_bins,
		# 	'x_bin_long': curr_long_most_likely_position_indicies, 'x_bin_long_confidence': curr_long_most_likely_position_confidence,
		# 	'x_bin_short': curr_short_most_likely_position_indicies, 'x_bin_short_confidence': curr_short_most_likely_position_confidence}) # , 'x': curr_most_likely_positions

		viewer.add_image(curr_long_epoch_p_x_given_n.astype(float), **(dict(name=f'long_p_x_given_n[{decoded_epoch_idx}]', blending='additive', colormap='red')))
		viewer.add_image(curr_short_epoch_p_x_given_n.astype(float), **(dict(name=f'short_p_x_given_n[{decoded_epoch_idx}]', blending='additive', colormap='bop blue')))

	# np.shape(curr_long_epoch_p_x_given_n)
	# _long_short_decoding_comparison_df.plot(y=['x_bin_long', 'x_bin_short'], color=['r','g'], ax=ax)

# _long_short_decoding_comparison_df

In [ ]:
np.max([np.shape(a_long_decoder_result.p_x_given_n_list[decoded_epoch_idx]) for decoded_epoch_idx in np.arange(a_long_decoder_result.num_filter_epochs)], axis=0)
# np.max([np.shape(a_short_decoder_result.p_x_given_n_list[decoded_epoch_idx]) for decoded_epoch_idx in np.arange(a_short_decoder_result.num_filter_epochs)], axis=0)


In [ ]:
a_long_decoder_result.p_x_given_n_list.shape

In [ ]:
# a_decoder_result.p_x_given_n_list[decoded_epoch_idx]
# a_decoder_result.most_likely_position_indicies_list


# curr_most_likely_positions

_df.plot()

In [ ]:
curr_time_bins.shape
curr_most_likely_positions.shape
curr_most_likely_position_indicies.shape

In [ ]:
a_decoder_result


In [ ]:
for decoded_epoch_idx in np.arange(a_decoder_result.num_filter_epochs):
	curr_epoch_time_bin_container = a_decoder_result.time_bin_containers[decoded_epoch_idx]
	curr_cell_decoded_epoch_time_bins.append(curr_epoch_time_bin_container)
	curr_time_bins = curr_epoch_time_bin_container.centers
	curr_epoch_p_x_given_n = a_decoder_result.p_x_given_n_list[decoded_epoch_idx] # .shape: (239, 5) - (n_x_bins, n_epoch_time_bins)
	assert curr_epoch_p_x_given_n.shape[0] == curr_cell_pf_curve.shape[0]
	
	## Need to exclude estimates from bins that didn't have any spikes in them (in general these glitch around):
	curr_total_spike_counts_per_window = np.sum(a_decoder_result.spkcount[decoded_epoch_idx], axis=0) # left_out_decoder_result.spkcount[i].shape # (69, 222) - (nCells, nTimeWindowCenters)
	curr_is_time_bin_non_firing = (curr_total_spike_counts_per_window == 0) # this would mean that no cells fired in this time bin
	curr_most_likely_position_indicies = a_decoder_result.most_likely_position_indicies_list[decoded_epoch_idx] # (n_epoch_time_bins, ) one position for each time bin in the replay
	
	# From the firing map of the placefields for this neuron (`decoder_1D.F.T[left_out_neuron_IDX]`) get the value for each position bin index in the epoch
	curr_epoch_expected_fr = np.squeeze(a_decoder_1D.F.T[left_out_neuron_IDX][curr_most_likely_position_indicies])
	# expected_num_spikes = curr_epoch_expected_fr * decoder_result.decoding_time_bin_size

	# Eqn 1:
	# p_n_given_x = lambda n: (1.0/factorial(n)) * pow(expected_num_spikes, n) * np.exp(-expected_num_spikes) # likelihood function
	
	all_cells_decoded_expected_firing_rates[left_out_aclu].append(curr_epoch_expected_fr)

In [ ]:
from neuropy.core.neuron_identities import NeuronIdentityDataframeAccessor

neuron_replay_stats_df = neuron_replay_stats_df.neuron_identity.make_neuron_indexed_df_global(curr_active_pipeline.get_session_context(), add_expanded_session_context_keys=False, add_extended_aclu_identity_columns=False)
neuron_replay_stats_df

In [ ]:
neuron_replay_stats_df.neuron_uid

In [ ]:
from neuropy.core.user_annotations import UserAnnotationsManager, SessionCellExclusivityRecord
from neuropy.utils.result_context import IdentifyingContext



# for a_ctx, a_val in annotation_man.get_hardcoded_specific_session_override_dict().items():
# 	annotation_man.annotations[a_ctx] = a_val

# for a_ctx, a_val in UserAnnotationsManager.get_user_annotations().items():
# 	annotation_man.annotations[a_ctx] = a_val

# for a_ctx, a_val in session_cell_exclusivity_annotations.items():
# 	# Not ideal. Adds a key 'session_cell_exclusivity' to the extant session context instead of being indexable by an entirely new context
# 	annotation_man.annotations[a_ctx] = annotation_man.annotations.get(a_ctx, {}) | dict(session_cell_exclusivity=a_val)
# 	# annotation_man.annotations[a_ctx.overwriting_context(user_annotation='session_cell_exclusivity')] = a_val

annotation_man = UserAnnotationsManager()
session_cell_exclusivity: SessionCellExclusivityRecord = annotation_man.annotations[curr_active_pipeline.get_session_context()].get('session_cell_exclusivity', None)
session_cell_exclusivity.LxC
session_cell_exclusivity.SxC


In [ ]:
## 2023-10-03 - My metric comes from the number of laps with an active long place vs. number of laps with an active short place
# curr_active_pipeline.sess.laps
# active_long_spikes_df = active_spikes_df[active_spikes_df.is_included_long_pf1D]
# active_short_spikes_df = active_spikes_df[active_spikes_df.is_included_short_pf1D]

# active_spikes_df: pd.DataFrame = long_session.spikes_df.copy()
active_spikes_df: pd.DataFrame = global_session.spikes_df.copy()
# active_spikes_df[active_spikes_df['lap'] != -1]

In [ ]:
from neuropy.core.neuron_identities import NeuronExtendedIdentityTuple, NeuronType


def pho_long_short_exclusivity_index(active_spikes_df: pd.DataFrame) -> pd.DataFrame:
	""" 2023-10-03 - Long/Short Exclusivity Metric based off of Pho's intuition. Looks at the total number of spikes over laps. Pletny of room for improvement. 

	"""
	# active_spikes_df['neuron_type'] = active_spikes_df.map(

	# Drop rows with missing data in columns: 'lin_pos', 't' and 8 other columns
	active_spikes_df = active_spikes_df.dropna(subset=['lin_pos', 't', 't_seconds', 't_rel_seconds', 'x', 'y', 'aclu', 'maze_id', 'lap', 'maze_relative_lap'])
	# Filter rows based on columns: 'lap', 'maze_id'
	active_spikes_df = active_spikes_df[(active_spikes_df['lap'] != -1) & (active_spikes_df['maze_id'] != -1)]

	# Convert the 'neuron_type' column of the dataframe to the categorical type if needed
	cat_type = NeuronType.get_pandas_categories_type()
	if active_spikes_df["neuron_type"].dtype != cat_type:
		# If this type check ever becomes a problem and we want a more liberal constraint, All instances of CategoricalDtype compare equal to the string 'category'.
		active_spikes_df["neuron_type"] = active_spikes_df["neuron_type"].apply(lambda x: x.hdfcodingClassName).astype(cat_type) # NeuronType can't seem to be cast directly to the new categorical type, it results in the column being filled with NaNs. Instead cast to string first.

	active_spikes_df = active_spikes_df.astype({'maze_id': 'category', 'lap': 'category', 'maze_relative_lap': 'category', 'aclu': 'category'})
	# display(active_spikes_df)

	# Performed 5 aggregations grouped on columns: 'aclu', 'maze_id', 'lap'
	active_spikes_df_agg = active_spikes_df.groupby(['aclu', 'maze_id', 'maze_relative_lap']).agg(t_count=('t', 'count'), lin_pos_mean=('lin_pos', 'mean'), lin_pos_std=('lin_pos', 'std'), lin_pos_min=('lin_pos', 'min'), lin_pos_max=('lin_pos', 'max')).reset_index()

	curr_unique_aclus = active_spikes_df_agg['aclu'].unique()
	curr_unique_maze_relative_laps = active_spikes_df_agg['maze_relative_lap'].unique()
	n_total_entries: int = len(active_spikes_df_agg['maze_relative_lap'].unique()) * len(active_spikes_df_agg['aclu'].unique())

	# Static lists of columns:
	relevant_columns_list = ['aclu', 'maze_relative_lap', 't_count', 'lin_pos_mean', 'lin_pos_std', 'lin_pos_min', 'lin_pos_max']
	relevant_index_column_names = ['aclu', 'maze_relative_lap']
	numerical_column_names = ['t_count', 'lin_pos_mean', 'lin_pos_std', 'lin_pos_min', 'lin_pos_max']

	long_maze_df = active_spikes_df_agg[active_spikes_df_agg['maze_id'] == 1][relevant_columns_list].reset_index()
	short_maze_df = active_spikes_df_agg[active_spikes_df_agg['maze_id'] == 2][relevant_columns_list].reset_index() #.set_index(['aclu', 'maze_relative_lap'])

	n_laps = len(active_spikes_df_agg['maze_relative_lap'].unique())
	n_long_laps = len(long_maze_df['maze_relative_lap'].unique()) 
	n_short_laps = len(short_maze_df['maze_relative_lap'].unique()) 

	n_min_laps = min(n_long_laps, n_short_laps)
	safe_lap_indicies = np.arange(n_min_laps) + 1
	if n_long_laps > n_min_laps:
		# truncate the long laps
		long_maze_df = long_maze_df[np.isin(long_maze_df['maze_relative_lap'], safe_lap_indicies)]
	if n_short_laps > n_min_laps:
		# truncate the short laps
		short_maze_df = short_maze_df[np.isin(short_maze_df['maze_relative_lap'], safe_lap_indicies)]

	## Compute the difference between the two
	long_short_maze_diff_df = long_maze_df[numerical_column_names] - short_maze_df[numerical_column_names]
	long_short_maze_diff_df[relevant_index_column_names] = long_maze_df[relevant_index_column_names].copy()
	# long_short_maze_diff_df = long_short_maze_diff_df.set_index(relevant_index_column_names)
	long_short_maze_diff_df = long_short_maze_diff_df.groupby(['aclu']).agg(t_count_sum=('t_count', 'sum'), t_count_mean=('t_count', 'mean')).reset_index() # Aggregate across all columns to get a value for each 'aclu'
	# , t_count_max=('t_count', 'max')
	# long_short_maze_diff_df.set_index('aclu')
	# Sort by column: 't_count_sum' (ascending)
	long_short_maze_diff_df = long_short_maze_diff_df.sort_values(['t_count_sum'], na_position='first')
	return long_short_maze_diff_df


active_spikes_df: pd.DataFrame = global_session.spikes_df.copy()
long_short_maze_diff_df = pho_long_short_exclusivity_index(active_spikes_df=active_spikes_df)
long_short_maze_diff_df

In [ ]:
from pyphocorehelpers.indexing_helpers import partition, safe_pandas_get_group
from pyphocorehelpers.indexing_helpers import partition_df

unique_aclu_values, aclu_group_dfs = partition_df(long_short_maze_diff_df, 'aclu')
n_aclus = len(unique_aclu_values)

fig, axes = plt.subplots(nrows=n_aclus, ncols=1, sharex=True, sharey=False)

for i, (aclu, aclu_df) in enumerate(zip(unique_aclu_values, aclu_group_dfs)):
	aclu_df.plot(x='maze_relative_lap', y='t_count', label=f'{aclu}', ax=axes[i])

In [ ]:
long_short_maze_diff_df = long_maze_df['t_count'].to_numpy() - short_maze_df['t_count'].to_numpy()
long_short_maze_diff_df.set_index()
long_short_maze_diff_df.shape

In [ ]:
active_spikes_df_agg.reindex([active_spikes_df_agg['aclu'].unique(), active_spikes_df_agg['maze_relative_lap'].unique()], fill_value=0)

In [ ]:
from pyphocorehelpers.indexing_helpers import partition, safe_pandas_get_group

unique_values, group_dfs = partition(active_spikes_df_agg, 'maze_id')

In [ ]:

unique_values, group_dfs = partition(active_spikes_df_agg, 'maze_id')
unique_values

In [ ]:
safe_pandas_get_group(active_spikes_df_agg, 

In [ ]:
group_dfs

In [ ]:
unique_values, group_dfs = partition(active_spikes_df, 'aclu')
unique_values

In [ ]:
for aclu 

In [ ]:
a_df = group_dfs[unique_values[0]]
# type(a_df)
a_df.shape # (4320, 8)



In [ ]:
active_spikes_df_agg[active_spikes_df_agg['maze_id'] == 1]

In [ ]:
active_spikes_df_agg['maze_id'].eq(1).astype(int).groupby(active_spikes_df_agg['aclu']).sum()

In [ ]:
# Performed 2 aggregations grouped on columns: 'aclu', 'maze_id', 'lap'
active_spikes_df = active_spikes_df.groupby(['aclu', 'maze_id', 'lap']).agg(flat_spike_idx_count=('flat_spike_idx', 'count'), lin_pos_var=('lin_pos', 'var')).reset_index()


In [ ]:
# Performed 5 aggregations grouped on columns: 'aclu', 'maze_id'
active_spikes_df = active_spikes_df.groupby(['aclu', 'maze_id']).agg(t_count=('t', 'count'), lin_pos_mean=('lin_pos', 'mean'), lin_pos_std=('lin_pos', 'std'), lin_pos_min=('lin_pos', 'min'), lin_pos_max=('lin_pos', 'max')).reset_index()

In [ ]:
'maze_id'==1

active_spikes_df[active_spikes_df['lap'] != -1]

In [ ]:
long_laps

In [ ]:
short_laps

In [ ]:
# column_rename_dict = {'neuron_type':'neuron_type'}
# input_df.rename(



# 2023-10-03 - Recompute `long_short_inst_spike_rate_groups`, fixing result for empty LxC/SxC

In [ ]:
curr_active_pipeline.global_computation_results.computation_config

In [ ]:
if curr_active_pipeline.global_computation_results.computation_config is None:
	global_computation_results.computation_config = DynamicContainer(instantaneous_time_bin_size_seconds=0.01)
else:
	# update the existing value
	curr_active_pipeline.global_computation_results.computation_config.instantaneous_time_bin_size_seconds = 0.01


In [ ]:
curr_active_pipeline.reload_default_computation_functions()
newly_computed_values = batch_extended_computations(curr_active_pipeline, include_includelist=['long_short_inst_spike_rate_groups'], include_global_functions=True, fail_on_exception=True, progress_print=True, force_recompute=True, debug_print=False)
newly_computed_values



In [ ]:
curr_active_pipeline.save_global_computation_results()

In [ ]:
neuron_replay_stats_df

## 2023-09-29 - Plot the LxC/SxC PhoJonathanPlots

In [9]:
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_perform_all_plots, BatchPhoJonathanFiguresHelper

In [10]:
## all cells, sorted by custom_frs_index:
# fig_1c_figures_all_dict = BatchPhoJonathanFiguresHelper.run(curr_active_pipeline, neuron_replay_stats_df.sort_values('custom_frs_index', ascending=True, inplace=False), included_unit_neuron_IDs=None,
# 	n_max_page_rows=20, write_vector_format=False, write_png=False,
# 	show_only_refined_cells=False, disable_top_row=True, split_by_short_long_shared=False)

fig_1c_figures_all_dict = BatchPhoJonathanFiguresHelper.run(curr_active_pipeline, neuron_replay_stats_df, included_unit_neuron_IDs=None,
	n_max_page_rows=20, write_vector_format=False, write_png=True,
	show_only_refined_cells=False, disable_top_row=False, split_by_short_long_shared=False)


include_includelist: ['maze1', 'maze2', 'maze']
long_epoch_name: maze1, short_epoch_name: maze2, global_epoch_name: maze
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data


C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\General\Mixins\CrossComputationComparisonHelpers.py:235: RuntimeWarning: invalid value encountered in divide
  _long_to_short_balances = (rdf.num_short_only_neuron_participating.values - rdf.num_long_only_neuron_participating.values) / (rdf.num_short_only_neuron_participating.values + rdf.num_long_only_neuron_participating.values) ## Working but (0, 1, 1) would clip to 0.5 despite (1, 13, 0) going all the way down to -1.0
C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\General\Mixins\CrossComputationComparisonHelpers.py:265: RuntimeWarning: invalid value encountered in divide
  _long_to_short_balances = (rdf.num_short_only_neuron_participating.values - rdf.num_long_only_neuron_participating.values) / (rdf.num_short_only_neuron_participating.values + rdf.num_long_only_neuron_participating.values) ## Working but (0, 1, 1) would clip to 0.5 despite (1, 13, 0) going all t

	 saved C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\EXTERNAL\Screenshots\ProgrammaticDisplayFunctionTesting\2023-10-16\kdiba\gor01\one\2006-6-09_1-22-43\BatchPhoJonathanReplayFRC_shared_1of6_(25,70,47,33,71,87,43,15,51,34,40,2,56,65,84,89,44,103,54,10).png
include_includelist: ['maze1', 'maze2', 'maze']
long_epoch_name: maze1, short_epoch_name: maze2, global_epoch_name: maze
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data


C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\General\Mixins\CrossComputationComparisonHelpers.py:235: RuntimeWarning: invalid value encountered in divide
  _long_to_short_balances = (rdf.num_short_only_neuron_participating.values - rdf.num_long_only_neuron_participating.values) / (rdf.num_short_only_neuron_participating.values + rdf.num_long_only_neuron_participating.values) ## Working but (0, 1, 1) would clip to 0.5 despite (1, 13, 0) going all the way down to -1.0
C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\General\Mixins\CrossComputationComparisonHelpers.py:265: RuntimeWarning: invalid value encountered in divide
  _long_to_short_balances = (rdf.num_short_only_neuron_participating.values - rdf.num_long_only_neuron_participating.values) / (rdf.num_short_only_neuron_participating.values + rdf.num_long_only_neuron_participating.values) ## Working but (0, 1, 1) would clip to 0.5 despite (1, 13, 0) going all t

	 saved C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\EXTERNAL\Screenshots\ProgrammaticDisplayFunctionTesting\2023-10-16\kdiba\gor01\one\2006-6-09_1-22-43\BatchPhoJonathanReplayFRC_shared_2of6_(6,19,97,79,12,38,30,81,72,98,92,48,35,24,93,39,14,31,80,66).png
include_includelist: ['maze1', 'maze2', 'maze']
long_epoch_name: maze1, short_epoch_name: maze2, global_epoch_name: maze
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data


...

C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\General\Mixins\CrossComputationComparisonHelpers.py:235: RuntimeWarning: invalid value encountered in divide
  _long_to_short_balances = (rdf.num_short_only_neuron_participating.values - rdf.num_long_only_neuron_participating.values) / (rdf.num_short_only_neuron_participating.values + rdf.num_long_only_neuron_participating.values) ## Working but (0, 1, 1) would clip to 0.5 despite (1, 13, 0) going all the way down to -1.0
C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\General\Mixins\CrossComputationComparisonHelpers.py:265: RuntimeWarning: invalid value encountered in divide
  _long_to_short_balances = (rdf.num_short_only_neuron_participating.values - rdf.num_long_only_neuron_participating.values) / (rdf.num_short_only_neuron_participating.values + rdf.num_long_only_neuron_participating.values) ## Working but (0, 1, 1) would clip to 0.5 despite (1, 13, 0) going all t

	 saved C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\EXTERNAL\Screenshots\ProgrammaticDisplayFunctionTesting\2023-10-16\kdiba\gor01\one\2006-6-09_1-22-43\BatchPhoJonathanReplayFRC_shared_5of6_(17,21,22,32,36,37,41,42,45,46,49,50,55,62,64,69,73,76,88,94).png
include_includelist: ['maze1', 'maze2', 'maze']
long_epoch_name: maze1, short_epoch_name: maze2, global_epoch_name: maze
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data


C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\General\Mixins\CrossComputationComparisonHelpers.py:235: RuntimeWarning: invalid value encountered in divide
  _long_to_short_balances = (rdf.num_short_only_neuron_participating.values - rdf.num_long_only_neuron_participating.values) / (rdf.num_short_only_neuron_participating.values + rdf.num_long_only_neuron_participating.values) ## Working but (0, 1, 1) would clip to 0.5 despite (1, 13, 0) going all the way down to -1.0
C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\General\Mixins\CrossComputationComparisonHelpers.py:265: RuntimeWarning: invalid value encountered in divide
  _long_to_short_balances = (rdf.num_short_only_neuron_participating.values - rdf.num_long_only_neuron_participating.values) / (rdf.num_short_only_neuron_participating.values + rdf.num_long_only_neuron_participating.values) ## Working but (0, 1, 1) would clip to 0.5 despite (1, 13, 0) going all t

	 saved C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\EXTERNAL\Screenshots\ProgrammaticDisplayFunctionTesting\2023-10-16\kdiba\gor01\one\2006-6-09_1-22-43\BatchPhoJonathanReplayFRC_shared_6of6_(96,99,100).png


c:\Users\pho\repos\Spike3DWorkEnv\Spike3D\.venv\lib\site-packages\ipykernel\eventloops.py:128: UserWarning: constrained_layout not applied because axes sizes collapsed to zero.  Try making figure larger or axes decorations smaller.
  el.exec() if hasattr(el, 'exec') else el.exec_()


In [ ]:
fig_1c_figures_out_dict = BatchPhoJonathanFiguresHelper.run(curr_active_pipeline, neuron_replay_stats_df, included_unit_neuron_IDs=long_exclusive.get_refined_track_exclusive_aclus(), n_max_page_rows=20, write_vector_format=False, write_png=True, show_only_refined_cells=True, disable_top_row=True)
fig_1c_figures_out_dict = BatchPhoJonathanFiguresHelper.run(curr_active_pipeline, neuron_replay_stats_df, included_unit_neuron_IDs=short_exclusive.get_refined_track_exclusive_aclus(), n_max_page_rows=20, write_vector_format=False, write_png=True, show_only_refined_cells=True, disable_top_row=True)


In [ ]:
fig_1c_figures_out_dict = BatchPhoJonathanFiguresHelper.run(curr_active_pipeline, neuron_replay_stats_df, included_unit_neuron_IDs=XOR_subset.track_exclusive_aclus, n_max_page_rows=20, write_vector_format=False, write_png=save_figure, disable_top_row=True) # active_out_figures_dict: {IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-07_16-40-19', 'BatchPhoJonathanReplayFRC', 'long_only', '(12,21,48)')>: <Figure size 1920x660 with 12 Axes>, IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-07_16-40-19', 'BatchPhoJonathanReplayFRC', 'short_only', '(18,19,65)')>: <Figure size 1920x660 with 12 Axes>}


In [ ]:

fig_1c_figures_out_dict = BatchPhoJonathanFiguresHelper.run(curr_active_pipeline, neuron_replay_stats_df, included_unit_neuron_IDs=XOR_subset.get_refined_track_exclusive_aclus(), n_max_page_rows=20, write_vector_format=False, write_png=True, show_only_refined_cells=True, disable_top_row=True)

In [ ]:
neuron_replay_stats_df = jonathan_firing_rate_analysis_result.neuron_replay_stats_df.copy()

np.sum(np.logical_xor(neuron_replay_stats_df['is_refined_LxC'], neuron_replay_stats_df['is_refined_SxC']))

In [ ]:
np.sum(np.logical_and(neuron_replay_stats_df['is_refined_LxC'], neuron_replay_stats_df['is_refined_SxC']))

In [ ]:
# uses global's 'extended_stats' results for relative entropy (surprise) analyses:

import tables as tb


# {
# 't': Tuple[float, float],
# 'snapshots': Tuple[PlacefieldSnapshot, PlacefieldSnapshot],
# 'relative_entropy_result_dict': {
# 		'long_short_rel_entr_curve': np.array,
# 		...
# 		'jensen_shannon_distance': np.array,
# 	} # 5 items
# } # 3 items
# Dict[float, PlacefieldSnapshot]

def relative_entropy_to_h5(global_results, file_path='output/test_relative_entropy_numpy_arrays.h5'):
	print(f'relative_entropy_to_h5(...)')

	active_extended_stats = global_results['extended_stats']
	active_relative_entropy_results = active_extended_stats['pf_dt_sequential_surprise'] # DynamicParameters
	post_update_times: np.ndarray = active_relative_entropy_results['post_update_times'] # (4152,) = (n_post_update_times,)
	snapshot_differences_result_dict = active_relative_entropy_results['snapshot_differences_result_dict']
	time_intervals: np.ndarray = active_relative_entropy_results['time_intervals']
	long_short_rel_entr_curves_frames: np.ndarray = active_relative_entropy_results['long_short_rel_entr_curves_frames'] # (4152, 108, 63) = (n_post_update_times, n_neurons, n_xbins)
	short_long_rel_entr_curves_frames: np.ndarray = active_relative_entropy_results['short_long_rel_entr_curves_frames'] # (4152, 108, 63) = (n_post_update_times, n_neurons, n_xbins)
	flat_relative_entropy_results: np.ndarray = active_relative_entropy_results['flat_relative_entropy_results'] # (149, 63) - (nSnapshots, nXbins)
	flat_jensen_shannon_distance_results: np.ndarray = active_relative_entropy_results['flat_jensen_shannon_distance_results'] # (149, 63) - (nSnapshots, nXbins)
	flat_jensen_shannon_distance_across_all_positions: np.ndarray = np.sum(np.abs(flat_jensen_shannon_distance_results), axis=1) # sum across all position bins # (4152,) - (nSnapshots)
	flat_surprise_across_all_positions: np.ndarray = np.sum(np.abs(flat_relative_entropy_results), axis=1) # sum across all position bins # (4152,) - (nSnapshots)
	
	# Create an HDF5 file
	with tb.open_file(file_path, mode="w") as hdf5_file:
		print(f'trying to write to "{file_path}"...')
		# Create groups for organization
		root = hdf5_file.root
		group = hdf5_file.create_group(root, 'relative_entropy_results')
		
		# Store np.ndarrays in the HDF5 file
		hdf5_file.create_array(group, 'post_update_times', post_update_times)
		hdf5_file.create_array(group, 'time_intervals', time_intervals)
		hdf5_file.create_array(group, 'long_short_rel_entr_curves_frames', long_short_rel_entr_curves_frames)
		hdf5_file.create_array(group, 'short_long_rel_entr_curves_frames', short_long_rel_entr_curves_frames)
		hdf5_file.create_array(group, 'flat_relative_entropy_results', flat_relative_entropy_results)
		hdf5_file.create_array(group, 'flat_jensen_shannon_distance_results', flat_jensen_shannon_distance_results)
		hdf5_file.create_array(group, 'flat_jensen_shannon_distance_across_all_positions', flat_jensen_shannon_distance_across_all_positions)
		hdf5_file.create_array(group, 'flat_surprise_across_all_positions', flat_surprise_across_all_positions)
	
	print(f'\t done!')
	

relative_entropy_to_h5(global_results, file_path='output/test_relative_entropy_numpy_arrays.h5')

In [ ]:
curr_active_pipeline.export_pipeline_to_h5()


In [ ]:
snapshot_differences_result_dict

flat_jensen_shannon_distance_results

In [ ]:
active_relative_entropy_results
# type(active_relative_entropy_results)

In [ ]:
neuron_replay_stats_df: pd.DataFrame = jonathan_firing_rate_analysis_result.neuron_replay_stats_df
neuron_replay_stats_df


In [ ]:
# Test classifying various x-positions as belonging to outside the outside_maze, the track_endcaps, or the track_body
from enum import Enum

class TrackPositionClassification(Enum):
    OUTSIDE_MAZE = "outside_maze"
    TRACK_ENDCAPS = "track_endcaps"
    TRACK_BODY = "track_body"


neuron_replay_stats_df: pd.DataFrame = jonathan_firing_rate_analysis_result.neuron_replay_stats_df
short_pf_peak_x = neuron_replay_stats_df.short_pf_peak_x
long_pf_peak_x = neuron_replay_stats_df.long_pf_peak_x

In [ ]:
rate_remapping_df[np.isin(rate_remapping_df.index, significant_distant_remapping_endcap_aclus)]

In [ ]:
inst_spike_rate_groups_result: InstantaneousSpikeRateGroupsComputation = curr_active_pipeline.global_computation_results.computed_data.long_short_inst_spike_rate_groups
# custom_InstSpikeRateTrends_df = inst_spike_rate_groups_result.all_incl_endPlatforms_InstSpikeRateTrends_df
# if not hasattr(inst_spike_rate_groups_result, 'all_incl_endPlatforms_InstSpikeRateTrends_df'):
# 	inst_spike_rate_groups_result.all_incl_endPlatforms_InstSpikeRateTrends_df = pd.DataFrame()

In [ ]:
# 2023-09-14 - Find cells outside the bounds of the short track
# Modifies the `jonathan_firing_rate_analysis_result.neuron_replay_stats_df` in-place instead of creating a copy:
# neuron_replay_stats_df = deepcopy(jonathan_firing_rate_analysis_result.neuron_replay_stats_df)
neuron_replay_stats_df: pd.DataFrame = jonathan_firing_rate_analysis_result.neuron_replay_stats_df
# Extract the peaks of the long placefields to find ones that have peaks outside the boundaries
# long_pf_peaks = neuron_replay_stats_df[neuron_replay_stats_df['has_long_pf']]['long_pf_peak_x'] - 150.0 # this shift of 150.0 is to center the midpoint of the track at 0. 
# # display(long_pf_peaks)
# is_left_cap = (long_pf_peaks < -72.0)
# is_right_cap = (long_pf_peaks > 72.0)
# is_either_cap =  np.logical_or(is_left_cap, is_right_cap)

# # Adds ['is_long_peak_left_cap', 'is_long_peak_right_cap', 'is_long_peak_either_cap'] columns: 
# neuron_replay_stats_df['is_long_peak_left_cap'] = False
# neuron_replay_stats_df['is_long_peak_right_cap'] = False
# neuron_replay_stats_df.loc[is_left_cap.index, 'is_long_peak_left_cap'] = is_left_cap # True
# neuron_replay_stats_df.loc[is_right_cap.index, 'is_long_peak_right_cap'] = is_right_cap # True
# neuron_replay_stats_df['is_long_peak_either_cap'] = np.logical_or(neuron_replay_stats_df['is_long_peak_left_cap'], neuron_replay_stats_df['is_long_peak_right_cap'])
# adds ['LS_pf_peak_x_diff'] column
# neuron_replay_stats_df['LS_pf_peak_x_diff'] = neuron_replay_stats_df['long_pf_peak_x'] - neuron_replay_stats_df['short_pf_peak_x']
neuron_replay_stats_df.is_long_peak_either_cap

## Extract just the endcap cells:
cap_cells_df = neuron_replay_stats_df[np.logical_and(neuron_replay_stats_df['has_long_pf'], neuron_replay_stats_df['is_long_peak_either_cap'])]
cap_aclus = cap_cells_df.index
num_total_endcap_cells = len(cap_aclus)
display(num_total_endcap_cells)

# "Disppearing" cells fall below the 1Hz firing criteria on the short track:
disappearing_endcap_cells_df = cap_cells_df[np.logical_not(cap_cells_df['has_short_pf'])]
disappearing_endcap_aclus = disappearing_endcap_cells_df.index
num_disappearing_endcap_cells = len(disappearing_endcap_aclus)
print(f'num_disappearing_endcap_cells/num_total_endcap_cells: {num_disappearing_endcap_cells}/{num_total_endcap_cells}')

non_disappearing_endcap_cells_df = cap_cells_df[cap_cells_df['has_short_pf']] # "non_disappearing" cells are those with a placefield on the short track as well
num_non_disappearing_endcap_cells = len(non_disappearing_endcap_cells_df)
print(f'num_non_disappearing_endcap_cells/num_total_endcap_cells: {num_non_disappearing_endcap_cells}/{num_total_endcap_cells}')

# display(non_disappearing_endcap_cells_df)
# non_disappearing_endcap_cells_df['LS_pf_peak_x_diff'] = non_disappearing_endcap_cells_df['long_pf_peak_x'] - non_disappearing_endcap_cells_df['short_pf_peak_x']
# display(non_disappearing_endcap_cells_df)

# Classify the non_disappearing cells into two groups:
# 1. Those that exhibit significant remapping onto somewhere else on the track
non_disappearing_endcap_cells_df['has_significant_distance_remapping'] = (np.abs(non_disappearing_endcap_cells_df['LS_pf_peak_x_diff']) >= 40.0) # The most a placefield could translate intwards would be (35 + (pf_width/2.0)) I think.
num_significant_position_remappping_endcap_cells = len(non_disappearing_endcap_cells_df[non_disappearing_endcap_cells_df['has_significant_distance_remapping'] == True])
print(f'num_significant_position_remappping_endcap_cells/num_non_disappearing_endcap_cells: {num_significant_position_remappping_endcap_cells}/{num_non_disappearing_endcap_cells}')

# 2. Those that seem to remain where they were on the long track, perhaps being "sampling-clipped" or translated adjacent to the platform. These two subcases can be distinguished by a change in the placefield's length (truncated cells would be a fraction of the length, although might need to account for scaling with new track length)
minorly_changed_endcap_cells_df = non_disappearing_endcap_cells_df[non_disappearing_endcap_cells_df['has_significant_distance_remapping'] == False]

non_disappearing_endcap_cells_df
# endcaps:
significant_distant_remapping_endcap_aclus = non_disappearing_endcap_cells_df[non_disappearing_endcap_cells_df['has_significant_distance_remapping']].index.to_numpy() # Int64Index([3, 5, 7, 11, 14, 38, 41, 53, 57, 61, 62, 75, 78, 79, 82, 83, 85, 95, 98, 100, 102], dtype='int64')
significant_distant_remapping_endcap_aclus
minor_remapping_endcap_aclus = minorly_changed_endcap_cells_df.index.to_numpy()
minor_remapping_endcap_aclus
jonathan_firing_rate_analysis_result.neuron_replay_stats_df

In [ ]:
significant_distant_remapping_endcap_df = jonathan_firing_rate_analysis_result.neuron_replay_stats_df[np.isin(jonathan_firing_rate_analysis_result.neuron_replay_stats_df.index, significant_distant_remapping_endcap_aclus)]
significant_distant_remapping_endcap_df

In [ ]:
# Will these remapped cells be included in replays after the delta?
num_short_replays = non_disappearing_endcap_cells_df['short_num_replays']
short_replay_mean_fr_Hz = non_disappearing_endcap_cells_df['replay_diff']


# 2023-10-11 - sanity check the Jensen-Shannon distance metric by computing for each placefield curve:
As a sanity check, compute the Jensen-Shannon Distance between each of the distributions for the long/short curve. Expected to see that non-trivially remapping cells had a greater JS-distance than the others.
### NO: It doesn't work, not even a little bit. Returns inf values for curves containing no infs

In [ ]:
jonathan_firing_rate_analysis_result: JonathanFiringRateAnalysisResult = curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis
# (epochs_df_L, epochs_df_S), (filter_epoch_spikes_df_L, filter_epoch_spikes_df_S), (good_example_epoch_indicies_L, good_example_epoch_indicies_S), (short_exclusive, long_exclusive, BOTH_subset, EITHER_subset, XOR_subset, NEITHER_subset), new_all_aclus_sort_indicies, assigning_epochs_obj = PAPER_FIGURE_figure_1_add_replay_epoch_rasters(curr_active_pipeline)

## long_short_endcap_analysis:
truncation_checking_result: TruncationCheckingResults = curr_active_pipeline.global_computation_results.computed_data.long_short_endcap
disappearing_endcap_aclus = truncation_checking_result.disappearing_endcap_aclus
trivially_remapping_endcap_aclus = truncation_checking_result.minor_remapping_endcap_aclus
significant_distant_remapping_endcap_aclus = truncation_checking_result.significant_distant_remapping_endcap_aclus
appearing_aclus = jonathan_firing_rate_analysis_result.neuron_replay_stats_df[jonathan_firing_rate_analysis_result.neuron_replay_stats_df['track_membership'] == SplitPartitionMembership.RIGHT_ONLY].index


In [ ]:
# long_pf1D.ratemap.unsmoothed_tuning_maps.shape

long_1DMaps = long_pf1D.get_by_id(trivially_remapping_endcap_aclus).ratemap.unsmoothed_tuning_maps # (15, 107)
short_1DMaps = short_pf1D.get_by_id(trivially_remapping_endcap_aclus).ratemap.unsmoothed_tuning_maps # .shape


In [ ]:
## NOTE: these results are NOT the same as the ones calculated in compute_snapshot_relative_entropy_surprise_differences.compute_surprise_relative_entropy_divergence, and I'm not quite sure why:
# Jensen-Shannon distance is an average of KL divergence:

from scipy.special import rel_entr
from scipy.stats import wasserstein_distance

# 2023-10-11 - Earth mover's distance
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment

# def compute_surprise_relative_entropy_divergence(long_curve, short_curve):
# 	""" Pre 2023-03-10 Refactoring:
# 	Given two tuning maps, computes the surprise (in terms of the KL-divergence a.k.a. relative entropy) between the two
# 	Returns a dictionary containing the results in both directions

# 	TODO 2023-03-08 02:41: - [ ] Convert naming convention from long_, short_ to lhs_, rhs_ to be general
# 	TODO 2023-03-08 02:47: - [ ] Convert output dict to a dataclass

# 	from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.ExtendedStats import compute_surprise_relative_entropy_divergence

# 	"""
# 	long_short_rel_entr_curve = rel_entr(long_curve, short_curve)
# 	long_short_relative_entropy = sum(long_short_rel_entr_curve) 
# 	short_long_rel_entr_curve = rel_entr(short_curve, long_curve)
# 	short_long_relative_entropy = sum(short_long_rel_entr_curve)
# 	# Jensen-Shannon distance is an average of KL divergence:
# 	mixture_distribution = 0.5 * (long_curve + short_curve)
# 	jensen_shannon_distance = 0.5 * (sum(rel_entr(mixture_distribution, long_curve)) + sum(rel_entr(mixture_distribution, short_curve))) # is this right? I'm confused by sum(...)

# 	return dict(long_short_rel_entr_curve=long_short_rel_entr_curve, long_short_relative_entropy=long_short_relative_entropy, short_long_rel_entr_curve=short_long_rel_entr_curve, short_long_relative_entropy=short_long_relative_entropy,
# 			jensen_shannon_distance=jensen_shannon_distance)



# def JS_Distance(long_short_rel_entr_curves_frames, short_long_rel_entr_curves_frames):
# 	""" seems incorrect. Returns a vector of Inf values for JS-Distance"""
# 	mixture_distribution = 0.5 * (long_short_rel_entr_curves_frames + short_long_rel_entr_curves_frames)
# 	print(f'mixture_distribution.shape: {np.shape(mixture_distribution)}') # (nSnapshots, n_neurons, n_xbins)
# 	# jensen_shannon_distance = 0.5 * (sum(rel_entr(mixture_distribution, long_short_rel_entr_curves_frames)) + sum(rel_entr(mixture_distribution, short_long_rel_entr_curves_frames))) # is this right? I'm confused by sum(...) # (n_neurons, n_xbins)
# 	jensen_shannon_distance = 0.5 * (np.sum(rel_entr(mixture_distribution, long_short_rel_entr_curves_frames), axis=1) + np.sum(rel_entr(mixture_distribution, short_long_rel_entr_curves_frames), axis=1)) # alt version: (nSnapshots, n_xbins)
# 	print(f'jensen_shannon_distance.shape: {np.shape(jensen_shannon_distance)}') # (n_neurons, n_xbins)
# 	return jensen_shannon_distance, mixture_distribution


def earth_movers_distance(long_1DMaps, short_1DMaps):
	n_cells = np.shape(short_1DMaps)[0]
	# return np.array([cdist(np.atleast_2d(np.squeeze(long_1DMaps[i,:])), np.atleast_2d(np.squeeze(short_1DMaps[i,:])), 'jensenshannon')[0,0] for i in np.arange(n_cells)])
	return np.array([wasserstein_distance(np.squeeze(long_1DMaps[i,:]), np.squeeze(short_1DMaps[i,:])) for i in np.arange(n_cells)])




# i = 0
# js_result_dict = compute_surprise_relative_entropy_divergence(np.squeeze(long_1DMaps[i,:]), np.squeeze(short_1DMaps[i,:]))
# js_result_dict
# js_result_dict['jensen_shannon_distance']
# js_result_dict['long_short_rel_entr_curve']

# jensen_shannon_distance, mixture_distribution = JS_Distance(long_1DMaps, short_1DMaps)
# jensen_shannon_distance

# earth_movers_distance = cdist(long_1DMaps, short_1DMaps, 'jensenshannon')

# n_cells = np.shape(short_1DMaps)[0]
# earth_movers_distance = np.array([cdist(np.atleast_2d(np.squeeze(long_1DMaps[i,:])), np.atleast_2d(np.squeeze(short_1DMaps[i,:])), 'jensenshannon')[0,0] for i in np.arange(n_cells)])
	


In [ ]:

# long_1DMaps = long_pf1D.get_by_id(trivially_remapping_endcap_aclus).ratemap.unsmoothed_tuning_maps # (15, 107)
# short_1DMaps = short_pf1D.get_by_id(trivially_remapping_endcap_aclus).ratemap.unsmoothed_tuning_maps # .shape

# appearing_earth_movers_distance = earth_movers_distance(long_pf1D.get_by_id(appearing_aclus).ratemap.unsmoothed_tuning_maps, short_pf1D.get_by_id(appearing_aclus).ratemap.unsmoothed_tuning_maps)

# get_curve_fn = lambda x: x.unsmoothed_tuning_maps
get_curve_fn = lambda x: x.unsmoothed_tuning_maps

trivially_remapping_endcap_earth_movers_distance = earth_movers_distance(get_curve_fn(long_pf1D.get_by_id(trivially_remapping_endcap_aclus).ratemap), get_curve_fn(short_pf1D.get_by_id(trivially_remapping_endcap_aclus).ratemap))
significant_distant_remapping_earth_movers_distance = earth_movers_distance(get_curve_fn(long_pf1D.get_by_id(significant_distant_remapping_endcap_aclus).ratemap), get_curve_fn(short_pf1D.get_by_id(significant_distant_remapping_endcap_aclus).ratemap))
disappearing_endcap_earth_movers_distance = earth_movers_distance(get_curve_fn(long_pf1D.get_by_id(disappearing_endcap_aclus).ratemap), get_curve_fn(short_pf1D.get_by_id(disappearing_endcap_aclus).ratemap))


# earth_movers_distance = cdist(np.atleast_2d(np.squeeze(long_1DMaps[i,:])), np.atleast_2d(np.squeeze(short_1DMaps[i,:])), 'jensenshannon')[0]
# trivially_remapping_endcap_earth_movers_distance
# earth_movers_distance.shape

trivially_remapping_endcap_earth_movers_distance

In [ ]:
significant_distant_remapping_earth_movers_distance


In [ ]:
np.mean(trivially_remapping_endcap_earth_movers_distance)



In [ ]:
np.mean(significant_distant_remapping_earth_movers_distance)
# fig, ax = plt.subplots()
# ax.scatter(


## 2023-09-20 - Plot the significantly remapping cells using the lower-level `plot_short_v_long_pf1D_comparison` function

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import plot_short_v_long_pf1D_comparison
from neuropy.utils.matplotlib_helpers import perform_update_title_subtitle
from pyphoplacecellanalysis.Pho2D.track_shape_drawing import add_vertical_track_bounds_lines, add_track_shapes
from neuropy.plotting.ratemaps import plot_ratemap_1D
from pyphoplacecellanalysis.General.Model.Configs.LongShortDisplayConfig import LongShortDisplayConfigManager

curr_active_pipeline.reload_default_display_functions()

graphics_output_dict = {}
active_context: IdentifyingContext = curr_active_pipeline.get_session_context()

long_short_display_config_manager = LongShortDisplayConfigManager()
long_epoch_config = long_short_display_config_manager.long_epoch_config.as_pyqtgraph_kwargs()
short_epoch_config = long_short_display_config_manager.short_epoch_config.as_pyqtgraph_kwargs()

long_epoch_matplotlib_config = long_short_display_config_manager.long_epoch_config.as_matplotlib_kwargs()
short_epoch_matplotlib_config = long_short_display_config_manager.short_epoch_config.as_matplotlib_kwargs()

shared_kwargs = dict(pad=1, cmap='hsv', active_context=curr_active_pipeline.get_session_context(), plot_zero_baselines=True, skip_figure_titles=True, use_flexitext_titles=True, flat_stack_mode=False)
top_level_shared_kwargs = dict(should_plot_vertical_track_bounds_lines=True, sortby='peak_long')
# top_level_shared_kwargs = dict(should_plot_vertical_track_bounds_lines=False, should_plot_linear_track_shapes=True) # Renders the linear track shape on the maze. Assumes `flat_stack_mode=True`


# # flat_stack_mode: all placefields are stacked up (z-wise) on top of each other on a single axis with no offsets:
# shared_kwargs = dict(pad=1, active_context=curr_active_pipeline.get_session_context(), plot_zero_baselines=True, skip_figure_titles=True, use_flexitext_titles=True, flat_stack_mode=True)
# top_level_shared_kwargs = dict(should_plot_vertical_track_bounds_lines=False, should_plot_linear_track_shapes=True) # Renders the linear track shape on the maze. Assumes `flat_stack_mode=True`

In [ ]:
# 2023-09-21 - Plot All
graphics_output_dict = graphics_output_dict | fig_remapping_cells(curr_active_pipeline)

In [ ]:
plt.suptitle('Translation Remapping Cells')

In [ ]:
fig_surprise_results

In [ ]:
curr_active_pipeline.active_sess_config

## 2023-10-11 - Refined PBE results


In [ ]:
long_short_fr_indicies_analysis_results = curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis']
long_short_fr_indicies_analysis_results



In [ ]:
# sess = curr_active_pipeline.sess
global_session.replay

In [ ]:
global_session.laps

In [ ]:
global_session.pbe #.epochs.to_epoch_obj()


In [ ]:
owning_pipeline_reference = curr_active_pipeline

non_running_periods = Epoch.from_PortionInterval(owning_pipeline_reference.sess.laps.as_epoch_obj().to_PortionInterval().complement())
non_replay_periods: Epoch = Epoch(Epoch.from_PortionInterval(owning_pipeline_reference.sess.replay.epochs.to_PortionInterval().complement()).time_slice(t_start=long_epoch_obj.t_start, t_stop=short_epoch_obj.t_stop).to_dataframe()[:-1]) #[:-1] # any period except the replay ones, drop the infinite last entry


# Split into long/short only periods:
long_only_non_replay_periods: Epoch  = non_replay_periods.time_slice(t_start=long_epoch_obj.t_start, t_stop=long_epoch_obj.t_stop) # any period except the replay ones
short_only_non_replay_periods: Epoch  = non_replay_periods.time_slice(t_start=short_epoch_obj.t_start, t_stop=short_epoch_obj.t_stop) # any period except the replay ones


In [ ]:
non_running_periods

## pre 2023-10-11 - Other

In [ ]:
# long_results = curr_active_pipeline.computation_results['maze1_PYR'].computed_data
# short_results = curr_active_pipeline.computation_results['maze2_PYR'].computed_data
# curr_any_context_neurons = _find_any_context_neurons(*[curr_active_pipeline.computation_results[k].computed_data.pf1D.ratemap.neuron_ids for k in ['maze1_PYR', 'maze2_PYR']])
# (fig_long_pf_1D, ax_long_pf_1D, long_sort_ind, long_neurons_colors_array), (fig_short_pf_1D, ax_short_pf_1D, short_sort_ind, short_neurons_colors_array) = plot_short_v_long_pf1D_comparison(long_results, short_results, disappearing_endcap_aclus, reuse_axs_tuple=None, single_figure=True, shared_kwargs=shared_kwargs, title_string="Disappearing Cells", subtitle_string=None, **top_level_shared_kwargs)
graphics_output_dict['disappearing_endcap_aclus'] = curr_active_pipeline.display('_display_short_long_pf1D_comparison', curr_active_pipeline.get_session_context(), included_any_context_neuron_ids=disappearing_endcap_aclus, reuse_axs_tuple=None, single_figure=True, shared_kwargs=shared_kwargs, title_string="Disappearing Cells", subtitle_string=None, **top_level_shared_kwargs)

In [ ]:
# (fig_long_pf_1D, ax_long_pf_1D, long_sort_ind, long_neurons_colors_array), (fig_short_pf_1D, ax_short_pf_1D, short_sort_ind, short_neurons_colors_array) = plot_short_v_long_pf1D_comparison(long_results, short_results, significant_distant_remapping_endcap_aclus, reuse_axs_tuple=None, single_figure=True, shared_kwargs=shared_kwargs, title_string="Significant Distance Remapping Cells", subtitle_string="1D Placefields", **top_level_shared_kwargs)
graphics_output_dict['significant_distant_remapping_endcap_aclus'] = curr_active_pipeline.display('_display_short_long_pf1D_comparison', curr_active_pipeline.get_session_context(), included_any_context_neuron_ids=significant_distant_remapping_endcap_aclus, reuse_axs_tuple=None, single_figure=True, shared_kwargs=shared_kwargs, title_string="Significant Distance Remapping Cells", subtitle_string="1D Placefields", **top_level_shared_kwargs)

In [ ]:
# (fig_long_pf_1D, ax_long_pf_1D, long_sort_ind, long_neurons_colors_array), (fig_short_pf_1D, ax_short_pf_1D, short_sort_ind, short_neurons_colors_array) = plot_short_v_long_pf1D_comparison(long_results, short_results, trivially_remapping_endcap_aclus, reuse_axs_tuple=None, single_figure=True, shared_kwargs=shared_kwargs, title_string="Trivially Remapping Cells", subtitle_string="1D Placefields", **top_level_shared_kwargs)
graphics_output_dict['trivially_remapping_endcap_aclus'] = curr_active_pipeline.display('_display_short_long_pf1D_comparison', curr_active_pipeline.get_session_context(), included_any_context_neuron_ids=trivially_remapping_endcap_aclus, reuse_axs_tuple=None, single_figure=True, shared_kwargs=shared_kwargs, title_string="Trivially Remapping Cells", subtitle_string="1D Placefields", **top_level_shared_kwargs)

In [ ]:
## 2023-09-18 - Shows the cells that exhibit simple remapping after delta
graphics_output_dict = curr_active_pipeline.display('_display_short_long_pf1D_comparison', curr_active_pipeline.get_session_context(), included_any_context_neuron_ids=None, reuse_axs_tuple=None, single_figure=True, shared_kwargs=shared_kwargs, **top_level_shared_kwargs)

In [ ]:
from pyphoplacecellanalysis.SpecificResults.fourthYearPresentation import fig_example_nontopo_remap

graphics_output_dict = fig_example_nontopo_remap(curr_active_pipeline)

In [ ]:
curr_active_pipeline.export_pipeline_to_h5('output/2023_09_26_new_pipeline_test.h5')
# ERROR: encountered exception !! 'InstantaneousSpikeRateGroupsComputation' object has no attribute 'all_incl_endPlatforms_InstSpikeRateTrends_df' ::::: (<class 'AttributeError'>, AttributeError("'InstantaneousSpikeRateGroupsComputation' object has no attribute 'all_incl_endPlatforms_InstSpikeRateTrends_df'"), <traceback object at 0x000001E5A3D3D040>) while trying to build the session HDF output.
# AttributeError: 'SpikeRateTrends' object has no attribute 'included_neuron_ids'
# _neuron_replay_stats_df - TypeError: Cannot serialize the column [track_membership] because its data contents are not [string] but [mixed] object dtype

In [ ]:
%matplotlib qt
active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'

graphics_output_dict = curr_active_pipeline.display('_display_short_long_pf1D_scalar_overlap_comparison', active_identifying_session_ctx, save_figure=False)
# fig, axs, plot_data = graphics_output_dict['fig'], graphics_output_dict['axs'], graphics_output_dict['plot_data']

In [ ]:
curr_active_pipeline.reload_default_display_functions()

In [ ]:
# _display_long_short_expected_v_observed_firing_rate
graphics_output_dict = curr_active_pipeline.display('_display_long_short_expected_v_observed_firing_rate', curr_active_pipeline.sess.get_context(), included_neuron_IDs=significant_distant_remapping_endcap_aclus)

In [ ]:
_temp_force_recompute=True

In [ ]:
## Recompute 'long_short_fr_indicies_analyses'
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_extended_computations

curr_active_pipeline.reload_default_computation_functions()

# _temp_force_recompute=True
_temp_force_recompute=False
# extended_computations_include_includelist = ['_perform_time_dependent_placefield_computation', 'long_short_fr_indicies_analyses', 'pf_dt_sequential_surprise', 'short_long_pf_overlap_analyses'] # do only specifiedl
extended_computations_include_includelist = ['jonathan_firing_rate_analysis', 'short_long_pf_overlap_analyses']
newly_computed_values = batch_extended_computations(curr_active_pipeline, include_includelist=extended_computations_include_includelist, include_global_functions=True, fail_on_exception=True, progress_print=True, force_recompute=_temp_force_recompute, debug_print=False)
newly_computed_values


In [ ]:
jonathan_firing_rate_analysis_result.rdf.rdf

In [ ]:
jonathan_firing_rate_analysis_result.irdf.irdf

In [ ]:
jonathan_firing_rate_analysis_result.neuron_replay_stats_df

In [ ]:
jonathan_firing_rate_analysis_result

In [ ]:
curr_long_short_fr_indicies_analysis_bak = curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis']

curr_long_short_fr_indicies_analysis = curr_active_pipeline.global_computation_results.computed_data.pop('long_short_fr_indicies_analysis')

print(list(curr_long_short_fr_indicies_analysis.keys())) # ['long_laps', 'long_replays', 'short_laps', 'short_replays', 'global_laps', 'global_replays', 'long_non_replays', 'short_non_replays', 'global_non_replays', 'long_mean_non_replays_frs', 'short_mean_non_replays_frs', 'long_mean_non_replays_all_frs', 'short_mean_non_replays_all_frs', 'non_replays_frs_index', 'long_mean_non_replays_all_inst_frs', 'short_mean_non_replays_all_inst_frs', 'non_replays_inst_frs_index', 'active_context']

print_keys_if_possible('curr_long_short_fr_indicies_analysis', curr_long_short_fr_indicies_analysis)

In [ ]:
curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis']

In [ ]:
curr_active_pipeline.sess.config

# 2023-09-12 - Assemble all neuron-level properties:

In [ ]:
from neuropy.core.neuron_identities import NeuronIdentityDataframeAccessor
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import build_merged_neuron_firing_rate_indicies

joined_neruon_fri_df = build_merged_neuron_firing_rate_indicies(curr_active_pipeline, enable_display_intermediate_results=False)
joined_neruon_fri_df

In [ ]:
joined_neruon_fri_df

In [ ]:
print(list(joined_neruon_fri_df.columns)) # ['aclu', 'lsfria_laps_frs_index', 'lsfria_laps_inst_frs_index', 'lsfria_replays_frs_index', 'lsfria_replays_inst_frs_index', 'lsfria_non_replays_frs_index', 'lsfria_non_replays_inst_frs_index', 'jfra_long_pf_peak_x', 'jfra_has_long_pf', 'jfra_short_pf_peak_x', 'jfra_has_short_pf', 'jfra_has_na', 'jfra_track_membership', 'jfra_long_non_replay_mean', 'jfra_short_non_replay_mean', 'jfra_non_replay_diff', 'jfra_long_replay_mean', 'jfra_short_replay_mean', 'jfra_replay_diff', 'jfra_long_mean', 'jfra_short_mean', 'jfra_mean_diff', 'jfra_neuron_IDX', 'jfra_num_replays', 'jfra_long_num_replays', 'jfra_short_num_replays', 'jfra_neuron_type', 'lspd_laps', 'lspd_replays', 'lspd_skew', 'lspd_max_axis_distance_from_center', 'lspd_distance_from_center', 'lspd_has_considerable_remapping']

recast_columns=['track_membership','neuron_type']

drop_columns=['jfra_aclu']
joined_neuron_fri_df.drop(columns=drop_columns)
['aclu', 'laps_frs_index', 'replays_frs_index', 'non_replays_frs_index', 'long_pf_peak_x', 'has_long_pf', 'short_pf_peak_x', 'has_short_pf', 'has_na', 'track_membership', 'long_non_replay_mean', 'short_non_replay_mean', 'non_replay_diff', 'long_replay_mean', 'short_replay_mean', 'replay_diff', 'long_mean', 'short_mean', 'mean_diff', 'neuron_IDX', 'num_replays', 'long_num_replays', 'short_num_replays', 'neuron_type', 'jfra_aclu', 'custom_frs_index', 'is_rate_extrema', 'is_refined_exclusive', 'is_refined_LxC', 'is_refined_SxC', 'laps', 'replays', 'skew', 'max_axis_distance_from_center', 'distance_from_center', 'has_considerable_remapping', 'session_uid', 'neuron_uid']

# Computing consolidated `long_short_fr_indicies_df`

In [ ]:
# 'long_short_fr_indicies_analysis'
curr_long_short_fr_indicies_analysis = curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis']
_curr_aclus = list(curr_long_short_fr_indicies_analysis['laps_frs_index'].keys()) # extract one set of keys for the aclus
_curr_frs_indicies_dict = {k:v.values() for k,v in curr_long_short_fr_indicies_analysis.items() if k in ['laps_frs_index', 'laps_inst_frs_index', 'replays_frs_index', 'replays_inst_frs_index', 'non_replays_frs_index', 'non_replays_inst_frs_index']} # extract the values
long_short_fr_indicies_df = pd.DataFrame(_curr_frs_indicies_dict, index=_curr_aclus)
long_short_fr_indicies_df

In [ ]:
long_short_fr_indicies_df

In [ ]:
curr_active_pipeline.reload_default_computation_functions()
curr_active_pipeline.perform_specific_computation('long_short_fr_indicies_analyses')


In [ ]:
# ax = long_short_fr_indicies_df.plot.scatter(x='non_replays_inst_frs_index' , y='replays_inst_frs_index', title='Replays v. Non-replay Firing Rate Index Comparison')
# long_short_fr_indicies_df.plot.scatter(x='laps_inst_frs_index' , y='replays_inst_frs_index', title='Replays v. Laps Firing Rate Index Comparison', ax=ax)

ax = long_short_fr_indicies_df.plot.scatter(x='non_replays_frs_index' , y='replays_frs_index', title='Replays v. Non-replay Firing Rate Index Comparison')
long_short_fr_indicies_df.plot.scatter(x='laps_frs_index' , y='replays_frs_index', title='Replays v. Laps Firing Rate Index Comparison', ax=ax)


In [ ]:
from pandas.plotting import scatter_matrix

# scatter_matrix(long_short_fr_indicies_df, figsize=(10, 10))
scatter_matrix(joined_neruon_fri_df, figsize=(10, 10))


# 2023-10-11 - Assigning Replays after delta to either the Long or Short track

In [ ]:
from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import TrackAssignmentState, TrackAssignmentDecision, AssigningEpochs

fig, ax, assigning_epochs_obj = AssigningEpochs.main_plot_iterative_epoch_track_assignments(curr_active_pipeline)




In [ ]:

assigning_epochs_obj.unassigned_epochs_df


In [ ]:
assigning_epochs_obj.filter_epochs_df

In [ ]:
is_assignable_by_long_exclusive_cells = np.logical_and(assigning_epochs_obj.filter_epochs_df['has_LONG_exclusive_aclu'], np.logical_not(assigning_epochs_obj.filter_epochs_df['has_SHORT_exclusive_aclu'])) # has long exclusive and no short-exclusive cells
np.sum(is_assignable_by_long_exclusive_cells)

In [ ]:
is_assignable_by_short_exclusive_cells = np.logical_and(assigning_epochs_obj.filter_epochs_df['has_SHORT_exclusive_aclu'], np.logical_not(assigning_epochs_obj.filter_epochs_df['has_LONG_exclusive_aclu'])) # has short exclusive and no long-exclusive cells
np.sum(is_assignable_by_short_exclusive_cells)

In [ ]:
assigning_epochs_obj

In [ ]:

## Get the active_unique_aclus during the epoch and score them based on their rate-remapping value:
assigning_epochs_obj.filter_epochs_df.active_unique_aclus

In [ ]:
assigning_epochs_obj.unassigned_epochs_df

## Plots the RateRemappingFiringRateIndex Number Line Figure


In [ ]:
# ## Extract rr_* variables from rate_remapping_df
# rr_aclus = rate_remapping_df.index.values
# rr_laps, rr_replays, rr_skew, rr_neuron_type = [rate_remapping_df[n].values for n in ['laps', 'replays', 'skew', 'neuron_type']]


## Extract rr_* variables from joined_neruon_fri_df
rr_aclus = joined_neruon_fri_df.index.values
rr_laps, rr_replays, rr_skew, rr_neuron_type = [joined_neruon_fri_df[n].values for n in ['lspd_laps', 'lspd_replays', 'lspd_skew', 'jfra_neuron_type']]

## Display:
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import plot_rr_aclu
n_debug_limit = 10
fig, axs, sort_indicies = plot_rr_aclu([str(aclu) for aclu in rr_aclus[:n_debug_limit]], rr_laps=rr_laps[:n_debug_limit], rr_replays=rr_replays[:n_debug_limit], rr_neuron_types=rr_neuron_type[:n_debug_limit])


In [ ]:
fri_index_dicts_list = [dict(x=rr_laps, marker=r'$\theta$', markersize=10, color='black', label='rr_laps'),
	dict(x=rr_replays, marker='o', markersize=10, fillstyle='none', color='black', label='rr_replays'),
]

In [ ]:

## Display Paginated multi-plot
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import plot_rr_aclu
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import RateRemappingPaginatedFigureController
active_identifying_session_ctx = curr_active_pipeline.sess.get_context()
_out_rr_pagination_controller = RateRemappingPaginatedFigureController.init_from_rr_data(rr_aclus, rr_laps, rr_replays, rr_neuron_type, max_subplots_per_page=30, a_name='TestRateRemappingPaginatedFigureController', active_context=active_identifying_session_ctx)
a_paginator = _out_rr_pagination_controller.plots_data.paginator

# 2023-09-28 - InstFrRate-based classification of LxC and SxC
2023-09-28 10:39am: We looked at the results and determined that it was much worse than the placefield critiera I had previously. This analysis should be used as a secondary refinement for the existing placefield-based LxC/SxC exclusion critiera. It includes information about non-lap endcap activity that the other analysis omits. Can be used with a threshold.

In [ ]:
## Plot the selected LxC/SxC cells cells on the PhoJonathan plots
curr_active_pipeline.reload_default_display_functions()
refined_LxC_aclus = long_exclusive.get_refined_track_exclusive_aclus()
print(f'refined_LxC_aclus: {refined_LxC_aclus}')
if len(refined_LxC_aclus):
	_out_LxC = curr_active_pipeline.display('_display_batch_pho_jonathan_replay_firing_rate_comparison', n_max_plot_rows=10, save_figure=False, included_unit_neuron_IDs=refined_LxC_aclus) # , included_unit_neuron_IDs=[4, 58]

refined_SxC_aclus = short_exclusive.get_refined_track_exclusive_aclus()
print(f'refined_SxC_aclus: {refined_SxC_aclus}')
if len(refined_SxC_aclus):
	_out_SxC = curr_active_pipeline.display('_display_batch_pho_jonathan_replay_firing_rate_comparison', n_max_plot_rows=10, save_figure=False, included_unit_neuron_IDs=refined_SxC_aclus) # , included_unit_neuron_IDs=[4, 58]


In [ ]:
loaded_df = pd.read_csv(Path(r'W:\Data\neuron_replay_stats_table_2023-09-29-GL.csv'))
loaded_df

In [ ]:
global_data_root_parent_path

In [ ]:
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionTables
 
neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.load_all_combined_tables(override_output_parent_path=global_data_root_parent_path, output_path_suffix=f'_{BATCH_DATE_TO_USE}')


In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import _fr_index
instantaneous_time_bin_size_seconds = 0.1
owning_pipeline_reference = curr_active_pipeline
long_epoch_obj, short_epoch_obj = [Epoch(owning_pipeline_reference.sess.epochs.to_dataframe().epochs.label_slice(an_epoch_name)) for an_epoch_name in [long_epoch_name, short_epoch_name]]

# non_running_periods = Epoch.from_PortionInterval(owning_pipeline_reference.sess.laps.as_epoch_obj().to_PortionInterval().complement())
non_replay_periods: Epoch = Epoch(Epoch.from_PortionInterval(owning_pipeline_reference.sess.replay.epochs.to_PortionInterval().complement()).time_slice(t_start=long_epoch_obj.t_start, t_stop=short_epoch_obj.t_stop).to_dataframe()[:-1]) #[:-1] # any period except the replay ones, drop the infinite last entry
long_only_non_replay_periods: Epoch  = non_replay_periods.time_slice(t_start=long_epoch_obj.t_start, t_stop=long_epoch_obj.t_stop) # any period except the replay ones
short_only_non_replay_periods: Epoch  = non_replay_periods.time_slice(t_start=short_epoch_obj.t_start, t_stop=short_epoch_obj.t_stop) # any period except the replay ones

# ~20sec computation
long_custom_InstSpikeRateTrends: SpikeRateTrends = SpikeRateTrends.init_from_spikes_and_epochs(spikes_df=deepcopy(owning_pipeline_reference.sess.spikes_df),
                                                                                        filter_epochs=long_only_non_replay_periods,
                                                                                        #    included_neuron_ids=long_exclusive.track_exclusive_aclus,
                                                                                        instantaneous_time_bin_size_seconds=instantaneous_time_bin_size_seconds)

short_custom_InstSpikeRateTrends: SpikeRateTrends = SpikeRateTrends.init_from_spikes_and_epochs(spikes_df=deepcopy(owning_pipeline_reference.sess.spikes_df),
                                                                                        filter_epochs=short_only_non_replay_periods,
                                                                                        #    included_neuron_ids=long_exclusive.track_exclusive_aclus,
                                                                                        instantaneous_time_bin_size_seconds=instantaneous_time_bin_size_seconds)

# # Note custom_InstSpikeRateTrends is global, not really needed:
# global_custom_InstSpikeRateTrends: SpikeRateTrends = SpikeRateTrends.init_from_spikes_and_epochs(spikes_df=deepcopy(owning_pipeline_reference.sess.spikes_df),
#                                                                                            filter_epochs=non_replay_periods,
#                                                                                         #    included_neuron_ids=long_exclusive.track_exclusive_aclus,
#                                                                                            instantaneous_time_bin_size_seconds=instantaneous_time_bin_size_seconds)

## Determine the included percentiles for `joined_neruon_fri_df`
custom_InstSpikeRateTrends_df = pd.DataFrame({'aclu': long_custom_InstSpikeRateTrends.included_neuron_ids, 'long_inst_fr': long_custom_InstSpikeRateTrends.cell_agg_inst_fr_list, 'short_inst_fr': short_custom_InstSpikeRateTrends.cell_agg_inst_fr_list}) #
# custom_InstSpikeRateTrends_df['global_inst_fr'] = global_custom_InstSpikeRateTrends.cell_agg_inst_fr_list

# Compute the single-dimensional firing rate index for the custom epochs and add it as a column to the dataframe:
custom_InstSpikeRateTrends_df['custom_frs_index'] = _fr_index(long_fr=long_custom_InstSpikeRateTrends.cell_agg_inst_fr_list, short_fr=short_custom_InstSpikeRateTrends.cell_agg_inst_fr_list)

# # Calculate 10th and 90th percentiles
# lower_bound = custom_InstSpikeRateTrends_df['custom_frs_index'].quantile(0.10)
# upper_bound = custom_InstSpikeRateTrends_df['custom_frs_index'].quantile(0.90)

# # Filter rows
# bottom_10_percent = custom_InstSpikeRateTrends_df[custom_InstSpikeRateTrends_df['custom_frs_index'] <= lower_bound]
# top_10_percent = custom_InstSpikeRateTrends_df[custom_InstSpikeRateTrends_df['custom_frs_index'] >= upper_bound]

# # Extract the aclus from these rows:
# LxC_10_percent_aclus = top_10_percent.aclu.to_numpy()
# SxC_10_percent_aclus = bottom_10_percent.aclu.to_numpy()

# print(f'LxC_10_percent_aclus: {LxC_10_percent_aclus}')
# print(f'SxC_10_percent_aclus: {SxC_10_percent_aclus}')

In [ ]:
## Really want to penalize for any firing on the opposite track.

In [ ]:
# I wanna look at the suprise metrics again please.

# can align each of the sessions (across days) to the track transition point (the "Delta") as the zero.

# print_keys_if_possible("global_computation_results", curr_active_pipeline.global_computation_results, max_depth=2)

# computed_data: pyphocorehelpers.DataStructure.dynamic_parameters.DynamicParameters
# 		│   ├── jonathan_firing_rate_analysis: pyphocorehelpers.DataStructure.dynamic_parameters.DynamicParameters
# 		│   ├── long_short_fr_indicies_analysis: pyphocorehelpers.DataStructure.dynamic_parameters.DynamicParameters
# 		│   ├── long_short_leave_one_out_decoding_analysis: pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations.LeaveOneOutDecodingAnalysis
# 		│   ├── long_short_post_decoding: pyphocorehelpers.DataStructure.dynamic_parameters.DynamicParameters


print_keys_if_possible("global_computation_results", curr_active_pipeline.global_computation_results.computed_data, max_depth=3)

# jonathan_firing_rate_analysis_result.rdf

# long_short_leave_one_out_decoding_analysis

In [ ]:
import seaborn as sns
from pyphocorehelpers.indexing_helpers import partition, safe_pandas_get_group
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import InstantaneousFiringRatesDataframeAccessor, AcrossSessionsVisualizations
from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import SingleBarResult

common_file_path = Path('output/active_across_session_scatter_plot_results.h5')
print(f'common_file_path: {common_file_path}')

# InstantaneousSpikeRateGroupsComputation, : pd.DataFrame
_shell_obj, loaded_result_df = InstantaneousFiringRatesDataframeAccessor.load_and_prepare_for_plot(common_file_path)
# Perform the actual plotting:
AcrossSessionsVisualizations.across_sessions_bar_graphs(_shell_obj, num_sessions=13, save_figure=False, enable_tiny_point_labels=False, enable_hover_labels=True)

In [ ]:
from datetime import datetime, timedelta

maximum_timedelta: timedelta = timedelta(1, 0, 0) # 1 Day maximum time
delta_since_last_compute: timedelta = curr_active_pipeline.get_time_since_last_computation()
print(f'delta_since_last_compute: {delta_since_last_compute}')
needs_recompute: bool = delta_since_last_compute > maximum_timedelta
print(f'\tneeds_recompute: {needs_recompute}')

In [ ]:
## Serialization of preprocessing parameters test
file_path = 'output/preprocessing_parameters.h5'

with h5py.File(file_path, "w") as f:
	preprocessing_group = f.create_group("preprocessing_parameters")

	# Serialize epoch_estimation_parameters
	epoch_estimation_group = preprocessing_group.create_group("epoch_estimation_parameters")

	laps_group = epoch_estimation_group.create_group("laps")
	laps_group.attrs["N"] = preprocessing_parameters_dict["epoch_estimation_parameters"]["laps"]["N"]
	laps_group.attrs["should_backup_extant_laps_obj"] = preprocessing_parameters_dict["epoch_estimation_parameters"]["laps"]["should_backup_extant_laps_obj"]

	PBEs_group = epoch_estimation_group.create_group("PBEs")
	PBEs_group.attrs["thresh"] = preprocessing_parameters_dict["epoch_estimation_parameters"]["PBEs"]["thresh"]
	PBEs_group.attrs["min_dur"] = preprocessing_parameters_dict["epoch_estimation_parameters"]["PBEs"]["min_dur"]
	PBEs_group.attrs["merge_dur"] = preprocessing_parameters_dict["epoch_estimation_parameters"]["PBEs"]["merge_dur"]
	PBEs_group.attrs["max_dur"] = preprocessing_parameters_dict["epoch_estimation_parameters"]["PBEs"]["max_dur"]

	replays_group = epoch_estimation_group.create_group("replays")
	replay_data = preprocessing_parameters_dict["epoch_estimation_parameters"]["replays"]
	# replay_dataframe = pd.DataFrame(replay_data)
	# replay_data.to_hdf(f, "/preprocessing_parameters/epoch_estimation_parameters/replays")
	## TODO: add the data here using Epoch's .to_hdf
	
	# Check for "None" values before setting attributes
	def check_and_set(key, value):
		if value is not None:
			replays_group.attrs[key] = value

	# Set attributes if not "None", otherwise skip writing
	check_and_set("min_epoch_included_duration", preprocessing_parameters_dict["epoch_estimation_parameters"].get('replays', {}).get("min_epoch_included_duration"))
	check_and_set("max_epoch_included_duration", preprocessing_parameters_dict["epoch_estimation_parameters"].get('replays', {}).get("max_epoch_included_duration"))
	check_and_set("maximum_speed_thresh", preprocessing_parameters_dict["epoch_estimation_parameters"].get('replays', {}).get("maximum_speed_thresh"))
	check_and_set("min_inclusion_fr_active_thresh", preprocessing_parameters_dict["epoch_estimation_parameters"].get('replays', {}).get("min_inclusion_fr_active_thresh"))
	check_and_set("min_num_unique_aclu_inclusions", preprocessing_parameters_dict["epoch_estimation_parameters"].get('replays', {}).get("min_num_unique_aclu_inclusions"))

In [ ]:
enable_default_neptune_plots = False
# enable_default_neptune_plots = True

## To file only:
with matplotlib_file_only():
	# Perform non-interactive Matplotlib operations with 'AGG' backend
	main_complete_figure_generations(curr_active_pipeline, enable_default_neptune_plots=enable_default_neptune_plots, save_figures_only=True, save_figure=True)

## Clear the Programmatically open figures:
plt.close('all') # this takes care of the matplotlib-backed figures.
curr_active_pipeline.clear_display_outputs()
curr_active_pipeline.clear_registered_output_files()

In [ ]:
# Showing
restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
# Perform interactive Matplotlib operations with 'Qt5Agg' backend
_out_figures = main_complete_figure_generations(curr_active_pipeline, enable_default_neptune_plots=False, save_figures_only=False, save_figure=True)

# Common Display Setup

In [19]:
import matplotlib
# configure backend here
matplotlib.use('Qt5Agg')
# backend_qt5agg
# %matplotlib qt5
# %matplotlib qt
# matplotlib.use('AGG') # non-interactive backend ## 2022-08-16 - Surprisingly this works to make the matplotlib figures render only to .png file, not appear on the screen!

import matplotlib as mpl
import matplotlib.pyplot as plt
_bak_rcParams = mpl.rcParams.copy()
# mpl.rcParams['toolbar'] = 'None' # disable toolbars

# Showing
restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')

In [ ]:
# import pylustrator # customization of figures
import matplotlib
# configure backend here
matplotlib.use('Qt5Agg')
import matplotlib as mpl
import matplotlib.pyplot as plt
_bak_rcParams = mpl.rcParams.copy()
import pylustrator # call `pylustrator.start()` before creating your first figure in code.

In [ ]:
pylustrator.start()

In [ ]:
# Showing
restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')

In [ ]:
# Plot The laps:
from neuropy.utils.matplotlib_helpers import plot_position_curves_figure
from pyphoplacecellanalysis.PhoPositionalData.plotting.laps import plot_laps_2d
pylustrator.start()
fig, out_axes_list = plot_laps_2d(global_session, legacy_plotting_mode=False, include_velocity=False, include_accel=False, figsize=(24, 10))
out_axes_list[0].set_title('Estimated Laps')
fig.canvas.manager.set_window_title('Estimated Laps')
ax = out_axes_list[0]
ax.set_xlim((1036.242685185441, 1441.769668184419))

# Hide y-axis ticklabels
plt.tick_params(axis='y', which='both', labelleft=False)

In [ ]:
fig = plt.gcf()
ax = fig.axes[0]
ax
ax.get_xlim()


laps_example_region_xlims = (1036.242685185441, 1441.769668184419)

fig, out_axes_list = plot_position_curves_figure(global_session.position, include_velocity=False, include_accel=False, figsize=(24, 10))

_out1 = curr_active_pipeline.display('_display_long_short_laps', include_velocity=False, include_accel=False)


In [ ]:
curr_active_pipeline.plot._display_decoder_result()

# 2023-09-07 - Track Graphics Testing

In [ ]:
from pyphocorehelpers.geometry_helpers import BoundsRect, point_tuple_mid_point, map_value
from pyphoplacecellanalysis.Pho2D.track_shape_drawing import LinearTrackDimensions
from pyphoplacecellanalysis.Pho2D.track_shape_drawing import add_vertical_track_bounds_lines, add_track_shapes, test_LinearTrackDimensions_2D_pyqtgraph
from neuropy.utils.mathutil import contiguous_regions, threshPeriods, compute_grid_bin_bounds, map_value
from pyphoplacecellanalysis.Pho2D.track_shape_drawing import test_LinearTrackDimensions_2D_Matplotlib
from pyphoplacecellanalysis.Pho2D.track_shape_drawing import LinearTrackInstance

grid_bin_bounds = BoundsRect.init_from_grid_bin_bounds(global_pf2D.config.grid_bin_bounds)
display(grid_bin_bounds)

# long_track_dims = LinearTrackDimensions.init_from_grid_bin_bounds(grid_bin_bounds)
# short_track_dims = LinearTrackDimensions.init_from_grid_bin_bounds(grid_bin_bounds)

long_track_dims = LinearTrackDimensions(track_length=170.0)
short_track_dims = LinearTrackDimensions(track_length=100.0)

common_1D_platform_height = 0.25
common_1D_track_height = 0.1
long_track_dims.minor_axis_platform_side_width = common_1D_platform_height
long_track_dims.track_width = common_1D_track_height # (short_track_dims.minor_axis_platform_side_width

short_track_dims.minor_axis_platform_side_width = common_1D_platform_height
short_track_dims.track_width = common_1D_track_height # (short_track_dims.minor_axis_platform_side_width

# instances:
long_track = LinearTrackInstance(long_track_dims, grid_bin_bounds=grid_bin_bounds)
short_track = LinearTrackInstance(short_track_dims, grid_bin_bounds=grid_bin_bounds)

print(long_track_dims)
print(short_track_dims)

In [ ]:
import napari

# global_pf1D_dt.config.grid_bin_bounds


def napari_add_grid_bin_bounds_rect(viewer: napari.viewer.Viewer, grid_bin_bounds):
	""" adds the animal's position as a track and a point layer to the napari viewer.


	Usage:

	from pyphoplacecellanalysis.GUI.Napari.napari_helpers import napari_add_animal_position
	napari_add_animal_position(viewer=viewer, position_df=global_pf1D_dt.all_time_filtered_pos_df[['t','x','binned_x']], time_intervals)


	"""
	# rect = np.array([[0, 0], [3, 1]])
	grid_bin_bounds_rect = grid_bin_bounds
	viewer.add_shapes(grid_bin_bounds_rect, shape_type='grid_bin_bounds_rect', edge_width=0.1)
    
napari_add_grid_bin_bounds_rect(viewer, grid_bin_bounds)

In [ ]:
## For napari need to map onto xbins axis:
#TODO: Not working, did manual rect drawing below instead.

assert len(viewer.dims.range) == 3
t_range_tuple, neuron_id_range_tuple, xbin_range_tuple = viewer.dims.range
# ((0.0, 3309.0, 1.0),
#  (-0.2906298631133275, 85.0, 1.0),
#  (-0.24635407377607876, 108.0, 1.0))
xbin_min, xbin_max = (xbin_range_tuple[0], (xbin_range_tuple[1]-xbin_range_tuple[2])) # the max range isn't included, so we need to subtract off the step to get the real max
xbin_width = (xbin_max - xbin_min) # total width in number of xbins
x_to_xbins_multiplier: float = xbin_width/grid_bin_bounds.size[0] # multiply a number in normal coordinates to get the equivalent number of xbins
x_to_xbins_multiplier

map_track_coords_to_xbins_space = lambda v: map_value(v, (grid_bin_bounds.xmin, grid_bin_bounds.xmax), (xbin_min, xbin_max)) # same map
# map_track_coords_to_ybins_space = lambda v: map_value(v, (grid_bin_bounds.ymin, grid_bin_bounds.ymax), (ybin_min, ybin_max)) # same map

bin_space_grid_bin_bounds = deepcopy(grid_bin_bounds)
bin_space_grid_bin_bounds.xmin = map_track_coords_to_xbins_space(grid_bin_bounds.xmin)
bin_space_grid_bin_bounds.xmax = map_track_coords_to_xbins_space(grid_bin_bounds.xmax)

bin_space_grid_bin_bounds.ymin = 0.0
bin_space_grid_bin_bounds.ymax = 5.0

bin_space_grid_bin_bounds

# map_track_coords_to_xbins_space(grid_bin_bounds.xmin), map_track_coords_to_xbins_space(grid_bin_bounds.xmax)
# bin_space_long_track_dims = LinearTrackDimensions.init_from_grid_bin_bounds(bin_space_grid_bin_bounds)
# bin_space_short_track_dims = LinearTrackDimensions.init_from_grid_bin_bounds(bin_space_grid_bin_bounds)

# common_1D_platform_height = 0.25
# common_1D_track_height = 0.1
# bin_space_long_track_dims.minor_axis_platform_side_width = common_1D_platform_height
# bin_space_long_track_dims.track_width = common_1D_track_height # (short_track_dims.minor_axis_platform_side_width

# bin_space_short_track_dims.minor_axis_platform_side_width = common_1D_platform_height
# bin_space_short_track_dims.track_width = common_1D_track_height # (short_track_dims.minor_axis_platform_side_width

bin_space_long_track_dims = deepcopy(long_track_dims)
bin_space_short_track_dims = deepcopy(short_track_dims)

bin_space_long_track_dims.axis_scale_factors = (x_to_xbins_multiplier, 1.0)
bin_space_short_track_dims.axis_scale_factors = (x_to_xbins_multiplier, 1.0)

bin_space_long_track = LinearTrackInstance(bin_space_long_track_dims, grid_bin_bounds=bin_space_grid_bin_bounds)
bin_space_short_track = LinearTrackInstance(bin_space_short_track_dims, grid_bin_bounds=bin_space_grid_bin_bounds)

bin_space_short_track

t_range_tuple, neuron_id_range_tuple, xbin_range_tuple = viewer.dims.range
t_range_tuple


bin_space_grid_bin_bounds
half_xbin_width = float(xbin_width)/2.0
half_xbin_width
bin_space_short_track.track_dimensions.scaled_total_length
# vertices are top-left and bottom-right corners
# [(x, 0.0, w, h) for x, y, w, h, *_unused in bin_space_short_track.rects]
# short_track_rectangle_data = [BoundsRect.init_from_x_y_w_h_tuple((x+np.abs(x), y, w, h)).corner_points for x, y, w, h, *_unused in bin_space_short_track.rects]
short_track_rectangle_data = [np.hstack((np.zeros((4,1)), BoundsRect.init_from_x_y_w_h_tuple((x+half_xbin_width, y, w, h)).corner_points))[:, :][:, [0, 2, 1]] for x, y, w, h, *_unused in bin_space_short_track.rects]
long_track_rectangle_data = np.vstack([np.hstack((np.zeros((4,1)), BoundsRect.init_from_x_y_w_h_tuple((x+half_xbin_width, y, w, h)).corner_points))[:, [0, 2, 1]] for x, y, w, h, *_unused in bin_space_long_track.rects])

# short_track_rectangle_data = np.vstack(short_track_rectangle_data)
short_track_rectangle_data

# array([[0, 2.375, 35], # bottom-left
    #    [0, 2.625, 35], # bottom-right
    #    [0, 2.375, 46.112], # top-left
    #    [0, 2.625, 46.112], # top-right

# [0,2,3,1]

In [ ]:
import itertools

# sync_point_y = -0.750314
sync_point_y = 0.0 # the bottom (y-axis) value of the track
track_platform_height_y = -5.19389 # the top coordinate (y-axis) value of the platforms

## Time indicies for tracks:
time_index = 0
long_short_change_time_index = 2097 # APPROXIMATE
t_range_tuple, neuron_id_range_tuple, xbin_range_tuple = viewer.dims.range
long_time_indicies = np.arange(t_range_tuple[0], long_short_change_time_index)
short_time_indicies = np.arange(long_short_change_time_index, (t_range_tuple[1]-t_range_tuple[2]))

### Long Track Shape Layer:
# Where the platform and the track connect:
track_platform_connect_x_min = 14.157
track_platform_connect_x_max = 91.6898
long_track_shapes_data = list(itertools.chain.from_iterable([
[
np.array([[time_index, track_platform_height_y, track_platform_connect_x_max],
        [time_index, track_platform_height_y, 106.093],
        [time_index, sync_point_y, 106.093],
        [time_index, sync_point_y, track_platform_connect_x_max]]),
 np.array([[time_index, -3.66162, 14.0037],
        [time_index, -3.66162, track_platform_connect_x_max],
        [time_index, sync_point_y, track_platform_connect_x_max],
        [time_index, sync_point_y, 14.0037]]),
 np.array([[time_index, track_platform_height_y, -0.246354],
        [time_index, track_platform_height_y, track_platform_connect_x_min],
        [time_index, sync_point_y, track_platform_connect_x_min],
        [time_index, sync_point_y, -0.246354]])
]
for time_index in long_time_indicies]))
long_track_shapes_layer = viewer.add_shapes(long_track_shapes_data, shape_type='rectangle', name='long_track', edge_width=0, face_color='#aa0000ff')
long_track_shapes_layer.editable = False


### Short Track Shape Layer:
# Where the platform and the track connect:
track_platform_connect_x_min = 28.2539
track_platform_connect_x_max = 73.7623

short_track_shapes_data =  list(itertools.chain.from_iterable([
[
np.array([[time_index, track_platform_height_y, track_platform_connect_x_max],
        [time_index, track_platform_height_y, 88.1655],
        [time_index, sync_point_y, 88.1655],
        [time_index, sync_point_y, track_platform_connect_x_max]]),
np.array([[time_index, -3.66162, 28.1006],
        [time_index, -3.66162, track_platform_connect_x_max],
        [time_index, sync_point_y, track_platform_connect_x_max],
        [time_index, sync_point_y, 28.1006]]),
np.array([[time_index, track_platform_height_y, 13.8505],
        [time_index, track_platform_height_y, track_platform_connect_x_min],
        [time_index, sync_point_y, track_platform_connect_x_min],
        [time_index, sync_point_y, 13.8505]])
]
for time_index in short_time_indicies]))

short_track_shapes_layer = viewer.add_shapes(short_track_shapes_data, shape_type='rectangle', name='short_track', edge_width=0, face_color='royalblue')
short_track_shapes_layer.editable = False

In [ ]:
long_track_shapes_layer.data

In [ ]:
shapes_layer = viewer.layers['short_track_rectangle_data']
shapes_layer.data





# Test classifying various x-positions as belonging to outside the outside_maze, the track_endcaps, or the track_body

In [ ]:
long_rects_outputs, short_rects_outputs = add_track_shapes(grid_bin_bounds, ax=None)

In [ ]:
# long_offset=(150.0, 0.5)
# short_offset=(150.0, -0.5)

long_offset=(grid_bin_bounds.center_point[0], 0.5)
short_offset=(grid_bin_bounds.center_point[0], -0.5)

# # Create a second y-axis sharing the same x-axis as ax
# ax2 = ax.twinx()
# # ax2.plot(x, y2, 'b-')
# ax2.set_ylabel('Track data', color='b')
# # Set the adjustable attribute to 'datalim'
# ax.set_adjustable('datalim')
# ax2.set_adjustable('datalim')
# long_track_dims.plot_rects(ax2, offset=long_offset)
# short_track_dims.plot_rects(ax2, offset=short_offset)

combined_item, rect_items, rects = long_track_dims.plot_rects(ax, offset=long_offset)
short_track_dims.plot_rects(ax, offset=short_offset)

In [ ]:
fig, ax1, ax2 = test_LinearTrackDimensions_2D_Matplotlib(long_track_dims, short_track_dims, long_offset=(150.0, 0.5), short_offset=(150.0, -0.5))

In [ ]:

# Find center from `grid_bin_bounds`
# long_pf2D.bin_info
grid_bin_bounds = deepcopy(long_pf2D.config.grid_bin_bounds)

x_diff = (grid_bin_bounds[0][1] - grid_bin_bounds[0][0])
y_diff = (grid_bin_bounds[1][1] - grid_bin_bounds[1][0])
print(f'x_diff: {x_diff}, y_diff: {y_diff}')

# Find the x, y midpoints:
x_midpoint = grid_bin_bounds[0][0] + (x_diff/2.0)
y_midpoint = grid_bin_bounds[1][0] + (y_diff/2.0)
print(f'x_midpoint: {x_midpoint}, y_midpoint: {y_midpoint}')


In [ ]:
grid_bin_bounds_center_point = (point_tuple_mid_point(grid_bin_bounds[0]), point_tuple_mid_point(grid_bin_bounds[1])) # (145.43, 140.61)
x_midpoint, y_midpoint = grid_bin_bounds_center_point

In [ ]:

# long_track_dims.total_length # 214.0
# short_track_dims.total_length # 144.0
# zero_alignment_point = self.total_length/2.0
long_notable_x_positions, _long_notable_y_positions = long_track_dims._build_component_notable_positions(offset_point=(x_midpoint, y_midpoint))
short_notable_x_positions, _short_notable_y_positions = short_track_dims._build_component_notable_positions(offset_point=(x_midpoint, y_midpoint))

In [ ]:
# def _add_vertical_track_bounds_lines(grid_bin_bounds, ax=None):
# 	""" 
# 	Captures: long_notable_x_positions, short_notable_x_positions
	
# 	Usage:
# 		grid_bin_bounds = deepcopy(long_pf2D.config.grid_bin_bounds)
# 		long_track_line_collection, short_track_line_collection = _add_vertical_track_bounds_lines(grid_bin_bounds=grid_bin_bounds, ax=ax)

# 	"""
# 	long_track_dims = LinearTrackDimensions(track_length=170.0)
# 	short_track_dims = LinearTrackDimensions(track_length=100.0)

# 	# Find center from `grid_bin_bounds` using `point_tuple_mid_point`
# 	x_midpoint, y_midpoint = (point_tuple_mid_point(grid_bin_bounds[0]), point_tuple_mid_point(grid_bin_bounds[1])) # grid_bin_bounds_center_point: (145.43, 140.61)

# 	long_notable_x_positions, _long_notable_y_positions = long_track_dims._build_component_notable_positions(offset_point=(x_midpoint, y_midpoint))
# 	short_notable_x_positions, _short_notable_y_positions = short_track_dims._build_component_notable_positions(offset_point=(x_midpoint, y_midpoint))

# 	# Omit the midpoint
# 	long_notable_x_platform_positions = long_notable_x_positions[[0,1,3,4]]
# 	short_notable_x_platform_positions = short_notable_x_positions[[0,1,3,4]]

# 	## Adds to current axes:
# 	if ax is None:
# 		fig = plt.gcf()
# 		axs = fig.get_axes()
# 		ax = axs[0]
# 	long_track_line_collection: matplotlib.collections.LineCollection = plt.vlines(long_notable_x_platform_positions, label='long_track_x_pos_lines', ymin=ax.get_ybound()[0], ymax=ax.get_ybound()[1], colors='#0000FFAA', linestyles='dashed') # matplotlib.collections.LineCollection
# 	short_track_line_collection: matplotlib.collections.LineCollection = plt.vlines(short_notable_x_platform_positions, label='short_track_x_pos_lines', ymin=ax.get_ybound()[0], ymax=ax.get_ybound()[1], colors='#FF0000AA', linestyles='dashed') # matplotlib.collections.LineCollection
# 	return long_track_line_collection, short_track_line_collection

grid_bin_bounds = deepcopy(long_pf2D.config.grid_bin_bounds)
long_track_line_collection, short_track_line_collection = add_vertical_track_bounds_lines(grid_bin_bounds=grid_bin_bounds, ax=None)

In [ ]:
rect_items, rects = long_track_dims.plot_rects(axs[0]) # , offset=(x, y) # , offset=(0.1, 100.0)


In [ ]:
long_track_line_collection.remove()

In [ ]:
short_track_line_collection.remove()

In [ ]:
long_track_dims.plot_rects(ax, offset=)


In [ ]:
fig = plt.gcf()
axs = fig.get_axes()
ax = axs[0]

In [ ]:
long_track_line_collection.remove()
short_track_line_collection.remove()

In [ ]:
from pyphoplacecellanalysis.PhoPositionalData.plotting.placefield import plot_1D_placecell_validation

_out = plot_1D_placecell_validation(long_pf1D, 0)

axes = _out[1]
ax = axes[0]
# ax = axes[1]
ax

In [ ]:
long_track_dims = LinearTrackDimensions.init_from_grid_bin_bounds(grid_bin_bounds=grid_bin_bounds, debug_print=True)

fig, axs = plt.subplots(2, 1)

rect_items, rects = long_track_dims.plot_rects(axs[0]) # , offset=(x, y) # , offset=(0.1, 100.0)


In [ ]:

# long_track_dims.plot_rects(ax)

In [ ]:
_out = curr_active_pipeline.display('_display_grid_bin_bounds_validation', curr_active_pipeline.get_session_context(), defer_render=False, save_figure=True)
fig = _out.figures[0]
ax = _out.axes[0]

In [ ]:
# Extract the bounding box coordinates and dimensions
bbox = ax.bbox
x, y, width, height = bbox.x0, bbox.y0, bbox.width, bbox.height
x, y, width, height

In [ ]:
ax_empty = ax

In [ ]:
rect_items, rects = long_track_dims.plot_rects(ax_empty) # , offset=(x, y) # , offset=(0.1, 100.0)


In [ ]:
(320.0, 983.7668560462606, 901.8181818181818, 91.99628790747863)
active_track_dims = LinearTrackDimensions(width=901.8181818181818)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.transforms import Bbox

# Step 2: Define a simple vector graphic (an arrow in this case)
def draw_arrow(ax, x, y, width, height):
    arrow = patches.FancyArrow(x, y, width, height, width=0.2*height, color="red")
    ax.add_patch(arrow)

# Step 3: Scale and draw the arrow within a bounding box
def draw_scaled_arrow_in_bbox(ax, bbox):
    # Extract the bounding box coordinates and dimensions
    x, y, width, height = bbox.x0, bbox.y0, bbox.width, bbox.height

    # Draw the bounding box (for visualization)
    rect = patches.Rectangle((x, y), width, height, linewidth=1, edgecolor='black', facecolor='none')
    ax.add_patch(rect)

    # Draw the scaled arrow within the bounding box
    draw_arrow(ax, x, y, width, height)

# Main code
fig, ax = plt.subplots()
bbox = Bbox.from_bounds(0.2, 0.2, 0.6, 0.6)  # Define bounding box with [x, y, width, height]

draw_scaled_arrow_in_bbox(ax, bbox)

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_aspect('equal')
plt.show()


In [ ]:
rect_items[0]

In [ ]:
rects

# 2023-09-21 - Continuous Surprise Figure - How do I display it?

In [ ]:
# '_perform_time_dependent_pf_sequential_surprise_computation'
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import plot_long_short_surprise_difference_plot, plot_long_short, plot_long_short_any_values
from pyphoplacecellanalysis.SpecificResults.fourthYearPresentation import fig_surprise_results

from neuropy.utils.matplotlib_helpers import draw_epoch_regions
from pyphocorehelpers.DataStructure.RenderPlots.MatplotLibRenderPlots import MatplotlibRenderPlots
from pyphoplacecellanalysis.General.Model.Configs.LongShortDisplayConfig import LongShortDisplayConfigManager
long_short_display_config_manager = LongShortDisplayConfigManager()
long_epoch_config = long_short_display_config_manager.long_epoch_config.as_matplotlib_kwargs()
short_epoch_config = long_short_display_config_manager.short_epoch_config.as_matplotlib_kwargs()

graphics_outputs_list = fig_surprise_results(curr_active_pipeline)


### FOUND! 2023-09-21 - Found code that generated surprise plots, strangely not stored anywhere!

In [ ]:
# note: nSnapshots == n_post_update_times
active_relative_entropy_results = active_extended_stats['pf_dt_sequential_surprise']
post_update_times = active_relative_entropy_results['post_update_times'] # (4123,) = (nSnapshots,)
snapshot_differences_result_dict = active_relative_entropy_results['snapshot_differences_result_dict']
time_intervals = active_relative_entropy_results['time_intervals']
long_short_rel_entr_curves_frames = active_relative_entropy_results['long_short_rel_entr_curves_frames'] # (4123, 108, 63) = (nSnapshots, n_neurons, n_xbins)
short_long_rel_entr_curves_frames = active_relative_entropy_results['short_long_rel_entr_curves_frames'] # (4123, 108, 63) = (nSnapshots, n_neurons, n_xbins)
flat_relative_entropy_results = active_relative_entropy_results['flat_relative_entropy_results'] # (4123, 63) - (nSnapshots, nXbins)
flat_jensen_shannon_distance_results = active_relative_entropy_results['flat_jensen_shannon_distance_results'] # (4123, 63) - (nSnapshots, nXbins)
flat_jensen_shannon_distance_across_all_positions = np.sum(np.abs(flat_jensen_shannon_distance_results), axis=1) # sum across all position bins # (4123,) - (nSnapshots)
flat_surprise_across_all_positions = np.sum(np.abs(flat_relative_entropy_results), axis=1) # sum across all position bins # (4123,) - (nSnapshots)
nSnapshots, n_neurons, n_xbins = np.shape(long_short_rel_entr_curves_frames)
print(f'nSnapshots: {nSnapshots}, n_neurons: {n_neurons}, n_xbins: {n_xbins}') # nSnapshots: 3308, n_neurons: 85, n_xbins: 107

In [ ]:
# post_update_times.shape
# np.shape(post_update_times)[0]



In [ ]:
np.shape(flat_jensen_shannon_distance_results)

In [ ]:
np.shape(flat_relative_entropy_results)

In [ ]:
np.shape(flat_jensen_shannon_distance_across_all_positions)

In [ ]:
np.shape(long_short_rel_entr_curves_frames)

In [ ]:
## NOTE: these results are NOT the same as the ones calculated in compute_snapshot_relative_entropy_surprise_differences.compute_surprise_relative_entropy_divergence, and I'm not quite sure why:
# Jensen-Shannon distance is an average of KL divergence:
from scipy.special import rel_entr
mixture_distribution = 0.5 * (long_short_rel_entr_curves_frames + short_long_rel_entr_curves_frames)
print(f'mixture_distribution.shape: {np.shape(mixture_distribution)}') # (nSnapshots, n_neurons, n_xbins)
# jensen_shannon_distance = 0.5 * (sum(rel_entr(mixture_distribution, long_short_rel_entr_curves_frames)) + sum(rel_entr(mixture_distribution, short_long_rel_entr_curves_frames))) # is this right? I'm confused by sum(...) # (n_neurons, n_xbins)
jensen_shannon_distance = 0.5 * (np.sum(rel_entr(mixture_distribution, long_short_rel_entr_curves_frames), axis=1) + np.sum(rel_entr(mixture_distribution, short_long_rel_entr_curves_frames), axis=1)) # alt version: (nSnapshots, n_xbins)
print(f'jensen_shannon_distance.shape: {np.shape(jensen_shannon_distance)}') # (n_neurons, n_xbins)

In [ ]:
np.isclose(flat_jensen_shannon_distance_results, jensen_shannon_distance).all()


In [ ]:
viewer.add_image(flat_jensen_shannon_distance_results)

In [ ]:
from silx.gui.plot import Plot2D

plot = Plot2D()  # Create the plot widget
plot.addImage(flat_jensen_shannon_distance_results, legend='flat_jensen_shannon_distance_results')  # Plot the 2D data set with default colormap
plot.setGraphTitle('flat_jensen_shannon_distance_results')
plot.getXAxis().setLabel('Position Bin')
plot.getYAxis().setLabel('Snapshot Timebin')
plot.show()  # Make the plot widget visible


In [ ]:
plot = Plot2D()  # Create the plot widget
plot.addImage(jensen_shannon_distance, legend='jensen_shannon_distance')  # Plot the 2D data set with default colormap
plot.setGraphTitle('jensen_shannon_distance')
plot.getXAxis().setLabel('Position Bin')
plot.getYAxis().setLabel('Snapshot Timebin')
plot.show()  # Make the plot widget visible

In [ ]:

plot = Plot2D()  # Create the plot widget
plot.addImage(long_short_rel_entr_curves_frames, legend='image')  # Plot the 2D data set with default colormap
plot.setGraphTitle('long_short_rel_entr_curves_frames')
# plot.getXAxis().setLabel('Position Bin')
# plot.getYAxis().setLabel('Snapshot Timebin')
plot.show()  # Make the plot widget visible


In [ ]:
from silx.gui.plot.StackView import StackViewMainWindow

# synthetic data, stack of 100 images of size 200x300
# mystack = np.fromfunction(
#     lambda i, j, k: np.sin(i/15.) + np.cos(j/4.) + 2 * np.sin(k/6.),
#     (100, 200, 300)
# )

sv = StackViewMainWindow()
sv.setColormap("jet", autoscale=True)
sv.setStack(long_short_rel_entr_curves_frames)
# (4152, 108, 63) = (n_post_update_times, n_neurons, n_xbins)
sv.setLabels(["post_update_time", "aclu", "Position (xbin)"])
sv.show()

In [ ]:


heatmap_pf1D_win, heatmap_pf1D_img = visualize_heatmap_pyqtgraph(flat_jensen_shannon_distance_results, show_yticks=False, title=f"flat_jensen_shannon_distance_results", defer_show=False)

In [ ]:
heatmap_pf1D_win, heatmap_pf1D_img = visualize_heatmap_pyqtgraph(flat_jensen_shannon_distance_across_all_positions, show_yticks=False, title=f"flat_jensen_shannon_distance_across_all_positions")

In [ ]:
from neuropy.utils.matplotlib_helpers import draw_epoch_regions
from neuropy.core.epoch import Epoch


fig, ax = plt.subplots()
ax.plot(post_update_times, flat_surprise_across_all_positions)
ax.set_ylabel('Relative Entropy across all positions')
ax.set_xlabel('t (seconds)')
epochs_collection, epoch_labels = draw_epoch_regions(curr_active_pipeline.sess.epochs, ax, facecolor=('r','cyan'), alpha=0.1, edgecolors=None, labels_kwargs={'y_offset': -0.05, 'size': 14}, defer_render=True, debug_print=False)
laps_epochs_collection, laps_epoch_labels = draw_epoch_regions(curr_active_pipeline.sess.laps.as_epoch_obj(), ax, facecolor='r', edgecolors='black', labels_kwargs={'y_offset': -16.0, 'size':8}, defer_render=True, debug_print=False)
# replays_epochs_collection, replays_epoch_labels = draw_epoch_regions(active_filter_epoch_obj, ax, facecolor='orange', edgecolors=None, labels_kwargs=None, defer_render=False, debug_print=False)
fig.suptitle('flat_surprise_across_all_positions')
fig.show()

In [ ]:
fig, ax = plt.subplots()
ax.plot(post_update_times, flat_jensen_shannon_distance_across_all_positions, label='JS_Distance')
ax.set_ylabel('J-S Distance across all positions')
ax.set_xlabel('t (seconds)')
epochs_collection, epoch_labels = draw_epoch_regions(curr_active_pipeline.sess.epochs, ax, facecolor=('red','cyan'), alpha=0.1, edgecolors=None, labels_kwargs={'y_offset': -0.05, 'size': 14}, defer_render=True, debug_print=False)
laps_epochs_collection, laps_epoch_labels = draw_epoch_regions(curr_active_pipeline.sess.laps.as_epoch_obj(), ax, facecolor='red', edgecolors='black', labels_kwargs={'y_offset': -16.0, 'size':8}, defer_render=True, debug_print=False)
# replays_epochs_collection, replays_epoch_labels = draw_epoch_regions(active_filter_epoch_obj, ax, facecolor='orange', edgecolors=None, labels_kwargs=None, defer_render=False, debug_print=False)
fig.suptitle('flat_jensen_shannon_distance_across_all_positions')
fig.show()

In [ ]:
# Show basic relative entropy vs. time plot:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.plot(post_update_times, flat_relative_entropy_results)
ax.set_ylabel('Relative Entropy')
ax.set_xlabel('t (seconds)')
epochs_collection, epoch_labels = draw_epoch_regions(curr_active_pipeline.sess.epochs, ax, facecolor=('red','cyan'), alpha=0.1, edgecolors=None, labels_kwargs={'y_offset': -0.05, 'size': 14}, defer_render=False, debug_print=False)
fig.show()

In [ ]:
win, plots = plot_long_short_surprise_difference_plot(curr_active_pipeline, long_results_obj, short_results_obj, long_epoch_name, short_epoch_name)

In [ ]:
win, (ax_long, ax_short), legend = plot_long_short(long_results_obj, short_results_obj)

In [ ]:
x_fn = lambda a_results_obj: a_results_obj.all_epochs_decoded_epoch_time_bins_mean[:,0]
# y_fn = lambda a_results_obj: a_results_obj.all_epochs_all_cells_one_left_out_posterior_to_scrambled_pf_surprises_mean
# y_fn = lambda a_results_obj: a_results_obj.all_epochs_all_cells_one_left_out_posterior_to_pf_surprises_mean
y_fn = lambda a_results_obj: a_results_obj.all_epochs_computed_one_left_out_posterior_to_pf_surprises

# (time_bins, neurons), (epochs, neurons), (epochs)
# all_epochs_computed_one_left_out_posterior_to_pf_surprises, all_epochs_computed_cell_one_left_out_posterior_to_pf_surprises_mean, all_epochs_all_cells_one_left_out_posterior_to_pf_surprises_mean
win, plots_tuple, legend = plot_long_short_any_values(long_results_obj, short_results_obj, x=x_fn, y=y_fn) #  limit_aclus=[4]

In [ ]:
##TODO 2023-09-21 16:19: - [ ] Does not work due to QCode issue

from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.MultiContextComparingDisplayFunctions import _context_nested_docks


# active_display_output = {}
# active_identifying_filtered_session_ctx = global_epoch_context
# display_output = active_display_output | curr_active_pipeline.display('_display_context_nested_docks', active_identifying_filtered_session_ctx, enable_gui=False, debug_print=False) # returns {'master_dock_win': master_dock_win, 'app': app, 'out_items': out_items}
# master_dock_win = display_output['master_dock_win']
# app = display_output['app']
# out_items = display_output['out_items']

include_includelist = curr_active_pipeline.active_completed_computation_result_names # ['maze', 'sprinkle']

out_items = {}
master_dock_win, app, out_items = _context_nested_docks(curr_active_pipeline, active_config_names=include_includelist, **{'enable_gui': True, 'debug_print': False})
master_dock_win

In [ ]:
neuron_replay_stats_df: pd.DataFrame = jonathan_firing_rate_analysis_result.neuron_replay_stats_df
short_pf_peak_x = neuron_replay_stats_df.short_pf_peak_x.values
long_pf_peak_x = neuron_replay_stats_df.long_pf_peak_x.values

# long_pf_peak_x_maze_classification = [classify_x_position(test_x, long_rects) for test_x in long_pf_peak_x.values]
# short_pf_peak_x_maze_classification = [classify_x_position(test_x, short_rects) for test_x in short_pf_peak_x.values]
# [classify_x_position(test_x, short_rects) for test_x in long_pf_peak_x.values]

long_track.is_on_maze(long_pf_peak_x)
# long_track.is_on_endcap(long_pf_peak_x)

## NOW TODO 2023-09-20 22:33: - [ ] Need to fix tests and determing why some peaks are falling outside the bounds (as assessed by the is_on_maze function to see if implementation is valid

In [ ]:

# assert short_track.is_on_maze(short_pf_peak_x)[np.logical_not(np.isnan(short_pf_peak_x))].all(), f"all valid short peaks should be located on the short track" # should all be true yeah?
assert long_track.is_on_maze(short_pf_peak_x)[np.logical_not(np.isnan(short_pf_peak_x))].all(), f"all valid short peaks should be located on the long track"
assert long_track.is_on_maze(long_pf_peak_x)[np.logical_not(np.isnan(long_pf_peak_x))].all(), f"all valid long peaks should be located on the long track. {long_track.is_on_maze(long_pf_peak_x)[np.logical_not(np.isnan(long_pf_peak_x))]}" # not working
assert short_track.is_on_maze(short_pf_peak_x)[np.logical_not(np.isnan(short_pf_peak_x))].all(), f"all valid short peaks should be located on the short track. {short_track.is_on_maze(short_pf_peak_x)[np.logical_not(np.isnan(short_pf_peak_x))]}" # not working


In [ ]:
long_track.is_on_endcap(long_pf_peak_x)

## Debug Plots for `LinearTrackDimensions` and `LinearTrackInstance`

In [ ]:
app, w, cw, (long_track_dims, long_rect_items, long_rects), (short_track_dims, short_rect_items, short_rects) = test_LinearTrackDimensions_2D_pyqtgraph(long_track_dims, short_track_dims)


In [ ]:
fig = plt.gcf()
axs = fig.get_axes()
ax = axs[0]
ax

In [ ]:
import matplotlib.gridspec as gridspec

## Adds a new subplot to an existing (fig, ax) without requiring modifications in the original code!

# Get the current gridspec from ax
gs = ax.get_subplotspec().get_gridspec()

# Create a new gridspec with an additional column
gs_new = gridspec.GridSpec(1, 2, width_ratios=[1, 0.5]) # new column is half the width of the current one

# Reposition the existing ax using the new gridspec
ax.set_position(gs_new[0, 0].get_position(fig))

# Add a new subplot in the new column
ax2 = fig.add_subplot(gs_new[0, 1])


ax2.plot(np.cos(np.linspace(0, 10, 100)))
# ax2.cla()
# long_track_dims.plot_rects(ax2)

# long_track_dims.plot_rects(ax2, offset=(0.1, 0.0))

plt.tight_layout()
plt.show()


In [ ]:
_out2 = curr_active_pipeline.display('_display_batch_pho_jonathan_replay_firing_rate_comparison', included_unit_neuron_IDs=[2], n_max_plot_rows=2, save_figure=False)


In [ ]:
from pyphoplacecellanalysis.SpecificResults.fourthYearPresentation import fig_example_handpicked_pho_jonathan_active_set_cells

_LxC_out, _SxC_out = fig_example_handpicked_pho_jonathan_active_set_cells(curr_active_pipeline, save_figure=True, included_LxC_example_neuron_IDs=[4, 58], included_SxC_example_neuron_IDs=[2])

In [ ]:
# from neuropy.analyses.placefields import PfnDMixin, plotRaw_v_time
from neuropy.analyses.placefields import PfnDMixin

curr_active_pipeline.reload_default_display_functions()

_out3 = long_pf1D.plotRaw_v_time(0, should_include_trajectory=True, should_include_spikes=False, should_include_labels=True, use_filtered_positions=True, use_pandas_plotting=False)

In [ ]:
_out4 = long_pf1D.plotRaw_v_time(0, should_include_trajectory=True, should_include_spikes=False, should_include_labels=True, use_filtered_positions=False, use_pandas_plotting=False)
# _out4 = long_pf1D.plotRaw_v_time(0, should_include_trajectory=True, should_include_spikes=False, should_include_labels=True, use_filtered_positions=False, use_pandas_plotting=True)

# 🟢👁️‍🗨️ PhoKamran2023Paper Results

## Figure 1) pf1D Ratemaps, Active set, etc

In [11]:
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import plot_multiple_raster_plot, plot_raster_plot
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap_pyqtgraph # used in `plot_kourosh_activity_style_figure`
from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import PAPER_FIGURE_figure_1_full, PAPER_FIGURE_figure_1_add_replay_epoch_rasters


In [ ]:
# curr_active_pipeline.reload_default_display_functions()

pf1d_compare_graphics, (example_epoch_rasters_L, example_epoch_rasters_S), example_stacked_epoch_graphics, fig_1c_figures_out_dict = PAPER_FIGURE_figure_1_full(curr_active_pipeline) # did not display the pf1

In [ ]:
rdf = jonathan_firing_rate_analysis_result.rdf.rdf
rdf


## Figure 2) `PaperFigureTwo`: LxC/SxC Analyses
Note: this fails when SxC or LxC are empty for this session (as it's not meaningful to produce a comparison bar plot). In this case, aggregate across multiple sessions.

In [ ]:
from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import PaperFigureTwo

_out_fig_2 = PaperFigureTwo(instantaneous_time_bin_size_seconds=0.01) # 10ms
_out_fig_2.compute(curr_active_pipeline=curr_active_pipeline)
_out_fig_2.display()

## Figure 3) `PAPER_FIGURE_figure_3`: Firing Rate Index and Long/Short Firing Rate Replays v. Laps

In [13]:
from neuropy.utils.matplotlib_helpers import FormattedFigureText
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import _plot_long_short_firing_rate_indicies
# curr_active_pipeline.reload_default_display_functions()

_out, _out2 = PAPER_FIGURE_figure_3(curr_active_pipeline, defer_render=False, save_figure=False)

## Other Paper-related explorations:

In [ ]:
_out2.figures
_out.axes

In [ ]:

['start']


# Any change in duration over the recording session duration?
rdf['duration']

# rdf.plot.scatter('start', 'duration')

# rdf.plot.scatter('duration', 'num_neuron_participating') # strong positive trend

# rdf.plot.scatter('start', 'num_neuron_participating') # num neurons participating seems to increase over time, especially after the short track is introduced. Contrary to my hypothesis.

rdf.plot.scatter('start', 'num_short_only_neuron_participating')




# ['num_neuron_participating']

In [ ]:
curr_active_pipeline.reload_default_display_functions()
a_plot_obj = curr_active_pipeline.plot
a_plot_obj._display_spike_rasters_pyqtplot_2D()
# '_display_spike_rasters_pyqtplot_2D': ' Plots a standalone 2D raster plot\n        ',
# '_display_spike_rasters_pyqtplot_3D': ' Plots a standalone 3D raster plot with independent/standalone controls built-in\n        ',
# '_display_spike_rasters_pyqtplot_3D_with_2D_controls': ' Plots a standalone 3D raster plot (via pyqtgraph) with a separate 2D raster plot as the window with which you can adjust the viewed window. \n        ',
# '_display_spike_rasters_vedo_3D': ' Plots a standalone 3D raster plot with independent/standalone controls built-in\n        ',
# '_display_spike_rasters_vedo_3D_with_2D_controls': ' Plots a standalone 3D raster plot (via Vedo) with a separate 2D raster plot as the window with which you can adjust the viewed window. \n        ',
# '_display_spike_rasters_window': ' Displays a Spike3DRasterWindowWidget with a configurable set of raster widgets and controls in it.\n        ',type

In [ ]:
curr_active_pipeline.registered_display_function_docs_dict

In [ ]:
pf1d_compare_fig_L, pf1d_compare_fig_S = pf1d_compare_graphics.figures


In [ ]:
from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import build_shared_sorted_neuronIDs
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap_pyqtgraph

ratemap = long_pf1D.ratemap
included_unit_neuron_IDs = EITHER_subset.track_exclusive_aclus
rediculous_final_sorted_all_included_neuron_ID, rediculous_final_sorted_all_included_pfmap = build_shared_sorted_neuronIDs(ratemap, included_unit_neuron_IDs, sort_ind=new_all_aclus_sort_indicies.copy())

In [ ]:
heatmap_pf1D_win, heatmap_pf1D_img = visualize_heatmap_pyqtgraph(rediculous_final_sorted_all_included_pfmap, show_yticks=False, title=f"pf1D Sorted Visualization", defer_show=True)

In [ ]:
active_curves_sorted = long_pf1D.ratemap.normalized_tuning_curves[is_included][included_new_all_aclus_sort_indicies]
heatmap_pf1D_win, heatmap_pf1D_img = visualize_heatmap_pyqtgraph(active_curves_sorted, show_yticks=False, title=f"pf1D Sorted Visualization", defer_show=True)

In [ ]:
from pyphoplacecellanalysis.Pho2D.stacked_epoch_slices import DecodedEpochSlicesPaginatedFigureController

## Stacked Epoch Plot
example_stacked_epoch_graphics = curr_active_pipeline.display('_display_long_and_short_stacked_epoch_slices', defer_render=False, save_figure=True)
pagination_controller_L, pagination_controller_S = example_stacked_epoch_graphics.plot_data['controllers']
ax_L, ax_S = example_stacked_epoch_graphics.axes
final_figure_context_L, final_context_S = example_stacked_epoch_graphics.context

In [ ]:
## After launching the interactive Stacked Epoch Plots for user epoch selection, try to update the selection with previously added annotations:
from neuropy.core.user_annotations import UserAnnotationsManager
from pyphoplacecellanalysis.GUI.Qt.Mixins.PaginationMixins import SelectionsObject
from pyphoplacecellanalysis.Pho2D.stacked_epoch_slices import DecodedEpochSlicesPaginatedFigureController

user_annotation_man = UserAnnotationsManager()
user_annotations = user_annotation_man.get_user_annotations()

final_context_L = curr_active_pipeline.build_display_context_for_session(display_fn_name='DecodedEpochSlices', epochs='replays', decoder='long_results_obj')
final_context_S = curr_active_pipeline.build_display_context_for_session(display_fn_name='DecodedEpochSlices', epochs='replays', decoder='short_results_obj')
# _out_pagination_controller.params.active_identifying_figure_ctx.adding_context(None,  user_annotation="selections")
selections_context_L = final_context_L.adding_context(None,  user_annotation="selections")
selections_context_S = final_context_S.adding_context(None,  user_annotation="selections")

saved_selection_L: SelectionsObject = pagination_controller_L.save_selection()
saved_selection_S: SelectionsObject = pagination_controller_S.save_selection()
# saved_selection_L = user_annotation_man.update_selections_from_annotations(saved_selection_L, user_annotations)
# saved_selection_S = user_annotation_man.update_selections_from_annotations(saved_selection_S, user_annotations)

saved_selection_L = saved_selection_L.update_selections_from_annotations(user_annotations, debug_print=False)
saved_selection_S = saved_selection_S.update_selections_from_annotations(user_annotations, debug_print=False)

## re-apply the selections:
pagination_controller_L.restore_selections(saved_selection_L, defer_render=True)
pagination_controller_S.restore_selections(saved_selection_S, defer_render=True)

ax_L[0].figure.canvas.draw()  # .figures.canvas.draw()
ax_S[0].figure.canvas.draw()  # .figures.canvas.draw()

# 2023-09-06 - Plot the decoded positions for each replay on the track:


In [ ]:
from pyphoplacecellanalysis.Pho2D.track_shape_drawing import LinearTrackDimensions


long_track_dims = LinearTrackDimensions(track_length=100.0)
short_track_dims = LinearTrackDimensions(track_length=70.0)


# long_track_dims.plot_rects(ax)

In [ ]:
from pyphoplacecellanalysis.Pho2D.track_shape_drawing import _test_LinearTrackDimensions_2D

app, w, cw, (long_track_dims, long_rect_items, long_rects), (short_track_dims, short_rect_items, short_rects) = _test_LinearTrackDimensions_2D()

In [ ]:
_out_fig_3_a, _out_fig_3_b = PAPER_FIGURE_figure_3(curr_active_pipeline, defer_render=False, save_figure=False)



In [ ]:
ax = _out_fig_3_a.axes[0]


In [ ]:
long_track_dims.plot_rects(ax)

In [ ]:
pagination_controller_L

In [ ]:
# Set the symbol properties
# symbol = pg.mkPen('w', width=1)  # Pen for the lines
size = 1.0  # Fixed x-width
symbol_brush = None  # No brush for the symbol (transparent fill)

# override_scatter_plot_kwargs = dict(pxMode=False, symbol='|', size=size, pen={'color': 'w', 'width': 1.0}) # , brush=symbol_brush

override_scatter_plot_kwargs = dict(pxMode=False, symbol='arrow_up', size=1.0, pen={'color': 'w', 'width': 1.0}, hoverable=True) # 

In [ ]:
filter_epoch_spikes_df_L.spikes.rebuild_fragile_linear_neuron_IDXs();
filter_epoch_spikes_df_S.spikes.rebuild_fragile_linear_neuron_IDXs();

In [ ]:
app_L, win_L, plots_L, plots_data_L = plot_multiple_raster_plot(epochs_df_L, filter_epoch_spikes_df_L, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=new_all_aclus_sort_indicies, unit_colors_list=unit_colors_list, scatter_plot_kwargs=override_scatter_plot_kwargs,
										epoch_id_key_name='replay_epoch_id', scatter_app_name="Long Decoded Example Replays")

In [ ]:
app_S, win_S, plots_S, plots_data_S = plot_multiple_raster_plot(epochs_df_S, filter_epoch_spikes_df_S, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=new_all_aclus_sort_indicies, unit_colors_list=unit_colors_list, scatter_plot_kwargs=override_scatter_plot_kwargs,
                                                                 epoch_id_key_name='replay_epoch_id', scatter_app_name="Short Decoded Example Replays")

In [ ]:
# Test single `plot_raster_plot` calls
an_epoch = list(epochs_df_L.itertuples())[0]
an_epoch_spikes_df = filter_epoch_spikes_df_L[filter_epoch_spikes_df_L['replay_epoch_id'] == an_epoch.Index]

_out_single_raster_plot = plot_raster_plot(an_epoch_spikes_df, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=None, unit_colors_list=unit_colors_list, scatter_plot_kwargs=override_scatter_plot_kwargs, scatter_app_name="test1")
_out_single_raster_plot2 = plot_raster_plot(an_epoch_spikes_df, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=new_all_aclus_sort_indicies, unit_colors_list=unit_colors_list, scatter_plot_kwargs=override_scatter_plot_kwargs, scatter_app_name="test2")

In [ ]:
app_alt, win_alt, plots_alt, plots_data_alt = plot_multiple_raster_plot(epochs_df_L, filter_epoch_spikes_df_L, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=None, unit_colors_list=unit_colors_list, scatter_plot_kwargs=override_scatter_plot_kwargs,
                                                                         epoch_id_key_name='replay_epoch_id', scatter_app_name="ALT Long Decoded Example Replays")

### Testing `plot_kourosh_activity_style_figure` for debugging:

In [ ]:
from pyphoplacecellanalysis.Analysis.Decoder.decoder_result import plot_kourosh_activity_style_figure
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.helpers import _helper_make_scatterplot_clickable

plot_aclus = EITHER_subset.track_exclusive_aclus.copy()
# plot_aclus = EITHER_subset.track_exclusive_aclus[new_all_aclus_sort_indicies].copy()
_out_A = plot_kourosh_activity_style_figure(long_results_obj, long_session, plot_aclus, unit_sort_order=new_all_aclus_sort_indicies, epoch_idx=13, callout_epoch_IDXs=None, skip_rendering_callouts=False)

In [ ]:
app, win, plots, plots_data = _out_A
# plots

In [ ]:
#TODO 2023-06-27 10:42: - [ ] Desperitely need a class that "explodes" the important variables and their types out of a DynamicParameters (dict-like) or other object.


In [ ]:
_out_n = plot_kourosh_activity_style_figure(long_results_obj, long_session, EITHER_subset.track_exclusive_aclus, unit_sort_order=new_all_aclus_sort_indicies, epoch_idx=49, callout_epoch_IDXs=np.arange(6), skip_rendering_callouts=False)

# 2023-07-14 - LxC and SxC PhoJonathanSession plots

In [ ]:
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import BatchPhoJonathanFiguresHelper

## Get global 'jonathan_firing_rate_analysis' results:
curr_jonathan_firing_rate_analysis = curr_active_pipeline.global_computation_results.computed_data['jonathan_firing_rate_analysis']
neuron_replay_stats_df, rdf, aclu_to_idx, irdf = curr_jonathan_firing_rate_analysis.neuron_replay_stats_df, curr_jonathan_firing_rate_analysis.rdf.rdf, curr_jonathan_firing_rate_analysis.rdf.aclu_to_idx, curr_jonathan_firing_rate_analysis.irdf.irdf

# with VizTracer(output_file=f"viztracer_display_BatchPhoJonathanFiguresHelper_PlusPDF_20.json", min_duration=200, tracer_entries=3000000, ignore_frozen=True) as tracer:
# active_out_figures_dict = BatchPhoJonathanFiguresHelper.run(curr_active_pipeline, neuron_replay_stats_df, included_unit_neuron_IDs=XOR_subset.track_exclusive_aclus, n_max_page_rows=20, write_vector_format=False, write_png=True) # active_out_figures_dict: {IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-07_16-40-19', 'BatchPhoJonathanReplayFRC', 'long_only', '(12,21,48)')>: <Figure size 1920x660 with 12 Axes>, IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-07_16-40-19', 'BatchPhoJonathanReplayFRC', 'short_only', '(18,19,65)')>: <Figure size 1920x660 with 12 Axes>}
# print(f'active_out_figures_dict: {active_out_figures_dict}')

# BOTH_subset, EITHER_subset, XOR_subset, NEITHER_subset
active_out_figures_dict = BatchPhoJonathanFiguresHelper.run(curr_active_pipeline, neuron_replay_stats_df, included_unit_neuron_IDs=EITHER_subset.track_exclusive_aclus, n_max_page_rows=20, write_vector_format=False, write_png=True) # active_out_figures_dict: {IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-07_16-40-19', 'BatchPhoJonathanReplayFRC', 'long_only', '(12,21,48)')>: <Figure size 1920x660 with 12 Axes>, IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-07_16-40-19', 'BatchPhoJonathanReplayFRC', 'short_only', '(18,19,65)')>: <Figure size 1920x660 with 12 Axes>}
print(f'active_out_figures_dict: {active_out_figures_dict}')



In [ ]:
# 2023-09-07 - Build Example LxC/SxC cells from handpicked examples: aclus = [4, 58]
# from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import build_extra_cell_info_label_string
curr_active_pipeline.reload_default_display_functions()
_out2 = curr_active_pipeline.display('_display_batch_pho_jonathan_replay_firing_rate_comparison', n_max_plot_rows=2, save_figure=False, included_unit_neuron_IDs=[4, 58]) # , included_unit_neuron_IDs=[4, 58]


In [ ]:
# curr_active_pipeline.display(display_function='_display_2d_placefield_result_plot_ratemaps_2D', active_session_configuration_context=long_epoch_context)


# curr_active_pipeline.display(display_function='_display_placemaps_pyqtplot_2D', active_session_configuration_context=long_epoch_context)
curr_active_pipeline.display(display_function='_display_1d_placefields', active_session_configuration_context=long_epoch_context)
# curr_active_pipeline.display(display_function='_display_2d_placefield_occupancy', active_session_configuration_context=long_epoch_context)



In [ ]:
curr_active_pipeline.display(display_function='_display_2d_placefield_occupancy', active_session_configuration_context=long_epoch_context)


In [ ]:
curr_active_pipeline.display(display_function='_display_2d_placefield_occupancy', active_session_configuration_context=short_epoch_context)


In [ ]:
curr_active_pipeline.display(display_function='_display_2d_placefield_result_plot_ratemaps_2D', active_session_configuration_context=short_epoch_context)


In [ ]:
curr_active_pipeline.display(display_function='_display_2d_placefield_result_plot_ratemaps_2D', active_session_configuration_context=long_epoch_context)

In [ ]:
fig = plt.figure() # new figure to hold the result
# can show the figures by looping through and calling
for a_ctxt, a_fig in active_out_figures_dict.items():
    print(f'showing: {a_ctxt}')
    a_fig.show()
    


In [ ]:
long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
long_session, short_session, global_session = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]] # only uses global_session
(epochs_df_L, epochs_df_S), (filter_epoch_spikes_df_L, filter_epoch_spikes_df_S), (good_example_epoch_indicies_L, good_example_epoch_indicies_S), (short_exclusive, long_exclusive, BOTH_subset, EITHER_subset, XOR_subset, NEITHER_subset), new_all_aclus_sort_indicies, assigning_epochs_obj = PAPER_FIGURE_figure_1_add_replay_epoch_rasters(curr_active_pipeline)

long_short_fr_indicies_analysis_results = curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis']
long_laps, long_replays, short_laps, short_replays, global_laps, global_replays = [long_short_fr_indicies_analysis_results[k] for k in ['long_laps', 'long_replays', 'short_laps', 'short_replays', 'global_laps', 'global_replays']]


In [ ]:
jonathan_firing_rate_analysis_result.neuron_replay_stats_df

In [ ]:
long_exclusive.track_exclusive_df

In [ ]:
long_exclusive.track_exclusive_df['lap_delta_minus_inst_fr'] = LxC_ThetaDeltaMinus.cell_agg_inst_fr_list # (n_cells, )
long_exclusive.track_exclusive_df['lap_delta_plus_inst_fr'] = LxC_ThetaDeltaPlus.cell_agg_inst_fr_list # (n_cells, )
long_exclusive.track_exclusive_df


In [ ]:
# d = LxC_ThetaDeltaMinus.epoch_agg_inst_fr_list # (n_epochs, n_cells)
# d = d[:,1]
d = LxC_ThetaDeltaMinus.cell_agg_inst_fr_list # (n_cells, )
d
long_exclusive.track_exclusive_df['lap_delta_minus_inst_fr'] = LxC_ThetaDeltaMinus.cell_agg_inst_fr_list # (n_cells, )


In [ ]:
## Drop out the very low (inactive) Epochs... I guess? But we want it to be influenced by the inactive epochs.
curr_active_pipeline.	

In [ ]:

len(LxC_ThetaDeltaMinus.inst_fr_signals_list) # n_epochs
d = LxC_ThetaDeltaMinus.inst_fr_df_list
d[0]
# (n_epochs, n_cells)
# print(f'{d.shape}')

## NOW: 2023-07-11 - Testing Batch-computed inst_firing_rates

In [ ]:
from pyphoplacecellanalysis.General.Batch.runBatch import BatchSessionCompletionHandler
from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import PaperFigureTwo, InstantaneousSpikeRateGroupsComputation
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsResults, AcrossSessionsVisualizations


## Load the saved across-session results:
inst_fr_output_filename = 'long_short_inst_firing_rate_result_handlers_2023-07-12.pkl'
across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=global_data_root_parent_path, inst_fr_output_filename=inst_fr_output_filename)
# across_sessions_instantaneous_fr_dict = loadData(global_batch_result_inst_fr_file_path)
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')

In [ ]:
## Aggregate across all of the sessions to build a new combined `InstantaneousSpikeRateGroupsComputation`, which can be used to plot the "PaperFigureTwo", bar plots for many sessions.
global_multi_session_context = IdentifyingContext(format_name='kdiba', num_sessions=0) # some global context across all of the sessions, not sure what to put here.

# To correctly aggregate results across sessions, it only makes sense to combine entries at the `.cell_agg_inst_fr_list` variable and lower (as the number of cells can be added across sessions, treated as unique for each session).

## Display the aggregate across sessions:
_out_fig_2 = PaperFigureTwo(instantaneous_time_bin_size_seconds=0.01) # WARNING: we didn't save this info
_out_fig_2.computation_result = across_session_inst_fr_computation
_out_fig_2.active_identifying_session_ctx = across_session_inst_fr_computation.active_identifying_session_ctx
# Set callback, the only self-specific property
_out_fig_2._pipeline_file_callback_fn = curr_active_pipeline.output_figure # lambda args, kwargs: self.write_to_file(args, kwargs, curr_active_pipeline)

In [ ]:
# Showing
restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
# Perform interactive Matplotlib operations with 'Qt5Agg' backend
_fig_2_theta_out, _fig_2_replay_out = _out_fig_2.display(active_context=global_multi_session_context, title_modifier_fn=lambda original_title: f"{original_title} ({num_sessions} sessions)", save_figure=True)
	
_out_fig_2.perform_save()

# `active_pf_nD`, `active_pf_nD_dt` visualizations

In [ ]:
from pyphoplacecellanalysis.Pho2D.PyQtPlots.TimeSynchronizedPlotters.TimeSynchronizedOccupancyPlotter import TimeSynchronizedOccupancyPlotter



In [ ]:

curr_sync_occupancy_plotter = TimeSynchronizedOccupancyPlotter(global_pf2D_dt)
curr_sync_occupancy_plotter.show()

# 2023-07-07 - `batch_extended_programmatic_figures` Testing

In [ ]:
curr_active_pipeline.reload_default_display_functions()

neptuner = batch_perform_all_plots(curr_active_pipeline, enable_neptune=False)

In [ ]:
%pdb off
%load_ext viztracer
from viztracer import VizTracer
from pyphocorehelpers.print_helpers import get_now_day_str, get_now_time_str, get_now_time_precise_str

In [ ]:

with VizTracer(output_file=f"viztracer_{get_now_time_str()}-batch_extended_programmatic_figures.json", min_duration=200, tracer_entries=3000000, ignore_frozen=True) as tracer:
    batch_extended_programmatic_figures(curr_active_pipeline, write_vector_format=False, write_png=False, debug_print=False)

In [ ]:
batch_extended_programmatic_figures(curr_active_pipeline, write_vector_format=False, write_png=False, debug_print=False)

In [ ]:
np.sum(np.logical_or(neuron_replay_stats_df['is_refined_LxC'], neuron_replay_stats_df['is_refined_SxC']))

In [ ]:
JonathanFiringRateAnalysisResult = None

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult

jonathan_firing_rate_analysis_result.refine_exclusivity_by_inst_frs_index(long_short_fr_indicies_df, frs_index_inclusion_magnitude=0.5)


In [ ]:
jonathan_firing_rate_analysis_result

# 2023-09-26 - Interactive `napari` testing

In [ ]:
import napari
from pyphoplacecellanalysis.GUI.Napari.napari_helpers import napari_add_surprise_data_layers, napari_add_animal_position, napari_set_time_windw_index

# create the viewer and window
viewer = napari.Viewer()
# napari_add_surprise_data_layers(viewer, active_relative_entropy_results)
napari_add_animal_position(viewer=viewer, position_df=global_pf1D_dt.all_time_filtered_pos_df[['t','x','binned_x']], time_intervals=time_intervals)

In [ ]:
drop_below_threshold:float=0.0000001
active_time_dependent_placefields = deepcopy(global_pf1D_dt)
curr_ratemap = active_time_dependent_placefields.ratemap
images = curr_ratemap.tuning_curves.copy() # 1D: (82, 112), 2D:(43, 63, 63)
print(f'images.shape: {np.shape(images)}')
occupancy = curr_ratemap.occupancy
print(f'occupancy: {occupancy.shape}')
# Compute Images:
included_unit_indicies = np.arange(np.shape(images)[0]) # include all unless otherwise specified
nMapsToShow = len(included_unit_indicies)
print(f'nMapsToShow: {nMapsToShow}')
# Get single cell info:
# neuron_IDX = curr_included_unit_index
# cell_ID = active_time_dependent_placefields.ratemap.neuron_ids[neuron_IDX]
# curr_cell_identifier_string = f'Cell[{cell_ID}]'
# curr_plot_identifier_string = f'TimeSynchronizedPlacefieldsPlotter.{curr_cell_identifier_string}'

# Build the image item:
# Update the image:
# image = np.squeeze(images[a_linear_index,:,:])
image = np.squeeze(images)
# Pre-filter the data:
with np.errstate(divide='ignore', invalid='ignore'):
	image = np.array(image) / np.nanmax(image) # note scaling by maximum here!
	if drop_below_threshold is not None:
		image[:, np.where(occupancy < drop_below_threshold)] = np.nan # null out the occupancy

image.shape

In [ ]:
from pyphoplacecellanalysis.GUI.Napari.napari_helpers import napari_export_video_frames



In [ ]:
viewer.layers['short_long_rel_entr_curves_frames'].blending # 'additive'

# 2023-09-27 - Exporting surprise ndarray to a video file for visualization!

In [ ]:

# Add the animal's closest current position (binned) as a single colored pixel along the top of the array:
len(global_pf1D_dt.historical_snapshots)

global_pf1D_dt.historical_snapshots

In [ ]:
# global_pf1D_dt.complete_time_range_computation(

custom_pf1D_dt = deepcopy(global_pf1D_dt)
custom_pf1D_dt.reset()

In [ ]:
# _out = custom_pf1D_dt.complete_time_range_computation(global_session.t_start, global_session.t_stop, assign_results_to_member_variables=True, should_snapshot=True)

_out = custom_pf1D_dt.batch_snapshotting(

.complete_time_range_computation(global_session.t_start, global_session.t_stop, assign_results_to_member_variables=True, should_snapshot=True)


In [ ]:
global_session.t_start				   
global_session.t_stop

In [ ]:
import cv2
from pyphocorehelpers.plotting.video_output_helpers import save_array_as_video

colormap = cv2.COLORMAP_JET


array = active_relative_entropy_results['snapshot_occupancy_weighted_tuning_maps'].copy()
gray_frames = cv2.normalize(array, None, 255, 0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U) # same size as array

# color_array = cv2.applyColorMap(gray_frames, colormap)
# video_out_path = save_array_as_video(array=color_array, video_filename='output/videos/snapshot_occupancy_weighted_tuning_maps_color.avi', isColor=True)
video_out_path = save_array_as_video(array=gray_frames, video_filename='output/videos/snapshot_occupancy_weighted_tuning_maps_gray.avi', isColor=False)
print(f'video_out_path: {video_out_path}')
reveal_in_system_file_manager(video_out_path)

In [ ]:
from pyphoplacecellanalysis.SpecificResults.fourthYearPresentation import export_active_relative_entropy_results_videos

video_output_parent_path = export_active_relative_entropy_results_videos(active_relative_entropy_results, active_context=curr_active_pipeline.get_session_context())
reveal_in_system_file_manager(video_output_parent_path)

# 2023-09-26 - Interactive `Spike3DRasterWindowWidget` exploration`

In [27]:
from pyphoplacecellanalysis.GUI.Qt.SpikeRasterWindows.Spike3DRasterWindowWidget import Spike3DRasterWindowWidget

spike_raster_window: Spike3DRasterWindowWidget = Spike3DRasterWindowWidget.find_or_create_if_needed(curr_active_pipeline, included_neuron_ids=EITHER_subset.track_exclusive_aclus)
active_2d_plot = spike_raster_window.spike_raster_plt_2d # <pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster.Spike2DRaster at 0x196c7244280>
active_3d_plot = spike_raster_window.spike_raster_plt_3d # <pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster.Spike2DRaster at 0x196c7244280>


found 1 existing Spike3DRasterWindowWidget windows using TopLevelWindowHelper.all_widgets(...). Will use the most recent.


In [37]:
spike_raster_window.close()

GlobalConnectionManagerAccessingMixin_on_destroy()
removed object with key Spike2DRaster from drivers list.
removed object with key Spike2DRaster from drivable list.
Window closed


True

In [29]:
spike_raster_window.show()

In [33]:
EITHER_subset.track_exclusive_aclus.shape

(84,)

In [ ]:
XOR_subset.

In [38]:
active_2d_plot, active_3d_plot, spike_raster_window = curr_active_pipeline.plot._display_spike_rasters_pyqtplot_2D().values() # included_neuron_ids=EITHER_subset.track_exclusive_aclus

INFO:com.PhoHale.Spike3D.display.SpikeRasterBase:==========================================================================================
========== Module Logger INIT "com.PhoHale.Spike3D.display.SpikeRasterBase" ==============================
INFO:com.PhoHale.Spike3D.display.SpikeRasterBase:SpikeRasterBase.__init__(...)


build_module_logger(module_name="Spike3D.display.SpikeRasterBase"):
	 Module logger com.PhoHale.Spike3D.display.SpikeRasterBase has file logging enabled and will log to EXTERNAL\TESTING\Logging\debug_com.PhoHale.Spike3D.display.SpikeRasterBase.log


INFO:com.PhoHale.Spike3D.display.SpikeRasterBase:Spike2DRaster.setup()
INFO:com.PhoHale.Spike3D.display.SpikeRasterBase:Spike2DRaster.setup(): adding "visualization_raster_y_location" column to spikes_df...
INFO:com.PhoHale.Spike3D.display.SpikeRasterBase:	done.
DEBUG:com.PhoHale.Spike3D.display.SpikeRasterBase:self.spikes_df.columns: Index(['t', 't_seconds', 't_rel_seconds', 'shank', 'cluster', 'aclu', 'qclu', 'x', 'y', 'speed', 'traj', 'lap', 'maze_relative_lap', 'maze_id', 'is_theta', 'is_ripple', 'theta_phase_radians', 'neuron_type', 'flat_spike_idx', 'x_loaded', 'y_loaded', 'lin_pos', 'fragile_linear_neuron_IDX', 'PBE_id', 'scISI', 'neuron_IDX', 'visualization_raster_y_location'], dtype='object')
INFO:com.PhoHale.Spike3D.display.SpikeRasterBase:Spike2DRaster.setup(): adding "visualization_raster_emphasis_state" column to spikes_df...
INFO:com.PhoHale.Spike3D.display.SpikeRasterBase:	done.
INFO:com.PhoHale.Spike3D.display.SpikeRasterBase:SpikeRasterBase.buildUI()
DEBUG:com.PhoHale.

DEBUG:com.PhoHale.Spike3D.display.SpikeRasterBase:Spike2DRaster._update_plots()


_run_delayed_gui_load_code() called!


DEBUG:com.PhoHale.Spike3D.display.SpikeRasterBase:Spike2DRaster._update_plots()


In [39]:
# spike_raster_window.ui.actionAddTimeIntervals_Bursts.setEnabled(True)

from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.RenderTimeEpochs.Render2DEventRectanglesHelper import Render2DEventRectanglesHelper


active_burst_intervals = curr_active_pipeline.computation_results[global_epoch_name].computed_data['burst_detection']['burst_intervals'] # this works
output_display_items = Render2DEventRectanglesHelper.add_event_rectangles(spike_raster_window.spike_raster_plt_2d, active_burst_intervals) # {'interval_rects_item': active_interval_rects_item}
active_interval_rects_item = output_display_items['interval_rects_item']

DEBUG:com.PhoHale.Spike3D.display.SpikeRasterBase:Spike2DRaster._update_plots()
DEBUG:com.PhoHale.Spike3D.display.SpikeRasterBase:Spike2DRaster._update_plots()
DEBUG:com.PhoHale.Spike3D.display.SpikeRasterBase:Spike2DRaster._update_plots()
DEBUG:com.PhoHale.Spike3D.display.SpikeRasterBase:Spike2DRaster._update_plots()
DEBUG:com.PhoHale.Spike3D.display.SpikeRasterBase:Spike2DRaster._update_plots()
DEBUG:com.PhoHale.Spike3D.display.SpikeRasterBase:Spike2DRaster._update_plots()
DEBUG:com.PhoHale.Spike3D.display.SpikeRasterBase:Spike2DRaster._update_plots()
DEBUG:com.PhoHale.Spike3D.display.SpikeRasterBase:Spike2DRaster._update_plots()
DEBUG:com.PhoHale.Spike3D.display.SpikeRasterBase:Spike2DRaster._update_plots()
DEBUG:com.PhoHale.Spike3D.display.SpikeRasterBase:Spike2DRaster._update_plots()
DEBUG:com.PhoHale.Spike3D.display.SpikeRasterBase:Spike2DRaster._update_plots()
DEBUG:com.PhoHale.Spike3D.display.SpikeRasterBase:Spike2DRaster._update_plots()
DEBUG:com.PhoHale.Spike3D.display.SpikeR

keyPressEvent(e.key(): 16777237)


DEBUG:com.PhoHale.Spike3D.display.SpikeRasterBase:Spike2DRaster._update_plots()
DEBUG:com.PhoHale.Spike3D.display.SpikeRasterBase:Spike2DRaster._update_plots()
DEBUG:com.PhoHale.Spike3D.display.SpikeRasterBase:Spike2DRaster._update_plots()
DEBUG:com.PhoHale.Spike3D.display.SpikeRasterBase:Spike2DRaster._update_plots()


keyPressEvent(e.key(): 16777237)
keyPressEvent(e.key(): 16777239)
keyPressEvent(e.key(): 16777238)
keyPressEvent(e.key(): 16777238)
keyPressEvent(e.key(): 16777249)
keyPressEvent(e.key(): 16777238)
keyPressEvent(e.key(): 16777238)
keyPressEvent(e.key(): 16777249)
keyPressEvent(e.key(): 16777239)


In [ ]:
16777238 # Page Up
16777239 # Page Down


In [40]:
active_2d_plot.temporal_axis_length

15.0

In [ ]:
active_2d_plot.animation_active_time_window

In [45]:
time_window.active_time_window


(1132.5138243586016, 1147.5138243586016)

15.0

In [55]:
shift_frames = (active_2d_plot.temporal_axis_length/spike_raster_window.animation_time_step) # determine the integral number of animation_time_step frames to shift by one window length
shift_frames

375.0

In [52]:
# window_duration_sec = active_2d_plot.temporal_axis_length # I think this is better because it accounts for the scale factor
time_window = active_2d_plot.animation_active_time_window # SpikesDataframeWindow
window_duration_sec = time_window.window_duration
proposed_next_window_start_time = time_window.active_window_start_time + window_duration_sec
proposed_next_window_start_time

1147.5138243586016

In [56]:
spike_raster_window.update_animation(proposed_next_window_start_time)

DEBUG:com.PhoHale.Spike3D.display.SpikeRasterBase:Spike2DRaster._update_plots()


In [ ]:
# curr_active_pipeline.

EITHER_subset.track_exclusive_aclus
# , neuron_colors=neuron_colors, neuron_sort_order=neuron_sort_order, application_name=application_name


# Interactive SpikeRasterWindow

In [ ]:
from neuropy.analyses.time_dependent_placefields import PfND_TimeDependent, PlacefieldSnapshot

global_pf1D_dt: PfND_TimeDependent = global_results.pf1D_dt
global_pf2D_dt: PfND_TimeDependent = global_results.pf2D_dt


In [ ]:
from pyphoplacecellanalysis.Pho2D.PyQtPlots.TimeSynchronizedPlotters.TimeSynchronizedOccupancyPlotter import TimeSynchronizedOccupancyPlotter
from pyphoplacecellanalysis.Pho2D.PyQtPlots.TimeSynchronizedPlotters.TimeSynchronizedPlacefieldsPlotter import TimeSynchronizedPlacefieldsPlotter
from pyphoplacecellanalysis.Pho2D.PyQtPlots.TimeSynchronizedPlotters.Mixins.helpers import build_combined_time_synchronized_plotters_window

active_pf_2D_dt = global_pf2D_dt
active_pf_2D_dt.reset()
active_pf_2D_dt.update(t=500.0, start_relative_t=True)
all_plotters, root_dockAreaWindow, app = build_combined_time_synchronized_plotters_window(active_pf_2D_dt, fixed_window_duration = 15.0)
controlling_widget, curr_sync_occupancy_plotter, curr_placefields_plotter = all_plotters # end up in some sort of `linkedViewChanged` loop

In [ ]:
# Draw a VERY simple and efficient 1D plot of the animal on the track.


In [ ]:
# Create a new `SpikeRaster2D` instance using `_display_spike_raster_pyqtplot_2D` and capture its outputs:
_out = curr_active_pipeline.plot._display_spike_rasters_pyqtplot_2D()

In [ ]:
from pyphocorehelpers.gui.Qt.TopLevelWindowHelper import TopLevelWindowHelper
import pyphoplacecellanalysis.External.pyqtgraph as pg # Used to get the app for TopLevelWindowHelper.top_level_windows
## For searching with `TopLevelWindowHelper.all_widgets(...)`:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster import Spike2DRaster
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike3DRaster import Spike3DRaster
from pyphoplacecellanalysis.GUI.Qt.SpikeRasterWindows.Spike3DRasterWindowWidget import Spike3DRasterWindowWidget

found_spike_raster_windows = TopLevelWindowHelper.all_widgets(pg.mkQApp(), searchType=Spike3DRasterWindowWidget)
# assert len(found_spike_raster_windows) == 1, f"found {len(found_spike_raster_windows)} Spike3DRasterWindowWidget windows using TopLevelWindowHelper.all_widgets(...) but require exactly one."
if len(found_spike_raster_windows) > 1:
	print(f'found {len(found_spike_raster_windows)} widgets with the searchType provided:')
	print(f'{found_spike_raster_windows}')
	print(f'defaulting to using the most recent [-1]...')
spike_raster_window = found_spike_raster_windows[-1]

# Extras:
active_2d_plot = spike_raster_window.spike_raster_plt_2d # <pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster.Spike2DRaster at 0x196c7244280>
active_3d_plot = spike_raster_window.spike_raster_plt_3d # <pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster.Spike2DRaster at 0x196c7244280>

In [ ]:
from pyphoplacecellanalysis.General.Mixins.SpikesRenderingBaseMixin import SpikeEmphasisState

active_pf_2D = global_pf2D
## Example 1: De-emphasize spikes excluded from the placefield calculations:
is_spike_included_in_pf = np.isin(spike_raster_window.spike_raster_plt_2d.spikes_df.index, active_pf_2D.filtered_spikes_df.index)
spike_raster_window.spike_raster_plt_2d.update_spike_emphasis(np.logical_not(is_spike_included_in_pf), SpikeEmphasisState.Deemphasized)


In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.RenderTimeEpochs.EpochRenderingMixin import EpochRenderingMixin
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.RenderTimeEpochs.Specific2DRenderTimeEpochs import General2DRenderTimeEpochs, Ripples_2DRenderTimeEpochs, inline_mkColor

add_renderables_menu = active_2d_plot.ui.menus.custom_context_menus.add_renderables[0].programmatic_actions_dict
menu_commands = ['AddTimeIntervals.PBEs', 'AddTimeIntervals.Ripples', 'AddTimeIntervals.Replays', 'AddTimeIntervals.Laps', 'AddTimeIntervals.Session.Epochs']
for a_command in menu_commands:
    add_renderables_menu[a_command].trigger()



In [ ]:
## Inline Concise: Position Replays, PBEs, and Ripples all below the scatter:
# active_2d_plot.interval_datasources.Replays.update_visualization_properties(lambda active_df, **kwargs: General2DRenderTimeEpochs._update_df_visualization_columns(active_df, y_location=-10.0, height=7.5, pen_color=inline_mkColor('orange', 0.8), brush_color=inline_mkColor('orange', 0.5), **kwargs)) ## Fully inline
# active_2d_plot.interval_datasources.PBEs.update_visualization_properties(lambda active_df, **kwargs: General2DRenderTimeEpochs._update_df_visualization_columns(active_df, y_location=-2.0, height=1.5, pen_color=inline_mkColor('pink', 0.8), brush_color=inline_mkColor('pink', 0.5), **kwargs)) ## Fully inline
# active_2d_plot.interval_datasources.Ripples.update_visualization_properties(lambda active_df, **kwargs: General2DRenderTimeEpochs._update_df_visualization_columns(active_df, y_location=-12.0, height=1.5, pen_color=inline_mkColor('cyan', 0.8), brush_color=inline_mkColor('cyan', 0.5), **kwargs)) ## Fully inline
# active_2d_plot.interval_datasources.SessionEpochs .update_visualization_properties(lambda active_df, **kwargs: General2DRenderTimeEpochs._update_df_visualization_columns(active_df, y_location=-12.0, height=1.5, pen_color=inline_mkColor('cyan', 0.8), brush_color=inline_mkColor('cyan', 0.5), **kwargs)) ## Fully inline
epochs_update_dict = {
    'Replays':dict(y_location=-10.0, height=7.5, pen_color=inline_mkColor('orange', 0.8), brush_color=inline_mkColor('orange', 0.5)),
    'PBEs':dict(y_location=-2.0, height=1.5, pen_color=inline_mkColor('pink', 0.8), brush_color=inline_mkColor('pink', 0.5)),
    'Ripples':dict(y_location=-12.0, height=1.5, pen_color=inline_mkColor('cyan', 0.8), brush_color=inline_mkColor('cyan', 0.5)),
    'SessionEpochs ':dict(y_location=-12.0, height=1.5, pen_color=inline_mkColor('cyan', 0.8), brush_color=inline_mkColor('cyan', 0.5)),
}
active_2d_plot.update_rendered_intervals_visualization_properties(epochs_update_dict)

In [ ]:

interval_info = active_2d_plot.list_all_rendered_intervals() ## get the existing intervals
rendered_interval_keys = list(interval_info.keys())
desired_interval_height_ratios = [2.0, 2.0, 1.0, 0.1, 1.0, 1.0, 1.0] # ratio of heights to each interval
required_vertical_offsets, required_interval_heights = EpochRenderingMixin.build_stacked_epoch_layout(desired_interval_height_ratios, epoch_render_stack_height=20.0, interval_stack_location='below')
stacked_epoch_layout_dict = {interval_key:dict(y_location=y_location, height=height) for interval_key, y_location, height in zip(rendered_interval_keys, required_vertical_offsets, required_interval_heights)} # Build a stacked_epoch_layout_dict to update the display
active_2d_plot.update_rendered_intervals_visualization_properties(stacked_epoch_layout_dict)


In [ ]:
is_spike_included_in_pf

In [ ]:
active_pf_2D_dt.update(t=1000.0, start_relative_t=True)

In [ ]:
from pyphoplacecellanalysis.Pho2D.PyQtPlots.TimeSynchronizedPlotters.TimeSynchronizedPlacefieldsPlotter import TimeSynchronizedPlacefieldsPlotter

curr_sync_occupancy_plotter = TimeSynchronizedOccupancyPlotter(global_pf2D_dt)
curr_sync_occupancy_plotter.show()

In [ ]:
global_pf2D_dt.historical_snapshots

In [ ]:
from pyphoplacecellanalysis.Pho2D.PyQtPlots.TimeSynchronizedPlotters.TimeSynchronizedPlacefieldsPlotter import TimeSynchronizedPlacefieldsPlotter

curr_sync_placefield_plotter = TimeSynchronizedPlacefieldsPlotter(global_pf2D_dt)
curr_sync_placefield_plotter.show()

In [ ]:
from pyphoplacecellanalysis.Pho2D.PyQtPlots.TimeSynchronizedPlotters.TimeSynchronizedPlacefieldsPlotter import TimeSynchronizedPlacefieldsPlotter

curr_sync_placefield_plotter1D = TimeSynchronizedPlacefieldsPlotter(global_pf1D_dt)
curr_sync_placefield_plotter1D.show()

# 2023-07-19 - Validation with 3D tools

In [20]:
import pyvista as pv

display_output = {}
# active_config_name = long_epoch_name
active_config_name = global_epoch_name
active_config = curr_active_pipeline.active_configs[active_config_name]
active_config.plotting_config.should_use_linear_track_geometry = True # indicate that it's a linear track so the better geometry can be used

# 3D Plotters

## 📣 Programmatically adding several epoch rectangles by calling the addRenderable context menu functions all at once for SpikeRaster2D

In [ ]:
add_renderables_menu = active_2d_plot.ui.menus.custom_context_menus.add_renderables[0].programmatic_actions_dict
menu_commands = ['AddTimeIntervals.PBEs', 'AddTimeIntervals.Ripples', 'AddTimeIntervals.Replays', 'AddTimeIntervals.Laps', 'AddTimeIntervals.Session.Epochs']
for a_command in menu_commands:
    add_renderables_menu[a_command].trigger()

In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.RenderTimeEpochs.Specific2DRenderTimeEpochs import General2DRenderTimeEpochs, Ripples_2DRenderTimeEpochs, inline_mkColor
## Inline Concise: Position Replays, PBEs, and Ripples all below the scatter:
# active_2d_plot.interval_datasources.Replays.update_visualization_properties(lambda active_df, **kwargs: General2DRenderTimeEpochs._update_df_visualization_columns(active_df, y_location=-10.0, height=7.5, pen_color=inline_mkColor('orange', 0.8), brush_color=inline_mkColor('orange', 0.5), **kwargs)) ## Fully inline
# active_2d_plot.interval_datasources.PBEs.update_visualization_properties(lambda active_df, **kwargs: General2DRenderTimeEpochs._update_df_visualization_columns(active_df, y_location=-2.0, height=1.5, pen_color=inline_mkColor('pink', 0.8), brush_color=inline_mkColor('pink', 0.5), **kwargs)) ## Fully inline
# active_2d_plot.interval_datasources.Ripples.update_visualization_properties(lambda active_df, **kwargs: General2DRenderTimeEpochs._update_df_visualization_columns(active_df, y_location=-12.0, height=1.5, pen_color=inline_mkColor('cyan', 0.8), brush_color=inline_mkColor('cyan', 0.5), **kwargs)) ## Fully inline
# active_2d_plot.interval_datasources.SessionEpochs .update_visualization_properties(lambda active_df, **kwargs: General2DRenderTimeEpochs._update_df_visualization_columns(active_df, y_location=-12.0, height=1.5, pen_color=inline_mkColor('cyan', 0.8), brush_color=inline_mkColor('cyan', 0.5), **kwargs)) ## Fully inline
epochs_update_dict = {
    'Replays':dict(y_location=-10.0, height=7.5, pen_color=inline_mkColor('orange', 0.8), brush_color=inline_mkColor('orange', 0.5)),
    'PBEs':dict(y_location=-2.0, height=1.5, pen_color=inline_mkColor('pink', 0.8), brush_color=inline_mkColor('pink', 0.5)),
    'Ripples':dict(y_location=-12.0, height=1.5, pen_color=inline_mkColor('cyan', 0.8), brush_color=inline_mkColor('cyan', 0.5)),
    'SessionEpochs ':dict(y_location=-12.0, height=1.5, pen_color=inline_mkColor('cyan', 0.8), brush_color=inline_mkColor('cyan', 0.5)),
}
active_2d_plot.update_rendered_intervals_visualization_properties(epochs_update_dict)


In [ ]:
rendered_interval_keys = list(interval_info.keys())
desired_interval_height_ratios = [2.0, 2.0, 1.0, 0.1, 1.0, 1.0, 1.0] # ratio of heights to each interval
required_vertical_offsets, required_interval_heights = EpochRenderingMixin.build_stacked_epoch_layout(desired_interval_height_ratios, epoch_render_stack_height=20.0, interval_stack_location='below')
stacked_epoch_layout_dict = {interval_key:dict(y_location=y_location, height=height) for interval_key, y_location, height in zip(rendered_interval_keys, required_vertical_offsets, required_interval_heights)} # Build a stacked_epoch_layout_dict to update the display
active_2d_plot.update_rendered_intervals_visualization_properties(stacked_epoch_layout_dict)

## 🪟 ipcDataExplorer - 3D Interactive Tuning Curves Plotter

In [ ]:
curr_active_pipeline.registered_display_function_docs_dict

'_display_3d_image_plotter'
'_display_long_short_laps'
'_display_3d_interactive_custom_data_explorer'


In [ ]:
# from pyphoplacecellanalysis.GUI.PyVista.InteractivePlotter.InteractiveDataExplorerBase import InteractiveDataExplorerBase
# from pyphoplacecellanalysis.GUI.PyVista.InteractivePlotter.InteractiveCustomDataExplorer import InteractiveCustomDataExplorer # TypeError: super(type, obj): obj must be an instance or subtype of type

active_config_name = global_epoch_context
# curr_active_pipeline.reload_default_display_functions()
# _out = curr_active_pipeline.display('_display_3d_interactive_custom_data_explorer', long_epoch_name) # long_epoch_context
display_dict = curr_active_pipeline.display('_display_3d_interactive_custom_data_explorer', active_config_name) # does not work, missing color info?
iplapsDataExplorer = display_dict['iplapsDataExplorer']
# plotter is available at
p = display_dict['plotter']
iplapsDataExplorer


In [ ]:
import ipywidgets as widgets

# df = pd.DataFrame([curr_active_pipeline.registered_display_function_docs_dict])
# df = curr_active_pipeline.registered_display_function_docs_dict
df = pd.DataFrame.from_dict(curr_active_pipeline.registered_display_function_docs_dict, orient='index', columns=['Value'])
display_widget = widgets.Output()
with display_widget:
	display(df)
display_widget

In [ ]:
from neuropy.utils.matplotlib_helpers import plot_position_curves_figure

curr_active_pipeline.reload_default_display_functions()

In [ ]:
active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'
graphics_output_dict = curr_active_pipeline.display('_display_long_short_laps', active_identifying_session_ctx)

In [ ]:

# _out3 = curr_active_pipeline.display('_display_long_and_short_stacked_epoch_slices', included_epoch_indicies=None, save_figure=False)
example_stacked_epoch_graphics = curr_active_pipeline.display('_display_long_and_short_stacked_epoch_slices', defer_render=False, save_figure=False, included_epoch_indicies=selection_idxs_L[:50], enable_radon_transform_info=False, max_subplots_per_page=25)


In [ ]:
example_stacked_epoch_graphics.figures[0].show()

In [ ]:
pagination_controller_L, pagination_controller_S = example_stacked_epoch_graphics.plot_data['controllers']
ax_L, ax_S = example_stacked_epoch_graphics.axes
final_figure_context_L, final_context_S = example_stacked_epoch_graphics.context


In [ ]:
from pyphoplacecellanalysis.PhoPositionalData.plotting.laps import plot_lap_trajectories_3d
# ## single_combined_plot == True mode (mode 1.):
# p, laps_pages = plot_lap_trajectories_3d(curr_active_pipeline.sess, single_combined_plot=True)
# p.show()

## single_combined_plot == False mode (mode 2.):        
p, laps_pages = plot_lap_trajectories_3d(curr_active_pipeline.sess, single_combined_plot=False, curr_num_subplots=len(curr_active_pipeline.sess.laps.lap_id), active_page_index=0)
p.show()

In [ ]:
from pyphoplacecellanalysis.PhoPositionalData.plotting.laps import plot_lap_trajectories_2d

p_laps_2D, axs_laps_2D, laps_2D_pages = plot_lap_trajectories_2d(curr_active_pipeline.sess, curr_num_subplots=5, active_page_index=0)

In [ ]:
p_laps_2D.suptitle('plot_lap_trajectories_2d')


In [ ]:
ax = axs_laps_2D[0,0]
ax_empty = axs_laps_2D[-1,-1]

In [ ]:
long_track_position_offset = 1.0 * long_track_dims.compute_position_offset(grid_bin_bounds=grid_bin_bounds) # array([49.43, 140.61])
long_track_position_offset

In [ ]:
grid_bin_bounds_center_point = (point_tuple_mid_point(grid_bin_bounds[0]), point_tuple_mid_point(grid_bin_bounds[1])) # (145.43, 140.61)
print(f'grid_bin_bounds_center_point: {grid_bin_bounds_center_point}')
x_midpoint, y_midpoint = grid_bin_bounds_center_point

In [ ]:
user_annotation_man = UserAnnotationsManager()
user_annotations = user_annotation_man.annotations # .get_user_annotations()

allow_interactive_selection = False

final_context_L = curr_active_pipeline.build_display_context_for_session(display_fn_name='DecodedEpochSlices', epochs='replays', decoder='long_results_obj')
final_context_S = curr_active_pipeline.build_display_context_for_session(display_fn_name='DecodedEpochSlices', epochs='replays', decoder='short_results_obj')
# _out_pagination_controller.params.active_identifying_figure_ctx.adding_context(None,  user_annotation="selections")
selections_context_L = final_context_L.adding_context(None,  user_annotation="selections")
selections_context_S = final_context_S.adding_context(None,  user_annotation="selections")

## try to get the user annotations for this session:
try:
	selection_idxs_L = user_annotations[selections_context_L]
	selection_idxs_S = user_annotations[selections_context_S]
except KeyError as e:
	if allow_interactive_selection:
		print(f'user annotations <good replay selections> are not found. Creating them interactively...')
		user_annotations = interactive_good_epoch_selections(annotations_man=user_annotation_man, curr_active_pipeline=curr_active_pipeline)  # perform interactive selection. Should block here.
		selection_idxs_L = user_annotations[selections_context_L]
		selection_idxs_S = user_annotations[selections_context_S]
	else:
		print(f'interactive annotation is not permitted. Failing.')
		raise e
except Exception as e:
	print('Unhandled exception: {e}')
	raise


(selection_idxs_L, selection_idxs_S)
# # for updating the filter_epochs_df (`filter_epochs_df`) from the selections:
# self.filter_epochs_df['long_is_user_included'] = np.isin(self.filter_epochs_df.index, selection_idxs_L)
# self.filter_epochs_df['short_is_user_included'] = np.isin(self.filter_epochs_df.index, selection_idxs_S)


In [ ]:

from pyphoplacecellanalysis.Pho2D.stacked_epoch_slices import DecodedEpochSlicesPaginatedFigureController
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.DecoderPredictionError import plot_decoded_epoch_slices_paginated, plot_decoded_epoch_slices, _subfn_update_decoded_epoch_slices


## Stacked Epoch Plot
defer_render=False
save_figure=False
included_epoch_indicies=selection_idxs_L[:50]
enable_radon_transform_info=False
max_subplots_per_page=25
kwargs = {}

## long_short_decoding_analyses:
curr_long_short_decoding_analyses = curr_active_pipeline.global_computation_results.computed_data['long_short_leave_one_out_decoding_analysis']
## Extract variables from results object:
long_results_obj, short_results_obj = curr_long_short_decoding_analyses.long_results_obj, curr_long_short_decoding_analyses.short_results_obj
long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()

pagination_controller_L, active_out_figure_paths_L, final_context_L = plot_decoded_epoch_slices_paginated(curr_active_pipeline, long_results_obj, curr_active_pipeline.build_display_context_for_session(display_fn_name='DecodedEpochSlices', epochs='replays', decoder='long_results_obj'), included_epoch_indicies=included_epoch_indicies, save_figure=save_figure, **kwargs)
fig_L = pagination_controller_L.plots.fig
ax_L = fig_L.get_axes()
if defer_render:
	widget_L = pagination_controller_L.ui.mw # MatplotlibTimeSynchronizedWidget
	widget_L.close()
	pagination_controller_L = None



In [ ]:
def _subfn_update_decoded_epoch_slices(params, plots_data, plots, ui, debug_print=False):
    """ attempts to update existing plots created by:
    
       params, plots_data, plots, ui = stacked_epoch_slices_matplotlib_build_view(epoch_slices, epoch_labels=epoch_labels, name=name, plot_function_name=plot_function_name, debug_test_max_num_slices=debug_test_max_num_slices, debug_print=debug_print)

       Requires: `plots_data.filter_epochs_decoder_result`
    """
    
    for i, curr_ax in enumerate(plots.axs):
        curr_time_bin_container = plots_data.filter_epochs_decoder_result.time_bin_containers[i]
        curr_time_bins = curr_time_bin_container.centers
        curr_posterior_container = plots_data.filter_epochs_decoder_result.marginal_x_list[i]
        curr_posterior = curr_posterior_container.p_x_given_n
        curr_most_likely_positions = curr_posterior_container.most_likely_positions_1D
        
        params, plots_data, plots, ui = _helper_update_decoded_single_epoch_slice_plot(curr_ax, params, plots_data, plots, ui, i, curr_time_bins, curr_posterior, curr_most_likely_positions, debug_print=debug_print)
        on_render_page_callbacks = params.get('on_render_page_callbacks', {})
        for a_callback_name, a_callback in on_render_page_callbacks.items():
            try:
                params, plots_data, plots, ui = a_callback(curr_ax, params, plots_data, plots, ui, i, curr_time_bins, curr_posterior, curr_most_likely_positions, debug_print=debug_print)
            except Exception as e:
                print(f'\t encountered exception in callback: {e}')
                pass            

In [ ]:

pagination_controller_S, active_out_figure_paths_S, final_context_S = plot_decoded_epoch_slices_paginated(curr_active_pipeline, short_results_obj, curr_active_pipeline.build_display_context_for_session(display_fn_name='DecodedEpochSlices', epochs='replays', decoder='short_results_obj'), included_epoch_indicies=included_epoch_indicies, save_figure=save_figure, **kwargs)
fig_S = pagination_controller_S.plots.fig
ax_S = fig_S.get_axes()
if defer_render:
	widget_S = pagination_controller_S.ui.mw # MatplotlibTimeSynchronizedWidget
	widget_S.close()
	pagination_controller_S = None


In [ ]:
from copy import deepcopy
import numpy as np
from neuropy.utils.mixins.binning_helpers import transition_matrix

### 1D Transition Matrix:

def _compute_position_transition_matrix(xbin_labels, binned_x: np.ndarray, n_powers:int=3):
	"""  1D Transition Matrix from binned positions (e.g. 'binned_x')

		pf1D.xbin_labels # array([  1,   2,   3,   4,  ...)
		pf1D.filtered_pos_df['binned_x'].to_numpy() # array([116, 115, 115, ...,  93,  93,  93], dtype=int64)
	"""
	num_position_states = len(xbin_labels)
	# binned_x = pos_1D.to_numpy()
	binned_x_indicies = binned_x - 1
	binned_x_transition_matrix = transition_matrix(deepcopy(binned_x_indicies), markov_order=1, max_state_index=num_position_states)
	# binned_x_transition_matrix_higher_order_list = [binned_x_transition_matrix, transition_matrix(deepcopy(binned_x_indicies), markov_order=2, max_state_index=num_position_states), transition_matrix(deepcopy(binned_x_indicies), markov_order=3, max_state_index=num_position_states)]

	binned_x_transition_matrix[np.isnan(binned_x_transition_matrix)] = 0.0
	binned_x_transition_matrix_higher_order_list = [binned_x_transition_matrix] + [np.linalg.matrix_power(binned_x_transition_matrix, n) for n in np.arange(2, n_powers+1)]
	# , np.linalg.matrix_power(binned_x_transition_matrix, 2), np.linalg.matrix_power(binned_x_transition_matrix, 3)
	# binned_x_transition_matrix.shape # (64, 64)
	return binned_x_transition_matrix_higher_order_list

def _build_decoded_positions_transition_matrix(active_one_step_decoder):
	""" Compute the transition_matrix from the decoded positions 

	TODO: make sure that separate events (e.g. separate replays) are not truncated creating erronious transitions

	"""
	# active_time_window_variable = active_one_step_decoder.time_window_centers # get time window centers (n_time_window_centers,) # (4060,)
	# active_most_likely_positions = active_one_step_decoder.most_likely_positions.T # (4060, 2) NOTE: the most_likely_positions for the active_one_step_decoder are tranposed compared to the active_two_step_decoder
	# active_most_likely_positions = active_two_step_decoder.most_likely_positions # (2, 4060)
	active_one_step_decoder.most_likely_position_flat_indicies
	# active_most_likely_positions = active_one_step_decoder.revised_most_likely_positions.T
	# active_most_likely_positions #.shape # (36246,)

	most_likely_position_indicies = np.squeeze(np.array(np.unravel_index(active_one_step_decoder.most_likely_position_flat_indicies, active_one_step_decoder.original_position_data_shape))) # convert back to an array
	most_likely_position_xbins = most_likely_position_indicies + 1 # add 1 to convert back to a bin label from an index
	# most_likely_position_indicies # (1, 36246)

	xbin_labels = np.arange(active_one_step_decoder.original_position_data_shape[0]) + 1

	decoded_binned_x_transition_matrix_higher_order_list = _compute_position_transition_matrix(xbin_labels, most_likely_position_indicies)
	return decoded_binned_x_transition_matrix_higher_order_list, xbin_labels


# pf1D = deepcopy(curr_active_pipeline.computation_results['maze1'].computed_data['pf1D'])
pf1D = deepcopy(global_pf1D)
# pf1D = deepcopy(short_pf1D)
# pf1D = deepcopy(long_pf1D)
binned_x_transition_matrix_higher_order_list = _compute_position_transition_matrix(pf1D.xbin_labels, pf1D.filtered_pos_df['binned_x'].to_numpy())

In [ ]:
pf1D.filtered_pos_df.plot(x='t',y='x')

In [ ]:
# Visualization ______________________________________________________________________________________________________ #
from pyphoplacecellanalysis.GUI.PyQtPlot.BinnedImageRenderingWindow import BasicBinnedImageRenderingWindow, LayoutScrollability
out = BasicBinnedImageRenderingWindow(binned_x_transition_matrix_higher_order_list[0], pf1D.xbin_labels, pf1D.xbin_labels, name='binned_x_transition_matrix', title="Transition Matrix for binned x (from, to)", variable_label='Transition Matrix', scrollability_mode=LayoutScrollability.NON_SCROLLABLE)
# out.add_data(row=1, col=0, matrix=active_eloy_analysis.pf_overlapDensity_2D, xbins=active_pf_2D_dt.xbin_labels, ybins=active_pf_2D_dt.ybin_labels, name='pf_overlapDensity', title='pf overlapDensity metric', variable_label='pf overlapDensity')

# out.add_data(row=1, col=0, matrix=binned_x_transition_matrix_higher_order_list[1], xbins=pf1D.xbin_labels, ybins=pf1D.ybin_labels, name='binned_x_transition_matrix^2', title='2nd Order Transition Matrix for binned x (from, to)', variable_label='2nd Order Transition Matrix') # , scrollability_mode=LayoutScrollability.NON_SCROLLABLE
# out.add_data(row=2, col=0, matrix=binned_x_transition_matrix_higher_order_list[2], xbins=pf1D.xbin_labels, ybins=pf1D.ybin_labels, name='binned_x_transition_matrix^3', title='3rd Order Transition Matrix for binned x (from, to)', variable_label='3rd Order Transition Matrix') # , scrollability_mode=LayoutScrollability.NON_SCROLLABLE

# Separate Windows for each:
# out2 = BasicBinnedImageRenderingWindow(binned_x_transition_matrix_higher_order_list[1], pf1D.xbin_labels, pf1D.xbin_labels, name='binned_x_transition_matrix^2', title="2nd Order Transition Matrix for binned x (from, to)", variable_label='2nd Order Transition Matrix', scrollability_mode=LayoutScrollability.NON_SCROLLABLE)
# out3 = BasicBinnedImageRenderingWindow(binned_x_transition_matrix_higher_order_list[2], pf1D.xbin_labels, pf1D.xbin_labels, name='binned_x_transition_matrix^3', title="3rd Order Transition Matrix for binned x (from, to)", variable_label='3rd Order Transition Matrix', scrollability_mode=LayoutScrollability.NON_SCROLLABLE)

# 1. Compute the transition_matrix from the decoded positions


In [ ]:
# BayesianPlacemapPositionDecoder
# active_one_step_decoder = deepcopy(long_one_step_decoder_1D) # long_results_obj.original_1D_decoder
long_decoded_binned_x_transition_matrix_higher_order_list, long_xbin_labels = _build_decoded_positions_transition_matrix(active_one_step_decoder=deepcopy(long_one_step_decoder_1D))
short_decoded_binned_x_transition_matrix_higher_order_list, short_xbin_labels = _build_decoded_positions_transition_matrix(active_one_step_decoder=deepcopy(short_one_step_decoder_1D))


In [ ]:
out_decoded = BasicBinnedImageRenderingWindow(decoded_binned_x_transition_matrix_higher_order_list[0], active_one_step_decoder.xbin_centers, active_one_step_decoder.xbin_centers, name='decoded_binned_x_transition_matrix', title="DECODED Transition Matrix for binned x (from, to)", variable_label='Transition Matrix', scrollability_mode=LayoutScrollability.NON_SCROLLABLE)


In [ ]:
i = 0
out = BasicBinnedImageRenderingWindow(binned_x_transition_matrix_higher_order_list[i], pf1D.xbin_labels, pf1D.xbin_labels, name='binned_x_transition_matrix', title="Transition Matrix for binned x (from, to)", variable_label='Transition Matrix', scrollability_mode=LayoutScrollability.NON_SCROLLABLE)
out.add_data(row=1, col=0, matrix=long_decoded_binned_x_transition_matrix_higher_order_list[i], xbins=long_xbin_labels, ybins=long_xbin_labels, name='long_decoded_binned_x_transition_matrix', title='Long DECODED Transition Matrix', variable_label='long decoded')
out.add_data(row=2, col=0, matrix=short_decoded_binned_x_transition_matrix_higher_order_list[i], xbins=short_xbin_labels, ybins=short_xbin_labels, name='short_decoded_binned_x_transition_matrix', title='Short DECODED Transition Matrix', variable_label='short decoded')
# out.add_data(row=1, col=0, matrix=binned_x_transition_matrix_higher_order_list[1], xbins=pf1D.xbin_labels, ybins=pf1D.ybin_labels, name='binned_x_transition_matrix^2', title='2nd Order Transition Matrix for binned x (from, to)', variable_label='2nd Order Transition Matrix') # , scrollability_mode=LayoutScrollability.NON_SCROLLABLE
# out.add_data(row=2, col=0, matrix=binned_x_transition_matrix_higher_order_list[2], xbins=pf1D.xbin_labels, ybins=pf1D.ybin_labels, name='binned_x_transition_matrix^3', title='3rd Order Transition Matrix for binned x (from, to)', variable_label='3rd Order Transition Matrix') # , scrollability_mode=LayoutScrollability.NON_SCROLLABLE


In [ ]:
active_one_step_decoder.most_likely_position_indicies

In [ ]:

## 2. Use the position transition matrix to determine how likely each decoded position's transition is

In [ ]:
time_binned_instantaneous_unit_specific_spike_rate = curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.time_binned_instantaneous_unit_specific_spike_rate
timestamps = time_binned_instantaneous_unit_specific_spike_rate.time_bins

value_df = time_binned_instantaneous_unit_specific_spike_rate.instantaneous_unit_specific_spike_rate_values
value_df

In [ ]:
# Number of spikes version:
time_binned_unit_specific_spike_rate = curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.time_binned_unit_specific_spike_rate
timestamps = time_binned_unit_specific_spike_rate.time_bins
value_df = time_binned_unit_specific_spike_rate.time_binned_unit_specific_binned_spike_rate
value_df